# Recalcule Creance pour toute les entreprises Comparaison Calcule vs Non Calcule (2023)

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Charger les données
df = pd.read_excel("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//new_millesime_CIR_2023_corrected_sans_doublon.xlsx")
#df = pd.read_parquet("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//new_millesime_CIR_2023.parquet")

def convertir_en_nombre(valeur, defaut=0):
    """Convertit une valeur en nombre de façon sécurisée"""
    if pd.isna(valeur) or valeur == '' or valeur is None:
        return defaut
    try:
        return float(valeur)
    except:
        return defaut

def comparer_cir_declare_recalcule():
    """Compare le CIR déclaré et le CIR recalculé"""
    print(f"Nombre total d'entreprises: {len(df):,}")

    # Colonnes nécessaires pour recalculer le CIR
    colonnes = [
        'siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN',
        'MT_NET_DEP_RD', 'MT_NET_DEP_RD_DOM', 'MT_TOT_CIR_CI_COLL_CII',
        'MT_CIR_RECH_YC_QP_MOINS_DE100', 'MT_CI_COLL_APRS_MINIMI', 'MT_CII_YC_QP', 'MT_CRC',
        'DOT_AMORT_IMMO', 'DOT_AMORT_IMMO_SINISTR', 'DEP_CHERCH_TECH', 'REM_SAL_INV',
        'DEP_JD', 'OTR_DEP_FONCT', 'FRAIS_BREV_COV', 'DEP_MAINT_BREV_COV',
        'DOT_AMORT_BREV', 'DEP_NORMALI', 'PRIM_COTIZ', 'DEP_VEIL_TECHNO',
        'MT_DEP_FONCT_TOT', 'MT_TOT_RD_1',
        'DEP_EXT_LIE_FR', 'DEP_EXT_LIE_ETR', 'DEP_EXT_NON_LIE_FR', 'DEP_EXT_NON_LIE_ETR',
        'MT_TOT_DEP_EXT_ORG_AGREE', 'PLAF_OP_EXT', 'MT_TOT_OP_SOUS_TRAIT',
        'PLAF_OP_EXT_ORG_AGRE_LIE', 'PLAF_OP_EXT_ORG_AGRE_NON_LIE', 'PLAF_GNRL_DEP_EXT', 'MT_DEP_EXT_PLAF',
        'MT_TOT_RD_2',
        'MT_AID_SUBV', 'MT_ENC_PRESTA', 'MT_DEP_CONSEILS_CIR', 'REMBST_SUBV',
        'FRAIS_COLL', 'FRAIS_DEF_DESSIN', 'MT_TOT_DEP_COLL', 'MT_AID_SUBV_COLL',
        'MT_DEP_CONSEILS_CIR_COLL', 'REMBST_SUBV_COLL', 'MT_NET_DEP_COLL', 'MT_NET_DEP_COLL_DOM',
        'DOT_AMORT_IMMO_INO', 'DEP_PERSONEL_INO', 'FRAIS_BREV_COV_INO',
        'FRAIS_DEF_BREV_INO', 'OP_INOV_EXT', 'MT_TOT_DEP_INO', 'MT_TOT_DEP_INO_PLAF',
        'MT_AID_SUBV_INO', 'MT_ENC_PRESTA_INO', 'MT_DEP_CONSEILS_CII', 'REMBST_SUBV_INO',
        'MT_NET_DEP_INO', 'MT_NET_DEP_INO_DOM', 'MT_NET_DEP_INO_MPE_CORSE', 'MT_NET_DEP_INO_ME_CORSE',
        'DEP_CRC', 'DEP_CRC_FR', 'DEP_CRC_ETR', 'DEP_CRC_PLAF', 'AIDE_PUB_CRC',
        'AIDE_PUB_REMB_CRC', 'MT_NET_DEP_CRC', 'MT_NET_DEP_CRC_PME', 'MT_CRC','MT_QP_COLL_RECU_MOINS_DE100',
        'MT_QP_CIR_RECU_BIS','MT_QP_COLL_RECU_BIS','MT_AIDE_MINIMI_COLL','MT_QP_CII','QP_MT_CRC','MT_CII_CORSE',
        'MT_CI_COLL_APRS_MINIMI_DOM', 'MT_CI_COLL_APRES_MINIMI_PLUS_DE100_DOM', 'MT_QP_CIR_RECU', 'MT_AIDE_MINIMI_MOINS_DE100'
    ]

    # Convertir en numérique et créer une copie défragmentée
    df_tmp = df.copy()
    for col in colonnes:
        if col in df_tmp.columns:
            if col not in ['siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN']:
                df_tmp[col] = df_tmp[col].apply(convertir_en_nombre)
        else:
            if col not in ['siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN']:
                df_tmp[col] = 0


    # Créer un dictionnaire pour stocker toutes les colonnes calculées
    # et les ajouter en une seule fois à la fin pour éviter la fragmentation
    calc_columns = {}

    ## I - DÉPENSES DE RECHERCHE OUVRANT DROIT AU CRÉDIT D'IMPÔT (CIR-RECHERCHE) 
    ## ANNÉE CIVILE 2023

    # 1. Dépenses internes (Section I-A)

    # Ligne 6: Autres dépenses de fonctionnement
    calc_columns['LIGNE_6_CALC'] = (df_tmp['DOT_AMORT_IMMO'] * 0.75) + \
                               ((df_tmp['DEP_CHERCH_TECH'] + df_tmp['REM_SAL_INV']) * 0.43) + \
                               df_tmp['DEP_JD']

    # Ligne 7: Total dépenses de fonctionnement
    calc_columns['LIGNE_7_CALC'] = df_tmp['DOT_AMORT_IMMO'] + df_tmp['DOT_AMORT_IMMO_SINISTR'] + \
                               df_tmp['DEP_CHERCH_TECH'] + df_tmp['REM_SAL_INV'] + \
                               df_tmp['DEP_JD'] + calc_columns['LIGNE_6_CALC']


    # Appliquer les plafonds pour les lignes concernées
    # Ligne 11: Dépenses liées à la normalisation, utiliser directement la valeur renseignée
    calc_columns['DEP_NORMALI_RECALC'] = df_tmp['DEP_NORMALI']  # Conserver la valeur telle quelle

    # Ligne 12: Primes et cotisations (plafond 60 000 €)
    calc_columns['PRIM_COTIZ_PLAFONNEES'] = np.minimum(df_tmp['PRIM_COTIZ'], 60000)

    # Ligne 13: Dépenses de veille technologique (plafond 60 000 €)
    calc_columns['DEP_VEIL_TECHNO_PLAFONNEES'] = np.minimum(df_tmp['DEP_VEIL_TECHNO'], 60000)


    # Ligne 14: Total dépenses internes avec plafonds appliqués
    calc_columns['LIGNE_14_CALC'] = calc_columns['LIGNE_7_CALC'] + df_tmp['FRAIS_BREV_COV'] + \
                                df_tmp['DEP_MAINT_BREV_COV'] + df_tmp['DOT_AMORT_BREV'] + \
                                calc_columns['DEP_NORMALI_RECALC'] + calc_columns['PRIM_COTIZ_PLAFONNEES'] + \
                                calc_columns['DEP_VEIL_TECHNO_PLAFONNEES']

    # 2. Dépenses externalisées (Section I-B)

    # Ligne 17: Total dépenses externalisées
    calc_columns['LIGNE_17_CALC'] = df_tmp['DEP_EXT_LIE_FR'] + df_tmp['DEP_EXT_LIE_ETR'] + \
                                df_tmp['DEP_EXT_NON_LIE_FR'] + df_tmp['DEP_EXT_NON_LIE_ETR']

    # Préparation pour les calculs complexes nécessitant des références à d'autres colonnes calculées
    # Ajoutons temporairement les colonnes calculées au DataFrame
    df_calc = pd.DataFrame(calc_columns)
    df_calc_temp = pd.concat([df_tmp, df_calc], axis=1)

    # Appliquer les plafonnements
    # Ligne 18: Plafond global (3x dépenses internes)
    calc_columns['LIGNE_18_CALC'] = np.minimum(df_calc_temp['LIGNE_17_CALC'], df_calc_temp['LIGNE_14_CALC'] * 3)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_18_CALC'] = calc_columns['LIGNE_18_CALC']

    # Pour les calculs complexes, utiliser des boucles au lieu de apply
    # Ligne 19: Plafond pour organismes liés (2M€)
    ligne_19_values = []
    for _, row in df_calc_temp.iterrows():
        dep_liees = row['DEP_EXT_LIE_FR'] + row['DEP_EXT_LIE_ETR']
        plafond = min(row['LIGNE_18_CALC'], 2000000)
        ligne_19_values.append(min(dep_liees, plafond))
    calc_columns['LIGNE_19_CALC'] = np.array(ligne_19_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_19_CALC'] = calc_columns['LIGNE_19_CALC']

    # Ligne 20: Plafond pour organismes non liés (10M€)
    ligne_20_values = []
    for _, row in df_calc_temp.iterrows():
        dep_non_liees = row['DEP_EXT_NON_LIE_FR'] + row['DEP_EXT_NON_LIE_ETR']
        plafond_restant = min(row['LIGNE_18_CALC'] - row['LIGNE_19_CALC'], 10000000)
        ligne_20_values.append(min(dep_non_liees, plafond_restant))
    calc_columns['LIGNE_20_CALC'] = np.array(ligne_20_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_20_CALC'] = calc_columns['LIGNE_20_CALC']

    # Ligne 21: Total après plafonnements
    ligne_21_values = []
    for _, row in df_calc_temp.iterrows():
        ligne_21_values.append(min(row['LIGNE_19_CALC'] + row['LIGNE_20_CALC'], 10000000))
    calc_columns['LIGNE_21_CALC'] = np.array(ligne_21_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_21_CALC'] = calc_columns['LIGNE_21_CALC']

    # 3. Montant total et net (Section I-C)

    # Ligne 22: Total dépenses
    calc_columns['LIGNE_22_CALC'] = df_calc_temp['LIGNE_14_CALC'] + df_calc_temp['LIGNE_21_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_22_CALC'] = calc_columns['LIGNE_22_CALC']

    # Ligne 26a: Montant net
    calc_columns['LIGNE_26A_CALC'] = df_calc_temp['LIGNE_22_CALC'] - df_tmp['MT_AID_SUBV'] - \
                                  df_tmp['MT_ENC_PRESTA'] - df_tmp['MT_DEP_CONSEILS_CIR'] + \
                                  df_tmp['REMBST_SUBV']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_26A_CALC'] = calc_columns['LIGNE_26A_CALC']

    # Ligne 26b: Utiliser la valeur de MT_NET_DEP_RD_DOM directement
    calc_columns['LIGNE_26B_CALC'] = df_tmp['MT_NET_DEP_RD_DOM']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_26B_CALC'] = calc_columns['LIGNE_26B_CALC']

    ## II - DÉPENSES DE COLLECTION OUVRANT DROIT AU CRÉDIT D'IMPÔT (CIR-COLLECTION)
    ## ANNÉE CIVILE 2023

    # Ligne 28: Frais de défense des dessins et modèles (plafond 60 000 €)
    calc_columns['FRAIS_DEF_DESSIN_PLAFONNES'] = np.minimum(df_tmp['FRAIS_DEF_DESSIN'], 60000)

    # Ligne 29: Total des dépenses de collection
    calc_columns['LIGNE_29_CALC'] = df_tmp['FRAIS_COLL'] + calc_columns['FRAIS_DEF_DESSIN_PLAFONNES']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_29_CALC'] = calc_columns['LIGNE_29_CALC']

    # Ligne 33a: Montant net des dépenses de collection
    calc_columns['LIGNE_33A_CALC'] = df_calc_temp['LIGNE_29_CALC'] - df_tmp['MT_AID_SUBV_COLL'] - \
                                  df_tmp['MT_DEP_CONSEILS_CIR_COLL'] + df_tmp['REMBST_SUBV_COLL']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_33A_CALC'] = calc_columns['LIGNE_33A_CALC']

    # Ligne 33b: Utiliser la valeur de MT_NET_DEP_COLL_DOM directement
    calc_columns['LIGNE_33B_CALC'] = df_tmp['MT_NET_DEP_COLL_DOM']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_33B_CALC'] = calc_columns['LIGNE_33B_CALC']

    # Ligne 34a: Montant net total des dépenses de recherche et de collection
    calc_columns['LIGNE_34A_CALC'] = df_calc_temp['LIGNE_26A_CALC'] + df_calc_temp['LIGNE_33A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_34A_CALC'] = calc_columns['LIGNE_34A_CALC']

    # Ligne 34b: Montant net total des dépenses de recherche et de collection DOM
    calc_columns['LIGNE_34B_CALC'] = df_calc_temp['LIGNE_26B_CALC'] + df_calc_temp['LIGNE_33B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_34B_CALC'] = calc_columns['LIGNE_34B_CALC']

    ## V - DÉPENSES DE RECHERCHE COLLABORATIVE (CRC)
    ## ANNÉE CIVILE 2023

    # Vérifier si données disponibles pour la recherche collaborative
    if all(col in df_tmp.columns for col in ['DEP_CRC', 'DEP_CRC_FR', 'DEP_CRC_ETR']):
        # Dépenses éligibles
        calc_columns['LIGNE_83_CALC'] = df_tmp['DEP_CRC_FR'] + df_tmp['DEP_CRC_ETR']

        # Plafonnement à 6 000 000 €
        calc_columns['LIGNE_84_CALC'] = np.minimum(calc_columns['LIGNE_83_CALC'], 6000000)

        # Montant net des dépenses de recherche collaborative
        calc_columns['MT_NET_DEP_CRC_CALC'] = calc_columns['LIGNE_84_CALC'] - df_tmp['AIDE_PUB_CRC'] + \
                                          df_tmp['AIDE_PUB_REMB_CRC']

        # CRC selon les taux
        calc_columns['LIGNE_89_CALC'] = ((calc_columns['MT_NET_DEP_CRC_CALC'] - df_tmp['MT_NET_DEP_CRC_PME']) * 0.4) + \
                                     (df_tmp['MT_NET_DEP_CRC_PME'] * 0.5)

        calc_columns['CIR_COLLAB_RECALCULE'] = calc_columns['LIGNE_89_CALC']
        calc_columns['LIGNE_86'] = calc_columns['MT_NET_DEP_CRC_CALC']  # Changé de LIGNE_87 à LIGNE_86 pour 2023
    else:
        # Si données non disponibles, mettre à 0
        calc_columns['CIR_COLLAB_RECALCULE'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_86'] = np.zeros(len(df_tmp))  # Changé de LIGNE_87 à LIGNE_86 pour 2023
        calc_columns['LIGNE_89_CALC'] = np.zeros(len(df_tmp))

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_COLLAB_RECALCULE', 'LIGNE_86', 'LIGNE_89_CALC']:  # Changé de LIGNE_87 à LIGNE_86 pour 2023
        if col in calc_columns:
            df_calc_temp[col] = calc_columns[col]

    ## III - CALCUL DU CRÉDIT D'IMPÔT AU TITRE DES DÉPENSES DE RECHERCHE ET DE COLLECTION

    # Identifier les entreprises <= 100M€ en utilisant la somme ligne 34a + ligne 86
    calc_columns['DEPENSES_MOINS_100M'] = (df_calc_temp['LIGNE_34A_CALC'] + df_calc_temp['LIGNE_86']) <= 100000000  # Changé de LIGNE_87 à LIGNE_86 pour 2023

    # Mise à jour du DataFrame temporaire
    df_calc_temp['DEPENSES_MOINS_100M'] = calc_columns['DEPENSES_MOINS_100M']

    # A. Dépenses <= 100 000 000 €

    # CIR Recherche pour entreprises <= 100M€
    ligne_36_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_36_values.append((row['LIGNE_26A_CALC'] - row['LIGNE_26B_CALC']) * 0.3 +
                      row['LIGNE_26B_CALC'] * 0.5)
        else:
            ligne_36_values.append(0)
    calc_columns['LIGNE_36_CALC'] = np.array(ligne_36_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_36_CALC'] = calc_columns['LIGNE_36_CALC']

    # Ligne 37: Quote-part de crédit d'impôt résultant de la participation sociétés de personnes
    calc_columns['LIGNE_37'] = df_tmp['MT_QP_CIR_RECU']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_37'] = calc_columns['LIGNE_37']

    # Calcul ligne 38a: Montant du crédit d'impôt pour dépenses de recherche
    calc_columns['LIGNE_38A_CALC'] = df_calc_temp['LIGNE_36_CALC'] + 0 #df_calc_temp['LIGNE_37'] quote-part à 0 en 2023

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_38A_CALC'] = calc_columns['LIGNE_38A_CALC']

    # Calcul ligne 38b: Montant du CIR pour dépenses de recherche DOM
    calc_columns['LIGNE_38B_CALC'] = df_calc_temp['LIGNE_26B_CALC'] * 0.5  # Taux de 50% pour DOM

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_38B_CALC'] = calc_columns['LIGNE_38B_CALC']

    # CIR Collection pour entreprises <= 100M€ (avant plafond de minimis)
    ligne_40_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_40_values.append((row['LIGNE_33A_CALC'] - row['LIGNE_33B_CALC']) * 0.3 +
                      row['LIGNE_33B_CALC'] * 0.5)
        else:
            ligne_40_values.append(0)
    calc_columns['LIGNE_40_CALC'] = np.array(ligne_40_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_40_CALC'] = calc_columns['LIGNE_40_CALC']

    # Ligne 41: Quote-part de crédit d'impôt collection résultant de la participation sociétés de personnes
    calc_columns['LIGNE_41'] = df_tmp['MT_QP_COLL_RECU_MOINS_DE100']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_41'] = calc_columns['LIGNE_41']

    # Calcul ligne 42a: Montant total du crédit d'impôt pour dépenses de collection avant plafonnement
    calc_columns['LIGNE_42A_CALC'] = df_calc_temp['LIGNE_40_CALC'] + 0 #df_calc_temp['LIGNE_41']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_42A_CALC'] = calc_columns['LIGNE_42A_CALC']

    # Calcul ligne 42b: Montant du crédit collection DOM avant plafonnement
    calc_columns['LIGNE_42B_CALC'] = df_calc_temp['LIGNE_33B_CALC'] * 0.5  # Taux de 50% pour DOM

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_42B_CALC'] = calc_columns['LIGNE_42B_CALC']

    # Ligne 43: Montant des aides de minimis
    calc_columns['LIGNE_43'] = df_tmp['MT_AIDE_MINIMI_MOINS_DE100']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_43'] = calc_columns['LIGNE_43']

    # Calcul ligne 44: Montant cumulé du crédit d'impôt et des aides de minimis
    calc_columns['LIGNE_44'] = df_calc_temp['LIGNE_42A_CALC'] + df_calc_temp['LIGNE_43']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_44'] = calc_columns['LIGNE_44']

    # Calcul ligne 45a: Montant du crédit d'impôt pour dépenses de collection après plafonnement
    ligne_45a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['LIGNE_43'] >= 200000:  # Si aides de minimis déjà à 200k€
            ligne_45a_values.append(0)
        elif row['LIGNE_44'] < 200000:  # Si cumul < 200k€
            ligne_45a_values.append(row['LIGNE_42A_CALC'])
        else:  # Si dépassement
            ligne_45a_values.append(200000 - row['LIGNE_43'])
    calc_columns['LIGNE_45A_CALC'] = np.array(ligne_45a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_45A_CALC'] = calc_columns['LIGNE_45A_CALC']

    # ligne 45b: Montant du crédit collection DOM après plafonnement
    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_45B_CALC'] = df_tmp['MT_CI_COLL_APRS_MINIMI_DOM']

    # Calcul ligne 46a: Montant total du crédit d'impôt au titre des dépenses de recherche et de collection
    calc_columns['LIGNE_46A_CALC'] = df_calc_temp['LIGNE_38A_CALC'] + df_calc_temp['LIGNE_45A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_46A_CALC'] = calc_columns['LIGNE_46A_CALC']

    # Calcul ligne 46b: Montant du crédit d'impôt recherche et collection exposées dans DOM
    calc_columns['LIGNE_46B_CALC'] = df_calc_temp['LIGNE_38B_CALC'] + df_calc_temp['LIGNE_45B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_46B_CALC'] = calc_columns['LIGNE_46B_CALC']

    # B. Dépenses > 100 000 000 €

    # Identifier les entreprises > 100M€
    calc_columns['DEPENSES_PLUS_100M'] = ~df_calc_temp['DEPENSES_MOINS_100M']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['DEPENSES_PLUS_100M'] = calc_columns['DEPENSES_PLUS_100M']

    # Limiter les dépenses à 100M€ - dépenses recherche collaborative
    ligne_47a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_47a_values.append(min(row['LIGNE_26A_CALC'], 100000000 - row['LIGNE_86']))  # Changé de LIGNE_87 à LIGNE_86 pour 2023
        else:
            ligne_47a_values.append(0)
    calc_columns['LIGNE_47A_CALC'] = np.array(ligne_47a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_47A_CALC'] = calc_columns['LIGNE_47A_CALC']

    # Proportion DOM dans la limite des 100M€ (ligne 47b)
    ligne_47b_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_47b_values.append(min(row['LIGNE_26B_CALC'], 100000000 - row['LIGNE_86']))  # Changé de LIGNE_87 à LIGNE_86 pour 2023
        else:
            ligne_47b_values.append(0)
    calc_columns['LIGNE_47B_CALC'] = np.array(ligne_47b_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_47B_CALC'] = calc_columns['LIGNE_47B_CALC']

    # Calcul CIR recherche première tranche (ligne 48)
    ligne_48_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_48_values.append((row['LIGNE_47A_CALC'] - row['LIGNE_47B_CALC']) * 0.3 +
                           row['LIGNE_47B_CALC'] * 0.5)
        else:
            ligne_48_values.append(0)
    calc_columns['LIGNE_48_CALC'] = np.array(ligne_48_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_48_CALC'] = calc_columns['LIGNE_48_CALC']

    # Dépenses > 100M€ (ligne 49)
    ligne_49_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_49_values.append(max(0, row['LIGNE_26A_CALC'] - (100000000 - row['LIGNE_86'])))  # Changé de LIGNE_87 à LIGNE_86 pour 2023
        else:
            ligne_49_values.append(0)
    calc_columns['LIGNE_49_CALC'] = np.array(ligne_49_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_49_CALC'] = calc_columns['LIGNE_49_CALC']

    # CIR recherche deuxième tranche (au-delà de 100M€) (ligne 50)
    calc_columns['LIGNE_50_CALC'] = df_calc_temp['LIGNE_49_CALC'] * 0.05

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_50_CALC'] = calc_columns['LIGNE_50_CALC']

    # CIR recherche total pour entreprises > 100M€ (ligne 51)
    calc_columns['LIGNE_51_CALC'] = df_calc_temp['LIGNE_48_CALC'] + df_calc_temp['LIGNE_50_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_51_CALC'] = calc_columns['LIGNE_51_CALC']

    # Ligne 52: Quote-part de crédit d'impôt recherche résultant de la participation sociétés de personnes
    calc_columns['LIGNE_52'] = df_tmp['MT_QP_CIR_RECU_BIS']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_52'] = calc_columns['LIGNE_52']

    # Calcul ligne 53a: Montant du crédit d'impôt pour dépenses de recherche >100M€
    calc_columns['LIGNE_53A_CALC'] = df_calc_temp['LIGNE_51_CALC'] + 0 #df_calc_temp['LIGNE_52'] quote-part à 0 en 2023

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_53A_CALC'] = calc_columns['LIGNE_53A_CALC']

    # Calcul ligne 53b: Montant du crédit d'impôt pour dépenses de recherche DOM >100M€
    # Comme pour 38b, c'est 50% des dépenses DOM dans la limite du plafond
    calc_columns['LIGNE_53B_CALC'] = df_calc_temp['LIGNE_47B_CALC'] * 0.5

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_53B_CALC'] = calc_columns['LIGNE_53B_CALC']

    # Calcul CIR collection pour entreprises > 100M€

    # Ligne 54a: Montant net total des dépenses de collection
    calc_columns['LIGNE_54A_CALC'] = df_calc_temp['LIGNE_33A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_54A_CALC'] = calc_columns['LIGNE_54A_CALC']

    # Ligne 54b: Montant net total des dépenses de collection DOM
    calc_columns['LIGNE_54B_CALC'] = df_calc_temp['LIGNE_33B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_54B_CALC'] = calc_columns['LIGNE_54B_CALC']

    # Plafond disponible après dépenses recherche (ligne 55)
    ligne_55_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_55_values.append(max(0, 100000000 - row['LIGNE_86'] - row['LIGNE_47A_CALC']))  # Changé de LIGNE_87 à LIGNE_86 pour 2023
        else:
            ligne_55_values.append(0)
    calc_columns['LIGNE_55_CALC'] = np.array(ligne_55_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_55_CALC'] = calc_columns['LIGNE_55_CALC']

    # CIR collection première tranche (ligne 56)
    ligne_56_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            dep_coll_plaf = min(row['LIGNE_54A_CALC'], row['LIGNE_55_CALC'])
            dep_coll_dom_plaf = min(row['LIGNE_54B_CALC'], row['LIGNE_55_CALC'])
            ligne_56_values.append((dep_coll_plaf - dep_coll_dom_plaf) * 0.3 + dep_coll_dom_plaf * 0.5)
        else:
            ligne_56_values.append(0)
    calc_columns['LIGNE_56_CALC'] = np.array(ligne_56_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_56_CALC'] = calc_columns['LIGNE_56_CALC']

    # CIR collection deuxième tranche (ligne 57)
    ligne_57_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_57_values.append(max(0, row['LIGNE_54A_CALC'] - row['LIGNE_55_CALC']) * 0.05)
        else:
            ligne_57_values.append(0)
    calc_columns['LIGNE_57_CALC'] = np.array(ligne_57_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_57_CALC'] = calc_columns['LIGNE_57_CALC']

    # CIR collection avant plafonnement de minimis (ligne 58)
    calc_columns['LIGNE_58_CALC'] = df_calc_temp['LIGNE_56_CALC'] + df_calc_temp['LIGNE_57_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_58_CALC'] = calc_columns['LIGNE_58_CALC']

    # Ligne 59: Quote-part de crédit d'impôt collection résultant de la participation sociétés de personnes
    calc_columns['LIGNE_59'] = df_tmp['MT_QP_COLL_RECU_BIS']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_59'] = calc_columns['LIGNE_59']

    # Ligne 60: Montant du crédit d'impôt collection avant plafonnement de minimis
    calc_columns['LIGNE_60'] = df_calc_temp['LIGNE_58_CALC'] + 0 #df_calc_temp['LIGNE_59'] quote-part à 0 en 2023

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_60'] = calc_columns['LIGNE_60']

    # Ligne 61: Montant des aides de minimis
    calc_columns['LIGNE_61'] = df_tmp['MT_AIDE_MINIMI_COLL']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_61'] = calc_columns['LIGNE_61']

    # Ligne 62: Montant cumulé du crédit d'impôt et des aides de minimis
    calc_columns['LIGNE_62'] = df_calc_temp['LIGNE_60'] + df_calc_temp['LIGNE_61']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_62'] = calc_columns['LIGNE_62']

    # Ligne 63a: Montant du crédit d'impôt pour dépenses de collection après plafonnement
    ligne_63a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['LIGNE_61'] >= 200000:  # Si aides de minimis déjà à 200k€
            ligne_63a_values.append(0)
        elif row['LIGNE_62'] < 200000:  # Si cumul < 200k€
            ligne_63a_values.append(row['LIGNE_60'])
        else:  # Si dépassement
            ligne_63a_values.append(200000 - row['LIGNE_61'])
    calc_columns['LIGNE_63A_CALC'] = np.array(ligne_63a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_63A_CALC'] = calc_columns['LIGNE_63A_CALC']

    # Ligne 63b: Montant du crédit d'impôt pour dépenses de collection DOM après plafonnement
    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_63B_CALC'] = df_tmp['MT_CI_COLL_APRES_MINIMI_PLUS_DE100_DOM']

    # Ligne 64a: Montant total du crédit d'impôt au titre des dépenses de recherche et de collection
    calc_columns['LIGNE_64A_CALC'] = df_calc_temp['LIGNE_53A_CALC'] + df_calc_temp['LIGNE_63A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_64A_CALC'] = calc_columns['LIGNE_64A_CALC']

    # Ligne 64b: Montant du crédit d'impôt recherche et collection exposées dans DOM
    calc_columns['LIGNE_64B_CALC'] = df_calc_temp['LIGNE_53B_CALC'] + df_calc_temp['LIGNE_63B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_64B_CALC'] = calc_columns['LIGNE_64B_CALC']

    # Somme du CIR recherche selon le cas (<=100M€ ou >100M€)
    cir_recherche_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            cir_recherche_values.append(row['LIGNE_38A_CALC'])
        else:
            cir_recherche_values.append(row['LIGNE_53A_CALC'])
    calc_columns['CIR_RECHERCHE_RECALCULE'] = np.array(cir_recherche_values)

    # Somme du CIR collection après plafonnement de minimis selon le cas
    cir_collection_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            cir_collection_values.append(row['LIGNE_45A_CALC'])
        else:
            cir_collection_values.append(row['LIGNE_63A_CALC'])
    calc_columns['CIR_COLLECTION_RECALCULE'] = np.array(cir_collection_values)

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_RECHERCHE_RECALCULE', 'CIR_COLLECTION_RECALCULE']:
        df_calc_temp[col] = calc_columns[col]

    ## IV - DÉPENSES D'INNOVATION (CIR-INNOVATION)
    ## ANNÉE CIVILE 2023
    
    # Vérifier si les données d'innovation sont disponibles
    if all(col in df_tmp.columns for col in ['DOT_AMORT_IMMO_INO', 'DEP_PERSONEL_INO']):
        # Total dépenses d'innovation
        # Modification par rapport à 2022 : pas de calcul des autres dépenses de fonctionnement (ligne 67 en 2022)
        # En 2023, les dépenses sont directement déclarées
        calc_columns['LIGNE_71_CALC'] = df_tmp['DOT_AMORT_IMMO_INO'] + df_tmp['DEP_PERSONEL_INO'] + \
                                     df_tmp['FRAIS_BREV_COV_INO'] + df_tmp['FRAIS_DEF_BREV_INO'] + \
                                     df_tmp['OP_INOV_EXT']

        # Plafonnement à 400 000 €
        calc_columns['LIGNE_72_CALC'] = np.minimum(calc_columns['LIGNE_71_CALC'], 400000)

        # Montant net des dépenses d'innovation
        calc_columns['LIGNE_77A_CALC'] = calc_columns['LIGNE_72_CALC'] - df_tmp['MT_AID_SUBV_INO'] - \
                                       df_tmp['MT_ENC_PRESTA_INO'] - df_tmp['MT_DEP_CONSEILS_CII'] + \
                                       df_tmp['REMBST_SUBV_INO']

        # Calcul des parts DOM et Corse
        calc_columns['LIGNE_77B_CALC'] = df_tmp['MT_NET_DEP_INO_DOM']
        calc_columns['LIGNE_77C_CALC'] = df_tmp['MT_NET_DEP_INO_MPE_CORSE']
        calc_columns['LIGNE_77D_CALC'] = df_tmp['MT_NET_DEP_INO_ME_CORSE']

        # Mise à jour du DataFrame temporaire
        for col in ['LIGNE_71_CALC', 'LIGNE_72_CALC', 'LIGNE_77A_CALC',
                    'LIGNE_77B_CALC', 'LIGNE_77C_CALC', 'LIGNE_77D_CALC']:
            df_calc_temp[col] = calc_columns[col]

        # CIR Innovation selon les taux - changement pour 2023 : taux passé de 20% à 30%
        calc_columns['LIGNE_78_CALC'] = ((df_calc_temp['LIGNE_77A_CALC'] - df_calc_temp['LIGNE_77B_CALC'] -
                                      df_calc_temp['LIGNE_77C_CALC'] - df_calc_temp['LIGNE_77D_CALC']) * 0.3) + \
                                     (df_calc_temp['LIGNE_77B_CALC'] * 0.6) + \
                                     (df_calc_temp['LIGNE_77C_CALC'] * 0.4) + \
                                     (df_calc_temp['LIGNE_77D_CALC'] * 0.35)

        # Ligne 79: Quote-part de crédit d'impôt innovation résultant de la participation sociétés de personnes
        calc_columns['LIGNE_79'] = df_tmp['MT_QP_CII']

        # Ligne 80a: Montant total du crédit d'impôt au titre des dépenses d'innovation
        calc_columns['LIGNE_80A_CALC'] = calc_columns['LIGNE_78_CALC'] + 0 #calc_columns['LIGNE_79'] on remplace quote part par 0 pour 2023

        # Ligne 80b: Montant du crédit d'impôt pour dépenses d'innovation DOM
        calc_columns['LIGNE_80B_CALC'] = df_tmp['MT_CII_YC_QP_DOM']

        # Ligne 80c: Montant du crédit d'impôt pour dépenses d'innovation Corse
        calc_columns['LIGNE_80C_CALC'] = df_tmp['MT_CII_CORSE']

        calc_columns['CIR_INNOVATION_RECALCULE'] = calc_columns['LIGNE_80A_CALC']
    else:
        # Si données non disponibles, mettre à 0 (afin d'éviter des erreurs)
        calc_columns['CIR_INNOVATION_RECALCULE'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_78_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_79'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_80A_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_80B_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_80C_CALC'] = np.zeros(len(df_tmp))

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_INNOVATION_RECALCULE', 'LIGNE_78_CALC', 'LIGNE_79', 'LIGNE_80A_CALC',
               'LIGNE_80B_CALC', 'LIGNE_80C_CALC']:
        if col in calc_columns:
            df_calc_temp[col] = calc_columns[col]

    # Ligne 81a: Montant total du crédit d'impôt au titre des dépenses de recherche, de collection et d'innovation
    ligne_81a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_81a_values.append(row['LIGNE_46A_CALC'] + row['LIGNE_80A_CALC'])
        else:
            ligne_81a_values.append(row['LIGNE_64A_CALC'] + row['LIGNE_80A_CALC'])
    calc_columns['LIGNE_81A_CALC'] = np.array(ligne_81a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_81A_CALC'] = calc_columns['LIGNE_81A_CALC']

    # Ligne 81b: Montant du crédit d'impôt recherche, collection et innovation DOM
    ligne_81b_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_81b_values.append(row['LIGNE_46B_CALC'] + row['LIGNE_80B_CALC'])
        else:
            ligne_81b_values.append(row['LIGNE_64B_CALC'] + row['LIGNE_80B_CALC'])
    calc_columns['LIGNE_81B_CALC'] = np.array(ligne_81b_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_81B_CALC'] = calc_columns['LIGNE_81B_CALC']

    # Calcul ligne 90: Quote-part de crédit d'impôt résultant de la participation sociétés de personnes
    calc_columns['LIGNE_90'] = df_tmp['QP_MT_CRC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_90'] = 0 #calc_columns['LIGNE_90'] on remplace quote part par 0 pour 2023

    # Calcul ligne 91: Montant total du crédit d'impôt au titre des dépenses de recherche collaborative
    calc_columns['LIGNE_91_CALC'] = df_calc_temp['LIGNE_89_CALC'] + df_calc_temp['LIGNE_90']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_91_CALC'] = calc_columns['LIGNE_91_CALC']

    ## TOTAL DU CRÉDIT D'IMPÔT FINAL (CIR + CII + CRC)

    # Initialiser le total avec les composantes Recherche et Collection
    calc_columns['CIR_TOTAL_RECALCULE'] = calc_columns['CIR_RECHERCHE_RECALCULE'] + \
                                          calc_columns['CIR_COLLECTION_RECALCULE']

    # Ajouter la composante Innovation (CII) si elle a été calculée
    if 'CIR_INNOVATION_RECALCULE' in calc_columns:
        # S'assurer que la valeur n'est pas NaN avant d'ajouter
        calc_columns['CIR_TOTAL_RECALCULE'] += np.nan_to_num(calc_columns['CIR_INNOVATION_RECALCULE'])

    # Ajouter la composante Collaborative (CRC) TOTALE (Ligne 91)
    if 'LIGNE_91_CALC' in calc_columns:
        # S'assurer que la valeur n'est pas NaN avant d'ajouter
        calc_columns['CIR_TOTAL_RECALCULE'] += np.nan_to_num(calc_columns['LIGNE_91_CALC'])

    # Mettre à jour le DataFrame temporaire
    if 'CIR_TOTAL_RECALCULE' not in df_calc_temp.columns:
         df_calc_temp['CIR_TOTAL_RECALCULE'] = 0 
    df_calc_temp['CIR_TOTAL_RECALCULE'] = calc_columns['CIR_TOTAL_RECALCULE']

    # ANALYSE DES ÉCARTS

    # Écarts sur toutes les lignes recalculées
    # 1. Dépenses internes
    calc_columns['ECART_LIGNE_6'] = calc_columns['LIGNE_6_CALC'] - df_tmp['OTR_DEP_FONCT']
    calc_columns['ECART_LIGNE_7'] = calc_columns['LIGNE_7_CALC'] - df_tmp['MT_DEP_FONCT_TOT']
    calc_columns['ECART_LIGNE_14'] = calc_columns['LIGNE_14_CALC'] - df_tmp['MT_TOT_RD_1']

    # 2. Dépenses externalisées
    calc_columns['ECART_LIGNE_17'] = calc_columns['LIGNE_17_CALC'] - df_tmp['MT_TOT_DEP_EXT_ORG_AGREE']
    calc_columns['ECART_LIGNE_18'] = calc_columns['LIGNE_18_CALC'] - df_tmp['PLAF_OP_EXT']
    calc_columns['ECART_LIGNE_19'] = calc_columns['LIGNE_19_CALC'] - df_tmp['PLAF_OP_EXT_ORG_AGRE_LIE']
    calc_columns['ECART_LIGNE_20'] = calc_columns['LIGNE_20_CALC'] - df_tmp['PLAF_OP_EXT_ORG_AGRE_NON_LIE']
    calc_columns['ECART_LIGNE_21'] = calc_columns['LIGNE_21_CALC'] - df_tmp['MT_DEP_EXT_PLAF']

    # 3. Montant total et net
    calc_columns['ECART_LIGNE_22'] = calc_columns['LIGNE_22_CALC'] - df_tmp['MT_TOT_RD_2']
    calc_columns['ECART_LIGNE_26A'] = calc_columns['LIGNE_26A_CALC'] - df_tmp['MT_NET_DEP_RD']
    calc_columns['ECART_LIGNE_26B'] = calc_columns['LIGNE_26B_CALC'] - df_tmp['MT_NET_DEP_RD_DOM']

    # 4. Dépenses de collection
    calc_columns['ECART_LIGNE_29'] = calc_columns['LIGNE_29_CALC'] - df_tmp['MT_TOT_DEP_COLL']
    calc_columns['ECART_LIGNE_33A'] = calc_columns['LIGNE_33A_CALC'] - df_tmp['MT_NET_DEP_COLL']
    calc_columns['ECART_LIGNE_33B'] = calc_columns['LIGNE_33B_CALC'] - df_tmp['MT_NET_DEP_COLL_DOM']

    # 5. Innovation
    if all(col in df_tmp.columns for col in ['MT_TOT_DEP_INO', 'MT_TOT_DEP_INO_PLAF', 'MT_NET_DEP_INO']):
        calc_columns['ECART_LIGNE_71'] = calc_columns['LIGNE_71_CALC'] - df_tmp['MT_TOT_DEP_INO']
        calc_columns['ECART_LIGNE_72'] = calc_columns['LIGNE_72_CALC'] - df_tmp['MT_TOT_DEP_INO_PLAF']
        calc_columns['ECART_LIGNE_77A'] = calc_columns['LIGNE_77A_CALC'] - df_tmp['MT_NET_DEP_INO']

    # 6. Collaboratif
    if all(col in df_tmp.columns for col in ['DEP_CRC_PLAF', 'MT_NET_DEP_CRC']):
        calc_columns['ECART_LIGNE_84'] = calc_columns['LIGNE_84_CALC'] - df_tmp['DEP_CRC_PLAF']
        calc_columns['ECART_MT_NET_DEP_CRC'] = calc_columns['MT_NET_DEP_CRC_CALC'] - df_tmp['MT_NET_DEP_CRC']

    # 7. Crédits d'impôt
    calc_columns['ECART_CIR_RECHERCHE'] = calc_columns['CIR_RECHERCHE_RECALCULE'] - df_tmp['MT_CIR_RECH_YC_QP_MOINS_DE100']
    calc_columns['ECART_CIR_COLLECTION'] = calc_columns['CIR_COLLECTION_RECALCULE'] - df_tmp['MT_CI_COLL_APRS_MINIMI']

    if 'CIR_INNOVATION_RECALCULE' in df_calc_temp.columns:
        calc_columns['ECART_CIR_INNOVATION'] = df_calc_temp['CIR_INNOVATION_RECALCULE'] - df_tmp['MT_CII_YC_QP']
    else:
        calc_columns['ECART_CIR_INNOVATION'] = -df_tmp['MT_CII_YC_QP']

    """if 'CIR_COLLAB_RECALCULE' in df_calc_temp.columns:
        calc_columns['ECART_CIR_COLLAB'] = df_calc_temp['CIR_COLLAB_RECALCULE'] - df_tmp['MT_CRC']
    else:
        calc_columns['ECART_CIR_COLLAB'] = -df_tmp['MT_CRC']"""
    
    if 'LIGNE_91_CALC' in calc_columns and 'MT_CRC' in df_tmp.columns:
        calc_columns['ECART_CIR_COLLAB'] = calc_columns['LIGNE_91_CALC'] - df_tmp['MT_CRC']
    elif 'LIGNE_91_CALC' in calc_columns: # Si MT_CRC n'existe pas mais qu'on a recalculé
        calc_columns['ECART_CIR_COLLAB'] = calc_columns['LIGNE_91_CALC']
    elif 'MT_CRC' in df_tmp.columns: # Si on a déclaré mais pas recalculé (devrait pas arriver si CRC est calculé)
        calc_columns['ECART_CIR_COLLAB'] = -df_tmp['MT_CRC']
    else:
        calc_columns['ECART_CIR_COLLAB'] = 0 # Ou une autre valeur par défaut

    # Écart total CIR
    calc_columns['ECART_CIR'] = df_calc_temp['CIR_TOTAL_RECALCULE'] - df_tmp['MT_TOT_CIR_CI_COLL_CII']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['ECART_CIR'] = calc_columns['ECART_CIR']

    # Indicateur de correspondance (avec seuil de tolérance de 1)
    correspondance_values = []
    for ecart in calc_columns['ECART_CIR']:
        if abs(ecart) <= 1:
            correspondance_values.append("Oui")
        else:
            correspondance_values.append("Non")
    calc_columns['CORRESPONDANCE'] = np.array(correspondance_values)

    # Écart relatif
    ecart_relatif_values = []
    for i, row in df_calc_temp.iterrows():
        if row['MT_TOT_CIR_CI_COLL_CII'] > 0:
            ecart_relatif_values.append((row['ECART_CIR'] / row['MT_TOT_CIR_CI_COLL_CII'] * 100))
        else:
            ecart_relatif_values.append(100 if row['ECART_CIR'] > 0 else 0)
    calc_columns['ECART_RELATIF'] = np.array(ecart_relatif_values)

    # Création du DataFrame final avec toutes les colonnes calculées
    df_calc_final = pd.DataFrame(calc_columns)

    # Création du DataFrame final pour l'analyse
    df_num = pd.concat([df_tmp, df_calc_final], axis=1)

    ## CRÉATION DU FICHIER DE COMPARAISON DÉTAILLÉ

    # Préparation du dictionnaire de mapping pour la sortie
    # Structure: 'colonne_originale': 'nom_colonne_sortie'
    mapping_colonnes = {
        # Colonnes d'identification
        'siren_declarant': 'SIREN_DECLARANT',
        'siren_deposant': 'SIREN_DEPOSANT',
        'DESIGN': 'DESIGNATION',
        'COMPLT_DESIGN': 'COMPLEMENT_DESIGNATION',

        # I-A. Dépenses internes
        'DOT_AMORT_IMMO': 'L1_DOTATION_AMORT_IMMO',
        'DOT_AMORT_IMMO_SINISTR': 'L2_DOTATION_AMORT_SINISTR',
        'DEP_CHERCH_TECH': 'L3_DEPENSES_PERSONNEL_CHERCHEURS',
        'REM_SAL_INV': 'L4_REMUNERATION_INVENTEURS',
        'DEP_JD': 'L5_DEPENSES_JEUNES_DOCTEURS',
        'OTR_DEP_FONCT': 'L6_AUTRES_DEP_FONCT_DECLARE',
        'LIGNE_6_CALC': 'L6_AUTRES_DEP_FONCT_CALCULE',
        'ECART_LIGNE_6': 'L6_ECART',
        'MT_DEP_FONCT_TOT': 'L7_TOTAL_DEP_FONCT_DECLARE',
        'LIGNE_7_CALC': 'L7_TOTAL_DEP_FONCT_CALCULE',
        'ECART_LIGNE_7': 'L7_ECART',
        'FRAIS_BREV_COV': 'L8_FRAIS_BREVETS_COV',
        'DEP_MAINT_BREV_COV': 'L9_DEPENSES_DEFENSE_BREVETS',
        'DOT_AMORT_BREV': 'L10_DOTATION_AMORT_BREVETS',
        'DEP_NORMALI': 'L11_DEPENSES_NORMALISATION_BRUT',
        'DEP_NORMALI_RECALC': 'L11_DEPENSES_NORMALISATION_DECLARE',
        'PRIM_COTIZ': 'L12_PRIMES_COTISATIONS_BRUT',
        'PRIM_COTIZ_PLAFONNEES': 'L12_PRIMES_COTISATIONS_PLAFONNEES',
        'DEP_VEIL_TECHNO': 'L13_VEILLE_TECHNO_BRUT',
        'DEP_VEIL_TECHNO_PLAFONNEES': 'L13_VEILLE_TECHNO_PLAFONNEE',
        'MT_TOT_RD_1': 'L14_TOTAL_DEPENSES_INTERNES_DECLARE',
        'LIGNE_14_CALC': 'L14_TOTAL_DEPENSES_INTERNES_CALCULE',
        'ECART_LIGNE_14': 'L14_ECART',

        # I-B. Dépenses externalisées
        'DEP_EXT_LIE_FR': 'L15A_DEPENSES_ORG_LIES_FR',
        'DEP_EXT_LIE_ETR': 'L15B_DEPENSES_ORG_LIES_ETR',
        'DEP_EXT_NON_LIE_FR': 'L16A_DEPENSES_ORG_NON_LIES_FR',
        'DEP_EXT_NON_LIE_ETR': 'L16B_DEPENSES_ORG_NON_LIES_ETR',
        'MT_TOT_DEP_EXT_ORG_AGREE': 'L17_TOTAL_DEP_EXTERNALISEES_DECLARE',
        'LIGNE_17_CALC': 'L17_TOTAL_DEP_EXTERNALISEES_CALCULE',
        'ECART_LIGNE_17': 'L17_ECART',
        'PLAF_OP_EXT': 'L18_PLAFOND_GLOBAL_DECLARE',
        'LIGNE_18_CALC': 'L18_PLAFOND_GLOBAL_CALCULE',
        'ECART_LIGNE_18': 'L18_ECART',
        'PLAF_OP_EXT_ORG_AGRE_LIE': 'L19_PLAFOND_ORG_LIES_DECLARE',
        'LIGNE_19_CALC': 'L19_PLAFOND_ORG_LIES_CALCULE',
        'ECART_LIGNE_19': 'L19_ECART',
        'PLAF_OP_EXT_ORG_AGRE_NON_LIE': 'L20_PLAFOND_ORG_NON_LIES_DECLARE',
        'LIGNE_20_CALC': 'L20_PLAFOND_ORG_NON_LIES_CALCULE',
        'ECART_LIGNE_20': 'L20_ECART',
        'MT_DEP_EXT_PLAF': 'L21_TOTAL_DEP_EXT_PLAFONNEES_DECLARE',
        'LIGNE_21_CALC': 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE',
        'ECART_LIGNE_21': 'L21_ECART',

        # I-C. Montant total et net
        'MT_TOT_RD_2': 'L22_TOTAL_DEPENSES_RECHERCHE_DECLARE',
        'LIGNE_22_CALC': 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE',
        'ECART_LIGNE_22': 'L22_ECART',
        'MT_AID_SUBV': 'L23A_SUBVENTIONS',
        'MT_ENC_PRESTA': 'L23B_SOMMES_ENCAISSEES_TIERS',
        'MT_DEP_CONSEILS_CIR': 'L24_DEPENSES_CONSEIL_CIR',
        'REMBST_SUBV': 'L25_REMBOURSEMENTS_SUBVENTIONS',
        'MT_NET_DEP_RD': 'L26A_MONTANT_NET_DEPENSES_DECLARE',
        'LIGNE_26A_CALC': 'L26A_MONTANT_NET_DEPENSES_CALCULE',
        'ECART_LIGNE_26A': 'L26A_ECART',
        'MT_NET_DEP_RD_DOM': 'L26B_MONTANT_NET_DEPENSES_DOM_DECLARE',
        'LIGNE_26B_CALC': 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE',
        'ECART_LIGNE_26B': 'L26B_ECART',

        # II. Dépenses de collection
        'FRAIS_COLL': 'L27_FRAIS_COLLECTION',
        'FRAIS_DEF_DESSIN': 'L28_FRAIS_DEFENSE_DESSINS_BRUT',
        'FRAIS_DEF_DESSIN_PLAFONNES': 'L28_FRAIS_DEFENSE_DESSINS_PLAFONNES',
        'MT_TOT_DEP_COLL': 'L29_TOTAL_DEPENSES_COLLECTION_DECLARE',
        'LIGNE_29_CALC': 'L29_TOTAL_DEPENSES_COLLECTION_CALCULE',
        'ECART_LIGNE_29': 'L29_ECART',
        'MT_AID_SUBV_COLL': 'L30_SUBVENTIONS_COLLECTION',
        'MT_DEP_CONSEILS_CIR_COLL': 'L31_DEPENSES_CONSEIL_COLLECTION',
        'REMBST_SUBV_COLL': 'L32_REMBOURSEMENTS_SUBVENTIONS_COLL',
        'MT_NET_DEP_COLL': 'L33A_MONTANT_NET_COLLECTION_DECLARE',
        'LIGNE_33A_CALC': 'L33A_MONTANT_NET_COLLECTION_CALCULE',
        'ECART_LIGNE_33A': 'L33A_ECART',
        'MT_NET_DEP_COLL_DOM': 'L33B_MONTANT_NET_COLLECTION_DOM_DECLARE',
        'LIGNE_33B_CALC': 'L33B_MONTANT_NET_COLLECTION_DOM_CALCULE',
        'ECART_LIGNE_33B': 'L33B_ECART',
        'LIGNE_34A_CALC': 'L34A_MONTANT_NET_TOTAL_RD_COLL',
        'LIGNE_34B_CALC': 'L34B_MONTANT_NET_TOTAL_RD_COLL_DOM',

        # III. Calcul CIR Recherche et Collection
        'LIGNE_36_CALC': 'L36_CREDIT_IMPOT_RECHERCHE_MOINS_100M',
        'LIGNE_37': 'L37_QUOTE_PART_RECHERCHE_SOC_PERSONNES',
        'LIGNE_38A_CALC': 'L38A_CREDIT_IMPOT_RECHERCHE_TOTAL',
        'LIGNE_38B_CALC': 'L38B_CREDIT_IMPOT_RECHERCHE_DOM',
        'LIGNE_40_CALC': 'L40_CREDIT_IMPOT_COLLECTION_MOINS_100M',
        'LIGNE_41': 'L41_QUOTE_PART_COLLECTION_SOC_PERSONNES',
        'LIGNE_42A_CALC': 'L42A_CREDIT_IMPOT_COLL_AVANT_PLAF',
        'LIGNE_42B_CALC': 'L42B_CREDIT_IMPOT_COLL_DOM_AVANT_PLAF',
        'LIGNE_43': 'L43_AIDES_MINIMIS',
        'LIGNE_44': 'L44_CUMUL_CREDIT_IMPOT_ET_AIDES',
        'LIGNE_45A_CALC': 'L45A_CREDIT_IMPOT_COLL_APRES_PLAF',
        'LIGNE_45B_CALC': 'L45B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF',
        'LIGNE_46A_CALC': 'L46A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION',
        'LIGNE_46B_CALC': 'L46B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM',
        'LIGNE_47A_CALC': 'L47A_DEPENSES_RECHERCHE_LIMITE_100M',
        'LIGNE_47B_CALC': 'L47B_DEPENSES_RECHERCHE_DOM_LIMITE',
        'LIGNE_48_CALC': 'L48_CIR_RECHERCHE_PREMIERE_TRANCHE',
        'LIGNE_49_CALC': 'L49_DEPENSES_RECHERCHE_SUP_100M',
        'LIGNE_50_CALC': 'L50_CIR_RECHERCHE_DEUXIEME_TRANCHE',
        'LIGNE_51_CALC': 'L51_CIR_RECHERCHE_PLUS_100M',
        'LIGNE_52': 'L52_QUOTE_PART_RECHERCHE_SOC_PERSONNES_PLUS_100M',
        'LIGNE_53A_CALC': 'L53A_CREDIT_IMPOT_RECHERCHE_TOTAL_PLUS_100M',
        'LIGNE_53B_CALC': 'L53B_CREDIT_IMPOT_RECHERCHE_DOM_PLUS_100M',
        'LIGNE_54A_CALC': 'L54A_MONTANT_NET_COLLECTION_PLUS_100M',
        'LIGNE_54B_CALC': 'L54B_MONTANT_NET_COLLECTION_DOM_PLUS_100M',
        'LIGNE_55_CALC': 'L55_PLAFOND_DISPO_COLLECTION',
        'LIGNE_56_CALC': 'L56_CIR_COLLECTION_PREMIERE_TRANCHE',
        'LIGNE_57_CALC': 'L57_CIR_COLLECTION_DEUXIEME_TRANCHE',
        'LIGNE_58_CALC': 'L58_CIR_COLLECTION_PLUS_100M',
        'LIGNE_59': 'L59_QUOTE_PART_COLLECTION_SOC_PERSONNES_PLUS_100M',
        'LIGNE_60': 'L60_CREDIT_IMPOT_COLL_AVANT_PLAF_PLUS_100M',
        'LIGNE_61': 'L61_AIDES_MINIMIS_PLUS_100M',
        'LIGNE_62': 'L62_CUMUL_CREDIT_IMPOT_ET_AIDES_PLUS_100M',
        'LIGNE_63A_CALC': 'L63A_CREDIT_IMPOT_COLL_APRES_PLAF_PLUS_100M',
        'LIGNE_63B_CALC': 'L63B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF_PLUS_100M',
        'LIGNE_64A_CALC': 'L64A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_PLUS_100M',
        'LIGNE_64B_CALC': 'L64B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M',

        # IV. Dépenses d'innovation
        'DOT_AMORT_IMMO_INO': 'L65_DOTATION_AMORT_IMMO_INNOVATION',
        'DEP_PERSONEL_INO': 'L66_DEPENSES_PERSONNEL_INNOVATION',
        'FRAIS_BREV_COV_INO': 'L67_FRAIS_BREVETS_INNOVATION', # Changé pour 2023
        'FRAIS_DEF_BREV_INO': 'L68_FRAIS_DEFENSE_BREVETS_INNOVATION',
        'OP_INOV_EXT': 'L69_OPERATIONS_CONFIEES_INNOVATION',
        'MT_TOT_DEP_INO': 'L70_TOTAL_DEPENSES_INNOVATION_DECLARE', # Changé pour 2023 (ligne 71->70)
        'LIGNE_71_CALC': 'L70_TOTAL_DEPENSES_INNOVATION_CALCULE', # Changé pour 2023 (ligne 71->70)
        'ECART_LIGNE_71': 'L70_ECART', # Changé pour 2023 (ligne 71->70)
        'MT_TOT_DEP_INO_PLAF': 'L71_DEPENSES_INNOVATION_PLAFONNEES_DECLARE', # Changé pour 2023 (ligne 72->71)
        'LIGNE_72_CALC': 'L71_DEPENSES_INNOVATION_PLAFONNEES_CALCULE', # Changé pour 2023 (ligne 72->71)
        'ECART_LIGNE_72': 'L71_ECART', # Changé pour 2023 (ligne 72->71)
        'MT_AID_SUBV_INO': 'L72_SUBVENTIONS_INNOVATION', # Changé pour 2023 (ligne 73->72)
        'MT_ENC_PRESTA_INO': 'L73_PRESTATIONS_INNOVATION', # Changé pour 2023 (ligne 74->73)
        'MT_DEP_CONSEILS_CII': 'L74_DEPENSES_CONSEIL_INNOVATION', # Changé pour 2023 (ligne 75->74)
        'REMBST_SUBV_INO': 'L75_REMBOURSEMENTS_SUBVENTIONS_INNO', # Changé pour 2023 (ligne 76->75)
        'MT_NET_DEP_INO': 'L76A_MONTANT_NET_INNOVATION_DECLARE', # Changé pour 2023 (ligne 77a->76a)
        'LIGNE_77A_CALC': 'L76A_MONTANT_NET_INNOVATION_CALCULE', # Changé pour 2023 (ligne 77a->76a)
        'ECART_LIGNE_77A': 'L76A_ECART', # Changé pour 2023 (ligne 77a->76a)
        'MT_NET_DEP_INO_DOM': 'L76B_MONTANT_NET_INNOVATION_DOM', # Changé pour 2023 (ligne 77b->76b)
        'MT_NET_DEP_INO_MPE_CORSE': 'L76C_MONTANT_NET_INNOVATION_CORSE_MPE', # Changé pour 2023 (ligne 77c->76c)
        'MT_NET_DEP_INO_ME_CORSE': 'L76D_MONTANT_NET_INNOVATION_CORSE_ME', # Changé pour 2023 (ligne 77d->76d)
        'LIGNE_78_CALC': 'L77_CREDIT_IMPOT_INNOVATION', # Changé pour 2023 (ligne 78->77)
        'LIGNE_79': 'L78_QUOTE_PART_INNOVATION_SOC_PERSONNES', # Changé pour 2023 (ligne 79->78)
        'LIGNE_80A_CALC': 'L79A_TOTAL_CREDIT_IMPOT_INNOVATION', # Changé pour 2023 (ligne 80a->79a)
        'LIGNE_80B_CALC': 'L79B_CREDIT_IMPOT_INNOVATION_DOM', # Changé pour 2023 (ligne 80b->79b)
        'LIGNE_80C_CALC': 'L79C_CREDIT_IMPOT_INNOVATION_CORSE', # Changé pour 2023 (ligne 80c->79c)
        'LIGNE_81A_CALC': 'L80A_TOTAL_CIR_RECH_COLL_INNO', # Changé pour 2023 (ligne 81a->80a)
        'LIGNE_81B_CALC': 'L80B_TOTAL_CIR_RECH_COLL_INNO_DOM', # Changé pour 2023 (ligne 81b->80b)

        # V. Recherche collaborative
        'DEP_CRC': 'L81_DEPENSES_RECHERCHE_COLLABORATIVE', # Changé pour 2023 (ligne 82->81)
        'DEP_CRC_FR': 'L82A_DEPENSES_RC_FRANCE', # Changé pour 2023 (ligne 83a->82a)
        'DEP_CRC_ETR': 'L82B_DEPENSES_RC_ETRANGER', # Changé pour 2023 (ligne 83b->82b)
        'LIGNE_83_CALC': 'L82_TOTAL_DEPENSES_RC', # Changé pour 2023 (ligne 83->82)
        'DEP_CRC_PLAF': 'L83_DEPENSES_RC_PLAFONNEES_DECLARE', # Changé pour 2023 (ligne 84->83)
        'LIGNE_84_CALC': 'L83_DEPENSES_RC_PLAFONNEES_CALCULE', # Changé pour 2023 (ligne 84->83)
        'ECART_LIGNE_84': 'L83_ECART', # Changé pour 2023 (ligne 84->83)
        'AIDE_PUB_CRC': 'L84_AIDES_PUBLIQUES_RC', # Changé pour 2023 (ligne 85->84)
        'AIDE_PUB_REMB_CRC': 'L85_REMBOURSEMENTS_AIDES_RC', # Changé pour 2023 (ligne 86->85)
        'MT_NET_DEP_CRC': 'L86_MONTANT_NET_RC_DECLARE', # Changé pour 2023 (ligne 87->86)
        'MT_NET_DEP_CRC_CALC': 'L86_MONTANT_NET_RC_CALCULE', # Changé pour 2023 (ligne 87->86)
        'ECART_MT_NET_DEP_CRC': 'L86_ECART', # Changé pour 2023 (ligne 87->86)
        'MT_NET_DEP_CRC_PME': 'L87_MONTANT_NET_RC_PME', # Changé pour 2023 (ligne 88->87)
        'LIGNE_89_CALC': 'L88_CREDIT_IMPOT_RC', # Changé pour 2023 (ligne 89->88)
        'LIGNE_90': 'L89_QUOTE_PART_RC_SOC_PERSONNES', # Changé pour 2023 (ligne 90->89)
        'LIGNE_91_CALC': 'L90_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE', # Changé pour 2023 (ligne 91->90)

        # Crédits d'impôt par composante
        'MT_CIR_RECH_YC_QP_MOINS_DE100': 'CIR_RECHERCHE_DECLARE',
        'CIR_RECHERCHE_RECALCULE': 'CIR_RECHERCHE_CALCULE',
        'ECART_CIR_RECHERCHE': 'CIR_RECHERCHE_ECART',

        'MT_CI_COLL_APRS_MINIMI': 'CIR_COLLECTION_DECLARE',
        'CIR_COLLECTION_RECALCULE': 'CIR_COLLECTION_CALCULE',
        'ECART_CIR_COLLECTION': 'CIR_COLLECTION_ECART',

        'MT_CII_YC_QP': 'CIR_INNOVATION_DECLARE',
        'CIR_INNOVATION_RECALCULE': 'CIR_INNOVATION_CALCULE',
        'ECART_CIR_INNOVATION': 'CIR_INNOVATION_ECART',

        'MT_CRC': 'CIR_COLLABORATIF_DECLARE',
        'CIR_COLLAB_RECALCULE': 'CIR_COLLABORATIF_CALCULE',
        'ECART_CIR_COLLAB': 'CIR_COLLABORATIF_ECART',

        # CIR Total
        'MT_TOT_CIR_CI_COLL_CII': 'CIR_TOTAL_DECLARE',
        'CIR_TOTAL_RECALCULE': 'CIR_TOTAL_CALCULE',
        'ECART_CIR': 'CIR_TOTAL_ECART',

        # Indicateurs
        'CORRESPONDANCE': 'CORRESPONDANCE_CIR',
        'ECART_RELATIF': 'ECART_RELATIF_POURCENT'
    }

    # Création du DataFrame de comparaison
    df_comparaison = pd.DataFrame()

    # Copier chaque colonne si elle existe
    for col_orig, col_dest in mapping_colonnes.items():
        if col_orig in df_num.columns:
            df_comparaison[col_dest] = df_num[col_orig]
        else:
            # Ne pas ajouter les colonnes qui n'existent pas
            pass

    # Concaténation de la désignation et du complément si disponible
    if 'COMPLEMENT_DESIGNATION' in df_comparaison.columns:
        df_comparaison['DESIGNATION'] = df_comparaison.apply(
            lambda x: f"{x['DESIGNATION']} {x['COMPLEMENT_DESIGNATION']}" if pd.notna(x['COMPLEMENT_DESIGNATION']) else x['DESIGNATION'],
            axis=1
        )
        df_comparaison.drop('COMPLEMENT_DESIGNATION', axis=1, inplace=True)

    # Formatage des montants
    montant_cols = [col for col in df_comparaison.columns if col not in ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'DESIGNATION', 'CORRESPONDANCE_CIR']]
    for col in montant_cols:
        df_comparaison[col] = df_comparaison[col].round(2)

    # Sauvegarde du fichier de sortie
    output_filename = f"Calcul_Creance_CIR_2023.csv"
    output_path = os.path.join("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB", output_filename)

    df_comparaison.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

    print(f"\nFichier de comparaison détaillée créé: {output_path}")

    # RÉSULTATS

    # Somme des CIR déclarés
    cir_declare_total = df_tmp['MT_TOT_CIR_CI_COLL_CII'].sum()

    # Somme des CIR recalculés
    cir_recalcule_total = df_calc_final['CIR_TOTAL_RECALCULE'].sum()

    # Nombre d'entreprises
    nb_entreprises_avec_cir = len(df_tmp[df_tmp['MT_TOT_CIR_CI_COLL_CII'] > 0])
    nb_entreprises_plus_100m = df_calc_final['DEPENSES_PLUS_100M'].sum()

    # Classement des écarts
    ecarts_positifs = df_num[df_num['ECART_CIR'] > 1]  # Plus de 1 en plus
    ecarts_negatifs = df_num[df_num['ECART_CIR'] < -1]  # Plus de 1 en moins
    ecarts_conformes = df_num[abs(df_num['ECART_CIR']) <= 1]  # Différence de 1 ou moins

    # AFFICHAGE DES RÉSULTATS
    print("\n===== COMPARAISON CIR DÉCLARÉ VS RECALCULÉ =====")

    print(f"\nNombre total d'entreprises analysées: {len(df_num):,}")
    print(f"Nombre d'entreprises avec CIR déclaré: {nb_entreprises_avec_cir:,}")
    print(f"Nombre d'entreprises avec dépenses > 100M€: {nb_entreprises_plus_100m:,}")

    print(f"\nCIR TOTAL:")
    print(f"CIR déclaré total: {cir_declare_total:,.2f} €")
    print(f"CIR recalculé total: {cir_recalcule_total:,.2f} €")

    difference = cir_recalcule_total - cir_declare_total
    print(f"Différence: {difference:,.2f} €")
    if cir_declare_total > 0:
        print(f"Écart relatif: {difference/cir_declare_total*100:.2f}%")

    print(f"\nANALYSE DES ÉCARTS:")
    print(f"Entreprises avec CIR conforme (écart ≤ 1€): {len(ecarts_conformes):,} ({len(ecarts_conformes)/len(df_num)*100:.2f}%)")

    print(f"Entreprises avec CIR recalculé > CIR déclaré (écart > 1€): {len(ecarts_positifs):,}")
    print(f"Montant total des écarts positifs: {ecarts_positifs['ECART_CIR'].sum():,.2f} €")

    print(f"Entreprises avec CIR recalculé < CIR déclaré (écart < -1€): {len(ecarts_negatifs):,}")
    print(f"Montant total des écarts négatifs: {ecarts_negatifs['ECART_CIR'].sum():,.2f} €")

    # Détail par composante du CIR
    print(f"\nDÉTAIL PAR COMPOSANTE DU CIR RECALCULÉ:")
    print(f"CIR Recherche: {df_num['CIR_RECHERCHE_RECALCULE'].sum():,.2f} €")
    print(f"CIR Collection: {df_num['CIR_COLLECTION_RECALCULE'].sum():,.2f} €")

    if 'CIR_INNOVATION_RECALCULE' in df_num.columns:
        print(f"CIR Innovation: {df_num['CIR_INNOVATION_RECALCULE'].sum():,.2f} €")
    else:
        print(f"CIR Innovation: 0.00 €")

    if 'CIR_COLLAB_RECALCULE' in df_num.columns:
        print(f"CIR Recherche Collaborative: {df_num['CIR_COLLAB_RECALCULE'].sum():,.2f} €")
    else:
        print(f"CIR Recherche Collaborative: 0.00 €")

    return df_comparaison

# Exécuter le calcul
try:
    df_resultat = comparer_cir_declare_recalcule()
    print("\nTraitement terminé avec succès!")
except Exception as e:
    print(f"\nErreur: {type(e).__name__}: {str(e)}")

Nombre total d'entreprises: 27,923


C:\Users\msamb\AppData\Local\Temp\ipykernel_15208\2350444244.py:996: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_comparaison[col_dest] = df_num[col_orig]
C:\Users\msamb\AppData\Local\Temp\ipykernel_15208\2350444244.py:996: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_comparaison[col_dest] = df_num[col_orig]
C:\Users\msamb\AppData\Local\Temp\ipykernel_15208\2350444244.py:996: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  


Fichier de comparaison détaillée créé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB\Calcul_Creance_CIR_2023.csv

===== COMPARAISON CIR DÉCLARÉ VS RECALCULÉ =====

Nombre total d'entreprises analysées: 27,923
Nombre d'entreprises avec CIR déclaré: 27,178
Nombre d'entreprises avec dépenses > 100M€: 21

CIR TOTAL:
CIR déclaré total: 7,986,816,564.00 €
CIR recalculé total: 7,868,711,719.29 €
Différence: -118,104,844.71 €
Écart relatif: -1.48%

ANALYSE DES ÉCARTS:
Entreprises avec CIR conforme (écart ≤ 1€): 26,191 (93.80%)
Entreprises avec CIR recalculé > CIR déclaré (écart > 1€): 733
Montant total des écarts positifs: 31,893,319.83 €
Entreprises avec CIR recalculé < CIR déclaré (écart < -1€): 999
Montant total des écarts négatifs: -149,998,039.80 €

DÉTAIL PAR COMPOSANTE DU CIR RECALCULÉ:
CIR Recherche: 7,323,998,043.04 €
CIR Collection: 27,543,075.80 €
CIR Innovation: 500,767,614.75 €
CIR Recherche Collaborative: 16,402,985.70 €

Traitement terminé avec succès!


# Difference Creance Déclarer vs Calculé

## Crc manquant


In [3]:
import pandas as pd
import numpy as np

# Fonction pour formater les SIREN correctement
def format_siren(siren):
    """Formate les SIREN pour avoir exactement 9 caractères en ajoutant des zéros au début si nécessaire"""
    if pd.isna(siren):
        return siren

    siren_str = str(siren).strip()

    # Si le SIREN a moins de 9 caractères, ajouter des zéros au début
    if len(siren_str) < 9:
        siren_str = siren_str.zfill(9)  # zfill ajoute des zéros au début jusqu'à atteindre la longueur spécifiée

    return siren_str

# Charger le fichier CSV avec converters pour traiter SIREN comme du texte
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str})

print(f"Nombre total d'entreprises: {len(df)}")
print(f"Nombre d'entreprises avec écarts (CORRESPONDANCE_CIR = 'Non'): {len(df[df['CORRESPONDANCE_CIR'] == 'Non'])}")

# 1. Corriger le format des SIREN
if 'SIREN_DECLARANT' in df.columns:
    # Afficher quelques exemples de SIREN avant correction
    print("\nExemples de SIREN_DECLARANT avant correction:")
    for siren in df['SIREN_DECLARANT'].head(5):
        print(f"  {siren} (type: {type(siren)}, longueur: {len(str(siren))})")

    # Identifier les SIREN qui n'ont pas 9 caractères
    siren_lengths = df['SIREN_DECLARANT'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
    unusual_siren_count = (siren_lengths != 9).sum()
    print(f"\nNombre de SIREN n'ayant pas exactement 9 caractères: {unusual_siren_count}")

    # Appliquer la correction aux SIREN
    df['SIREN_DECLARANT'] = df['SIREN_DECLARANT'].apply(format_siren)

    # Si la colonne SIREN_DEPOSANT existe également, la corriger
    if 'SIREN_DEPOSANT' in df.columns:
        df['SIREN_DEPOSANT'] = df['SIREN_DEPOSANT'].apply(format_siren)

    # Vérifier les SIREN après correction
    print("\nExemples de SIREN_DECLARANT après correction:")
    for siren in df['SIREN_DECLARANT'].head(5):
        print(f"  {siren} (type: {type(siren)}, longueur: {len(str(siren))})")

    # Vérifier qu'il n'y a plus de SIREN avec une longueur incorrecte
    siren_lengths_after = df['SIREN_DECLARANT'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
    unusual_siren_count_after = (siren_lengths_after != 9).sum()
    print(f"Nombre de SIREN n'ayant pas exactement 9 caractères après correction: {unusual_siren_count_after}")

# 2. Identifier les cas où l'écart est dû au CRC manquant
tolerance = 1.0  # 1€ de tolérance

# Créer un masque pour identifier les cas où l'écart correspond au CRC calculé
mask_crc_missing = (
    # Cas où l'écart est positif (montant calculé > montant déclaré)
    (df['CIR_TOTAL_ECART'] > 0) &
    # L'écart est approximativement égal au CRC calculé
    (abs(df['CIR_TOTAL_ECART'] - df['CIR_COLLABORATIF_CALCULE']) < tolerance) &
    # Le CRC déclaré est nul ou très petit comparé au CRC calculé
    ((df['CIR_COLLABORATIF_DECLARE'].isna()) |
     (df['CIR_COLLABORATIF_DECLARE'] == 0) |
     (df['CIR_COLLABORATIF_CALCULE'] > df['CIR_COLLABORATIF_DECLARE'] * 1.5))
)

# Compter les cas identifiés
count_crc_missing = mask_crc_missing.sum()
print(f"\nNombre d'entreprises avec CRC apparemment manquant dans le montant déclaré: {count_crc_missing}")

# 3. Sauvegarder les valeurs originales (pour information uniquement)
df['CIR_TOTAL_DECLARE_ORIGINAL'] = df['CIR_TOTAL_DECLARE'].copy()
df['CIR_TOTAL_ECART_ORIGINAL'] = df['CIR_TOTAL_ECART'].copy()
df['CORRESPONDANCE_CIR_ORIGINAL'] = df['CORRESPONDANCE_CIR'].copy()

# 4. Corriger directement le montant déclaré en ajoutant le CRC calculé
df.loc[mask_crc_missing, 'CIR_TOTAL_DECLARE'] = df.loc[mask_crc_missing, 'CIR_TOTAL_DECLARE'] + df.loc[mask_crc_missing, 'CIR_COLLABORATIF_CALCULE']

# 5. Recalculer l'écart
df['CIR_TOTAL_ECART'] = df['CIR_TOTAL_CALCULE'] - df['CIR_TOTAL_DECLARE']

# 6. Mettre à jour la correspondance
df['CORRESPONDANCE_CIR'] = np.where(abs(df['CIR_TOTAL_ECART']) <= 1, "Oui", "Non")

# 7. Analyser les résultats
count_original_non = (df['CORRESPONDANCE_CIR_ORIGINAL'] == 'Non').sum()
count_corrected_non = (df['CORRESPONDANCE_CIR'] == 'Non').sum()
count_fixed = count_original_non - count_corrected_non

print(f"\nRésultats après correction:")
print(f"Nombre d'entreprises avec écarts avant correction: {count_original_non}")
print(f"Nombre d'entreprises avec écarts après correction: {count_corrected_non}")
print(f"Nombre d'écarts résolus par l'ajout du CRC: {count_fixed} ({count_fixed/count_original_non*100:.2f}%)")

# 8. Exemples d'entreprises corrigées (pour information uniquement)
print("\nExemples d'entreprises où l'écart a été résolu par l'ajout du CRC:")
df_resolved = df[(df['CORRESPONDANCE_CIR_ORIGINAL'] == 'Non') & (df['CORRESPONDANCE_CIR'] == 'Oui')].copy()
if len(df_resolved) > 0:
    sample_size = min(3, len(df_resolved))
    for i, (_, row) in enumerate(df_resolved.head(sample_size).iterrows(), 1):
        print(f"\nExemple {i}: {row.get('DESIGNATION', 'N/A')} (SIREN: {row.get('SIREN_DECLARANT', 'N/A')})")
        print(f"  CIR déclaré original: {row['CIR_TOTAL_DECLARE_ORIGINAL']:.2f}")
        print(f"  CIR déclaré corrigé: {row['CIR_TOTAL_DECLARE']:.2f}")
        print(f"  CIR calculé: {row['CIR_TOTAL_CALCULE']:.2f}")
        print(f"  Écart original: {row['CIR_TOTAL_ECART_ORIGINAL']:.2f}")
        print(f"  Écart corrigé: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"  CRC calculé: {row.get('CIR_COLLABORATIF_CALCULE', 'N/A')}")

# 9. Supprimer les colonnes originales pour ne garder que les valeurs corrigées
df = df.drop(['CIR_TOTAL_DECLARE_ORIGINAL', 'CIR_TOTAL_ECART_ORIGINAL', 'CORRESPONDANCE_CIR_ORIGINAL'], axis=1)

# 10. Écraser le fichier original avec les données corrigées
# S'assurer que les SIREN sont toujours formatés comme du texte lors de l'export
df.to_csv(file_path, sep=';', encoding='utf-8-sig', index=False)

print(f"\nFichier original mis à jour: {file_path}")
print(f"- Format des SIREN corrigé")
print(f"- CRC manquant ajouté au montant déclaré")

# 11. Résumé des écarts restants
if count_corrected_non > 0:
    print(f"\nIl reste {count_corrected_non} entreprises avec des écarts à analyser.")

    # Identifier les principales sources d'écarts restants
    df_remaining_ecarts = df[df['CORRESPONDANCE_CIR'] == 'Non']

    # Obtenir toutes les colonnes d'écart
    ecart_cols = [col for col in df.columns if '_ECART' in col and col != 'CIR_TOTAL_ECART' and col != 'ECART_RELATIF_POURCENT']

    # Compter les écarts significatifs par colonne
    ecart_counts = {}
    for col in ecart_cols:
        if col in df_remaining_ecarts.columns:
            non_zero = df_remaining_ecarts[abs(df_remaining_ecarts[col]) > 1]
            if len(non_zero) > 0:
                ecart_counts[col] = len(non_zero)

    # Afficher les 5 principales sources d'écarts
    sorted_ecarts = sorted(ecart_counts.items(), key=lambda x: x[1], reverse=True)
    if sorted_ecarts:
        print("\nPrincipales sources d'écarts restants:")
        for i, (col, count) in enumerate(sorted_ecarts[:5], 1):
            pct = count / count_corrected_non * 100
            print(f"{i}. {col}: {count} entreprises ({pct:.2f}%)")

    # Rechercher d'autres cas potentiels similaires au problème du CRC
    print("\nRecherche d'autres patterns d'écarts similaires:")

    # 1. Cas où l'écart correspond au CIR Innovation
    if 'CIR_INNOVATION_CALCULE' in df_remaining_ecarts.columns:
        innovation_missing = (
            (df_remaining_ecarts['CIR_TOTAL_ECART'] > 0) &
            (abs(df_remaining_ecarts['CIR_TOTAL_ECART'] - df_remaining_ecarts['CIR_INNOVATION_CALCULE']) < tolerance)
        )
        count_innovation = innovation_missing.sum()
        if count_innovation > 0:
            print(f"  • Possible CIR Innovation manquant: {count_innovation} entreprises")

    # 2. Cas où l'écart correspond au CIR Collection
    if 'CIR_COLLECTION_CALCULE' in df_remaining_ecarts.columns:
        collection_missing = (
            (df_remaining_ecarts['CIR_TOTAL_ECART'] > 0) &
            (abs(df_remaining_ecarts['CIR_TOTAL_ECART'] - df_remaining_ecarts['CIR_COLLECTION_CALCULE']) < tolerance)
        )
        count_collection = collection_missing.sum()
        if count_collection > 0:
            print(f"  • Possible CIR Collection manquant: {count_collection} entreprises")

Nombre total d'entreprises: 27923
Nombre d'entreprises avec écarts (CORRESPONDANCE_CIR = 'Non'): 1732

Exemples de SIREN_DECLARANT avant correction:
  5520325.0 (type: <class 'str'>, longueur: 9)
  5580436.0 (type: <class 'str'>, longueur: 9)
  5680145.0 (type: <class 'str'>, longueur: 9)
  6380042.0 (type: <class 'str'>, longueur: 9)
  6580195.0 (type: <class 'str'>, longueur: 9)

Nombre de SIREN n'ayant pas exactement 9 caractères: 27916

Exemples de SIREN_DECLARANT après correction:
  5520325.0 (type: <class 'str'>, longueur: 9)
  5580436.0 (type: <class 'str'>, longueur: 9)
  5680145.0 (type: <class 'str'>, longueur: 9)
  6380042.0 (type: <class 'str'>, longueur: 9)
  6580195.0 (type: <class 'str'>, longueur: 9)
Nombre de SIREN n'ayant pas exactement 9 caractères après correction: 27915

Nombre d'entreprises avec CRC apparemment manquant dans le montant déclaré: 11943

Résultats après correction:
Nombre d'entreprises avec écarts avant correction: 1732
Nombre d'entreprises avec écar

## Erreur depense personnel

In [4]:
import pandas as pd
import numpy as np

# Charger le fichier original
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Identifier les lignes concernées (ajout du critère correspondance)
mask = (
    (df['L3_DEPENSES_PERSONNEL_CHERCHEURS'] == 0.0) &
    (df['L2_DOTATION_AMORT_SINISTR'] != 0.0) &
    (df['CORRESPONDANCE_CIR'] == "Non")
)
indices_to_update = df[mask].index
print(f"Nombre de lignes à corriger: {len(indices_to_update)}")


# Sauvegarder les valeurs originales pour analyse
cir_calcule_avant = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].copy()

# Pour chaque ligne à corriger
for idx in indices_to_update:
    # 1. Transférer la valeur de dotation aux amortissements sinistrés vers dépenses de personnel
    val_amort_sinistr = df.at[idx, 'L2_DOTATION_AMORT_SINISTR']
    df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS'] = val_amort_sinistr
    df.at[idx, 'L2_DOTATION_AMORT_SINISTR'] = 0.0

    # 2. Recalculer la ligne 6 (autres dépenses de fonctionnement)
    dot_amort_immo = df.at[idx, 'L1_DOTATION_AMORT_IMMO']
    dep_cherch_tech = df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS']  # Nouvelle valeur
    rem_sal_inv = df.at[idx, 'L4_REMUNERATION_INVENTEURS']
    dep_jd = df.at[idx, 'L5_DEPENSES_JEUNES_DOCTEURS']

    autres_dep_fonct = (dot_amort_immo * 0.75) + ((dep_cherch_tech + rem_sal_inv) * 0.43) + dep_jd
    df.at[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct

    # 3. Recalculer la ligne 7 (total dépenses de fonctionnement)
    total_dep_fonct = dot_amort_immo + 0.0 + dep_cherch_tech + rem_sal_inv + dep_jd + autres_dep_fonct
    df.at[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct

    # 4. Recalculer la ligne 14 (total dépenses internes)
    frais_brev_cov = df.at[idx, 'L8_FRAIS_BREVETS_COV']
    dep_maint_brev_cov = df.at[idx, 'L9_DEPENSES_DEFENSE_BREVETS']
    dot_amort_brev = df.at[idx, 'L10_DOTATION_AMORT_BREVETS']
    dep_normali = df.at[idx, 'L11_DEPENSES_NORMALISATION_DECLARE']
    prim_cotiz = min(df.at[idx, 'L12_PRIMES_COTISATIONS_BRUT'], 60000)
    dep_veil_techno = min(df.at[idx, 'L13_VEILLE_TECHNO_BRUT'], 60000)

    total_dep_internes = total_dep_fonct + frais_brev_cov + dep_maint_brev_cov + dot_amort_brev + dep_normali + prim_cotiz + dep_veil_techno
    df.at[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes

    # 5. Recalculer la ligne 22 (total dépenses de recherche)
    total_dep_ext = df.at[idx, 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']
    total_dep_recherche = total_dep_internes + total_dep_ext
    df.at[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche

    # 6. Recalculer la ligne 26A (montant net des dépenses)
    mt_aid_subv = df.at[idx, 'L23A_SUBVENTIONS']
    mt_enc_presta = df.at[idx, 'L23B_SOMMES_ENCAISSEES_TIERS']
    mt_dep_conseils_cir = df.at[idx, 'L24_DEPENSES_CONSEIL_CIR']
    rembst_subv = df.at[idx, 'L25_REMBOURSEMENTS_SUBVENTIONS']

    montant_net = total_dep_recherche - mt_aid_subv - mt_enc_presta - mt_dep_conseils_cir + rembst_subv
    df.at[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net

    # 7. Recalculer le CIR recherche (30% du montant net)
    cir_recherche = montant_net * 0.30
    df.at[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche

    # 8. Recalculer le CIR total
    cir_collection = df.at[idx, 'CIR_COLLECTION_CALCULE']
    cir_innovation = df.at[idx, 'CIR_INNOVATION_CALCULE']
    cir_collaboratif = df.at[idx, 'CIR_COLLABORATIF_CALCULE']

    cir_total = cir_recherche + cir_collection + cir_innovation + cir_collaboratif
    df.at[idx, 'CIR_TOTAL_CALCULE'] = cir_total

    # 9. Recalculer l'écart et mettre à jour la correspondance
    ecart_cir = cir_total - df.at[idx, 'CIR_TOTAL_DECLARE']
    df.at[idx, 'CIR_TOTAL_ECART'] = ecart_cir
    df.at[idx, 'CORRESPONDANCE_CIR'] = "Oui" if abs(ecart_cir) <= 1 else "Non"

# Afficher les résultats
somme_declare = df.loc[indices_to_update, 'CIR_TOTAL_DECLARE'].sum()
somme_calcule_avant = cir_calcule_avant.sum()
somme_calcule_apres = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].sum()

print(f"\nRésultats de la correction:")
print(f"Somme CIR_TOTAL_DECLARE: {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE avant correction: {somme_calcule_avant:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE après correction: {somme_calcule_apres:,.2f} €")
print(f"Écart avant correction: {somme_calcule_avant - somme_declare:,.2f} €")
print(f"Écart après correction: {somme_calcule_apres - somme_declare:,.2f} €")

# Sauvegarder le fichier corrigé
df.to_csv("lignes_restantes_a_analyser_corrigees.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees.csv")

Fichier chargé: 27923 lignes
Nombre de lignes à corriger: 8

Résultats de la correction:
Somme CIR_TOTAL_DECLARE: 814,166.00 €
Somme CIR_TOTAL_CALCULE avant correction: 741,658.50 €
Somme CIR_TOTAL_CALCULE après correction: 1,015,295.11 €
Écart avant correction: -72,507.50 €
Écart après correction: 201,129.11 €

Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees.csv


### mise à jour ligne restante

In [5]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres :
# - L5_DEPENSES_JEUNES_DOCTEURS = 0.0
# - L2_DOTATION_AMORT_SINISTR différent de 0.0
df_filtered = df[
    (df['L3_DEPENSES_PERSONNEL_CHERCHEURS'] == 0.0) &
    (df['L2_DOTATION_AMORT_SINISTR'] != 0.0) &
    (df['CORRESPONDANCE_CIR'] == "Non") 
]

print(f"Nombre de lignes après filtrage: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

# Afficher quelques statistiques sur les lignes trouvées
if len(df_filtered) > 0:
    print("\nAperçu des données trouvées:")
    print(f"Nombre d'entreprises: {len(df_filtered)}")

Fichier chargé: 27923 lignes
Nombre de lignes après filtrage: 8
Somme CIR_TOTAL_DECLARE : 814,166.00 €
Somme CIR_TOTAL_CALCULE : 741,658.50 €
Résultats exportés dans: lignes_jeunes_docteurs_0_amort_sinistr_non0.csv

Aperçu des données trouvées:
Nombre d'entreprises: 8


In [6]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')  # Nouveau fichier

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])  # Nouveau set

# Combiner tous les SIREN à exclure
siren_a_exclure =  siren_jeunes_docteurs  # Ajout du nouveau set

# Filtrer le DataFrame principal
# - Exclure les SIREN déjà analysés
# - Garder uniquement les lignes où CORRESPONDANCE_CIR est "Non"
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) & 
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 1721
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur DEP Fonctionnement

In [7]:
import pandas as pd
import numpy as np

# Charger le fichier original
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Identifier les lignes à corriger
mask = (df['L7_TOTAL_DEP_FONCT_DECLARE'] == df['L8_FRAIS_BREVETS_COV']) & (df['L7_TOTAL_DEP_FONCT_DECLARE'] != 0.0)
indices_to_update = df[mask].index
print(f"Nombre de lignes à corriger: {len(indices_to_update)}")

# Sauvegarder les valeurs originales pour analyse
cir_calcule_avant = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].copy()
l7_original = df.loc[indices_to_update, 'L7_TOTAL_DEP_FONCT_DECLARE'].copy()

# Pour chaque ligne à corriger
for idx in indices_to_update:
    # 1. Calculer correctement le montant de la ligne 7 (total dépenses de fonctionnement)
    dot_amort_immo = df.at[idx, 'L1_DOTATION_AMORT_IMMO']
    dot_amort_sinistr = df.at[idx, 'L2_DOTATION_AMORT_SINISTR']
    dep_cherch_tech = df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS']
    rem_sal_inv = df.at[idx, 'L4_REMUNERATION_INVENTEURS']
    dep_jd = df.at[idx, 'L5_DEPENSES_JEUNES_DOCTEURS']

    # Calculer L6 si nécessaire
    autres_dep_fonct = (dot_amort_immo * 0.75) + ((dep_cherch_tech + rem_sal_inv) * 0.43) + dep_jd

    # Mettre à jour L6 calculé
    df.at[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct

    # Calculer le vrai montant de L7
    total_dep_fonct = dot_amort_immo + dot_amort_sinistr + dep_cherch_tech + rem_sal_inv + dep_jd + autres_dep_fonct

    # 2. Mettre à jour L7
    df.at[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct # laisser L7 tel quel (inchangé)

    # Garder L8_FRAIS_BREVETS_COV tel quel (inchangé)
    frais_brev_cov = df.at[idx, 'L8_FRAIS_BREVETS_COV'] # mettre à 0

    # 3. Recalculer la ligne 14 (total dépenses internes)
    dep_maint_brev_cov = df.at[idx, 'L9_DEPENSES_DEFENSE_BREVETS']
    dot_amort_brev = df.at[idx, 'L10_DOTATION_AMORT_BREVETS']
    dep_normali = df.at[idx, 'L11_DEPENSES_NORMALISATION_DECLARE']
    prim_cotiz = df.at[idx, 'L12_PRIMES_COTISATIONS_PLAFONNEES']
    dep_veil_techno = df.at[idx, 'L13_VEILLE_TECHNO_PLAFONNEE']

    total_dep_internes = total_dep_fonct + frais_brev_cov + dep_maint_brev_cov + dot_amort_brev + dep_normali + prim_cotiz + dep_veil_techno
    df.at[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes

    # 4. Recalculer la ligne 22 (total dépenses de recherche)
    total_dep_ext = df.at[idx, 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']
    total_dep_recherche = total_dep_internes + total_dep_ext
    df.at[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche

    # 5. Recalculer la ligne 26A (montant net des dépenses)
    mt_aid_subv = df.at[idx, 'L23A_SUBVENTIONS']
    mt_enc_presta = df.at[idx, 'L23B_SOMMES_ENCAISSEES_TIERS']
    mt_dep_conseils_cir = df.at[idx, 'L24_DEPENSES_CONSEIL_CIR']
    rembst_subv = df.at[idx, 'L25_REMBOURSEMENTS_SUBVENTIONS']

    montant_net = total_dep_recherche - mt_aid_subv - mt_enc_presta - mt_dep_conseils_cir + rembst_subv
    df.at[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net

    # 6. Récupérer le montant DOM pour le calcul du CIR
    montant_net_dom = df.at[idx, 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE']

    # 7. Recalculer le CIR recherche (30% ou 50% selon localisation)
    cir_recherche = ((montant_net - montant_net_dom) * 0.30) + (montant_net_dom * 0.50)
    df.at[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche

    # 8. Recalculer le CIR total
    cir_collection = df.at[idx, 'CIR_COLLECTION_CALCULE']
    cir_innovation = df.at[idx, 'CIR_INNOVATION_CALCULE']
    cir_collaboratif = df.at[idx, 'CIR_COLLABORATIF_CALCULE']

    cir_total = cir_recherche + cir_collection + cir_innovation + cir_collaboratif
    df.at[idx, 'CIR_TOTAL_CALCULE'] = cir_total

    # 9. Recalculer l'écart et mettre à jour la correspondance
    ecart_cir = cir_total - df.at[idx, 'CIR_TOTAL_DECLARE']
    df.at[idx, 'CIR_TOTAL_ECART'] = ecart_cir
    df.at[idx, 'CORRESPONDANCE_CIR'] = "Oui" if abs(ecart_cir) <= 1 else "Non"

# Afficher le résumé des modifications
l7_corrige = df.loc[indices_to_update, 'L7_TOTAL_DEP_FONCT_CALCULE']
somme_declare = df.loc[indices_to_update, 'CIR_TOTAL_DECLARE'].sum()
somme_calcule_avant = cir_calcule_avant.sum()
somme_calcule_apres = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].sum()

print(f"\nRésumé des corrections :")
print(f"Somme L7_TOTAL_DEP_FONCT_DECLARE avant correction: {l7_original.sum():,.2f} €")
print(f"Somme L7_TOTAL_DEP_FONCT_CALCULE après correction: {l7_corrige.sum():,.2f} €")
print(f"Différence sur L7: {l7_corrige.sum() - l7_original.sum():,.2f} €")

print(f"\nImpact sur le CIR :")
print(f"Somme CIR_TOTAL_DECLARE: {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE avant correction: {somme_calcule_avant:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE après correction: {somme_calcule_apres:,.2f} €")
print(f"Écart avant correction: {somme_calcule_avant - somme_declare:,.2f} €")
print(f"Écart après correction: {somme_calcule_apres - somme_declare:,.2f} €")
print(f"Amélioration de l'écart: {abs(somme_calcule_avant - somme_declare) - abs(somme_calcule_apres - somme_declare):,.2f} €")

# Compter les lignes où la correspondance s'est améliorée
count_improved = 0
for idx in indices_to_update:
    if df.at[idx, 'CORRESPONDANCE_CIR'] == "Oui" and abs(cir_calcule_avant[idx] - df.at[idx, 'CIR_TOTAL_DECLARE']) > 1:
        count_improved += 1

print(f"\nNombre de lignes où la correspondance est passée de 'Non' à 'Oui': {count_improved}")

# Sauvegarder le fichier corrigé
df.to_csv("lignes_restantes_a_analyser_corrigees_L7L8.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv")

# Afficher des exemples de modifications
print("\nExemples de modifications significatives:")
# Calculer l'impact en pourcentage sur le CIR
df_impact = pd.DataFrame({
    'SIREN': df.loc[indices_to_update, 'SIREN_DECLARANT'],
    'Désignation': df.loc[indices_to_update, 'DESIGNATION'],
    'L7 avant': l7_original,
    'L7 après': l7_corrige,
    'CIR avant': cir_calcule_avant,
    'CIR après': df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'],
    'Impact': abs(df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'] - cir_calcule_avant)
})

# Afficher les 5 cas avec le plus grand impact en valeur absolue
for _, row in df_impact.sort_values('Impact', ascending=False).head(5).iterrows():
    print(f"SIREN: {row['SIREN']}")
    print(f"  Désignation: {row['Désignation']}")
    print(f"  L7 avant: {row['L7 avant']:,.2f} €")
    print(f"  L7 après: {row['L7 après']:,.2f} €")
    print(f"  CIR avant: {row['CIR avant']:,.2f} €")
    print(f"  CIR après: {row['CIR après']:,.2f} €")
    print(f"  Impact: {row['Impact']:,.2f} € ({row['Impact']/row['CIR avant']*100 if row['CIR avant'] != 0 else 0:.2f}%)")
    print()

Fichier chargé: 1721 lignes
Nombre de lignes à corriger: 0

Résumé des corrections :
Somme L7_TOTAL_DEP_FONCT_DECLARE avant correction: 0.00 €
Somme L7_TOTAL_DEP_FONCT_CALCULE après correction: 0.00 €
Différence sur L7: 0.00 €

Impact sur le CIR :
Somme CIR_TOTAL_DECLARE: 0.00 €
Somme CIR_TOTAL_CALCULE avant correction: 0.00 €
Somme CIR_TOTAL_CALCULE après correction: 0.00 €
Écart avant correction: 0.00 €
Écart après correction: 0.00 €
Amélioration de l'écart: 0.00 €

Nombre de lignes où la correspondance est passée de 'Non' à 'Oui': 0

Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv

Exemples de modifications significatives:


### ligne liée au erreur dep fonctionnement

In [8]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV
df_filtered = df[(df['L7_TOTAL_DEP_FONCT_DECLARE'] == df['L8_FRAIS_BREVETS_COV']) & (df['L7_TOTAL_DEP_FONCT_DECLARE'] != 0.0)]

print(f"Nombre de lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_L7_egal_L8.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1721 lignes
Nombre de lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV: 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_L7_egal_L8.csv


In [9]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')  # Nouveau fichier
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT']) 
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure
siren_a_exclure =  siren_jeunes_docteurs | siren_l7_l8 

# Filtrer le DataFrame principal
# - Exclure les SIREN déjà analysés
# - Garder uniquement les lignes où CORRESPONDANCE_CIR est "Non"
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) & 
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 1721
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Recherche

In [10]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("lignes_restantes_a_analyser.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres :
# - L22_TOTAL_DEPENSES_RECHERCHE_DECLARE différent de 0.0
# - CIR_RECHERCHE_DECLARE = 0.0
df_filtered = df[
    (df['L22_TOTAL_DEPENSES_RECHERCHE_DECLARE'] != 0.0) &
    (df['CIR_RECHERCHE_DECLARE'] == 0.0)
]

print(f"Nombre de lignes après filtrage: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_depenses_non0_cir_recherche_0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

# Afficher quelques statistiques sur les lignes trouvées
if len(df_filtered) > 0:
    print("\nAperçu des données trouvées:")
    print(f"Nombre d'entreprises: {len(df_filtered)}")

Fichier chargé: 1721 lignes
Nombre de lignes après filtrage: 18
Somme CIR_TOTAL_DECLARE : 65,519,439.00 €
Somme CIR_TOTAL_CALCULE : 66,542,918.10 €
Résultats exportés dans: lignes_depenses_non0_cir_recherche_0.csv

Aperçu des données trouvées:
Nombre d'entreprises: 18


In [11]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du nouveau cas)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8  | siren_depenses_non0_cir0

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 1703
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur depense Externalisée

In [12]:
import pandas as pd

# Chemin du fichier
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//lignes_restantes_a_analyser.csv"

# Charger le fichier
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Colonnes d'écart pour les dépenses externalisées
colonnes_ext = ['L17_ECART', 'L18_ECART', 'L19_ECART', 'L20_ECART', 'L21_ECART']

# Extraire les lignes qui répondent à tous les critères
mask_correspondance = df['CORRESPONDANCE_CIR'] == "Non"
mask_cir = df['CIR_RECHERCHE_ECART'] != 0
mask_ext = df[colonnes_ext].abs().sum(axis=1) > 0

# Combiner tous les filtres
df_resultat = df[mask_correspondance & mask_cir & mask_ext]

print(f"Nombre de lignes extraites: {len(df_resultat)}")

# Exporter vers un fichier CSV
output_file = "lignes_ecart_cir_recherche_externalise.csv"
df_resultat.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Lignes exportées dans: {output_file}")

Fichier chargé: 1703 lignes
Nombre de lignes extraites: 144
Lignes exportées dans: lignes_ecart_cir_recherche_externalise.csv


In [13]:
import pandas as pd
import numpy as np

# Charger le fichier CSV généré précédemment
fichier = "lignes_ecart_cir_recherche_externalise.csv"
df = pd.read_csv(fichier, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Convertir les colonnes en numérique pour s'assurer que les calculs seront corrects
df['L26A_ECART'] = pd.to_numeric(df['L26A_ECART'], errors='coerce')
df['CIR_TOTAL_ECART'] = pd.to_numeric(df['CIR_TOTAL_ECART'], errors='coerce')

# Créer une colonne pour vérifier si L26A_ECART = CIR_TOTAL_ECART / 0.3
# Permettre une petite marge d'erreur pour les arrondis (0.1%)
df['RATIO'] = df['L26A_ECART'] / (df['CIR_TOTAL_ECART'] / 0.3)
df['EST_EGAL'] = np.isclose(df['RATIO'], 1, rtol=1)  # Tolérance relative de 0.1%

# Compter combien de lignes respectent cette relation
nb_egal = df['EST_EGAL'].sum()
pourcentage = (nb_egal / len(df)) * 100

print(f"\nRésultats de l'analyse:")
print(f"- Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: {nb_egal} sur {len(df)}")
print(f"- Pourcentage: {pourcentage:.2f}%")

# Afficher quelques exemples où la relation est respectée
print("\nExemples où L26A_ECART = CIR_TOTAL_ECART / 3:")
exemples_egal = df[df['EST_EGAL']].head(5)
for _, row in exemples_egal.iterrows():
    print(f"SIREN: {row['SIREN_DECLARANT']}, L26A_ECART: {row['L26A_ECART']}, CIR_TOTAL_ECART/3: {row['CIR_TOTAL_ECART']/3}")
    # Filtrer uniquement les lignes où L26A_ECART = CIR_TOTAL_ECART / 3 est vrai
    df_resultat = df[df['EST_EGAL']]

    # Afficher le nombre de lignes correspondantes
    print(f"Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: {len(df_resultat)}")

    # Exporter les résultats filtrés dans un fichier CSV
    df_resultat.to_csv("lignes_L26A_egal_CIR_TOTAL_div_3.csv", sep=';', encoding='utf-8-sig', index=False)
    # Calculer les sommes
    somme_declare = df_resultat['CIR_TOTAL_DECLARE'].sum()
    somme_calcule = df_resultat['CIR_TOTAL_CALCULE'].sum()

    print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
    print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
    print(f"Résultats exportés dans: lignes_L26A_egal_CIR_TOTAL_div_3.csv")
# Afficher quelques exemples où la relation n'est pas respectée
print("\nExemples où L26A_ECART ≠ CIR_TOTAL_ECART / 3:")
exemples_different = df[~df['EST_EGAL']].head(5)
for _, row in exemples_different.iterrows():
    print(f"SIREN: {row['SIREN_DECLARANT']}, L26A_ECART: {row['L26A_ECART']}, CIR_TOTAL_ECART/3: {row['CIR_TOTAL_ECART']/3}, Ratio: {row['RATIO']:.4f}")

# Exporter les résultats avec la colonne d'analyse supplémentaire
df.to_csv("analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nRésultats détaillés exportés dans: analyse_L26A_vs_CIR_TOTAL.csv")

Fichier chargé: 144 lignes

Résultats de l'analyse:
- Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 127 sur 144
- Pourcentage: 88.19%

Exemples où L26A_ECART = CIR_TOTAL_ECART / 3:
SIREN: 60502291.0, L26A_ECART: -436289.0, CIR_TOTAL_ECART/3: -43629.0
Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 127
Somme CIR_TOTAL_DECLARE : 64,076,281.00 €
Somme CIR_TOTAL_CALCULE : 61,614,223.96 €
Résultats exportés dans: lignes_L26A_egal_CIR_TOTAL_div_3.csv
SIREN: 300702305.0, L26A_ECART: -1999999.63, CIR_TOTAL_ECART/3: -199999.96333333338
Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 127
Somme CIR_TOTAL_DECLARE : 64,076,281.00 €
Somme CIR_TOTAL_CALCULE : 61,614,223.96 €
Résultats exportés dans: lignes_L26A_egal_CIR_TOTAL_div_3.csv
SIREN: 301165841.0, L26A_ECART: -560734.74, CIR_TOTAL_ECART/3: -56073.23999999999
Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 127
Somme CIR_TOTAL_DECLARE : 64,076,281.00 €
Somme CIR_TOTAL_CALCULE : 61,614,223.96 €
Résultats exportés dans: 

In [14]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas analyse_L26A_vs_CIR_TOTAL.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 1559
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur doublement sans motif de la créance total

In [15]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer selon les critères :
# - ECART_RELATIF_POURCENT == -50 ou 50
# - CIR_INNOVATION_ECART == 0
# - CIR_RECHERCHE_ECART entre -1 et 1 inclus
mask = (
    df['ECART_RELATIF_POURCENT'].isin([-50.0, 50.0]) &
    (df['CIR_INNOVATION_ECART'] == 0.0) &
    (df['CIR_RECHERCHE_ECART'].between(-1, 1))
)

df_filtered = df[mask]
print(f"Lignes extraites: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_ecart_relatif_50_innovation0_recherche1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1559 lignes
Lignes extraites: 7
Somme CIR_TOTAL_DECLARE : 1,847,283.00 €
Somme CIR_TOTAL_CALCULE : 923,641.52 €
Résultats exportés dans: lignes_ecart_relatif_50_innovation0_recherche1.csv


In [16]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas analyse_L26A_vs_CIR_TOTAL.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 1552
Résultats exportés dans: lignes_restantes_a_analyser.csv


## erreur diff Cir total compris entre -500 et 500

In [17]:
import pandas as pd

# Charger le fichier des lignes restantes à analyser
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART est entre -500 et 500 et CORRESPONDANCE_CIR == "Non"
mask_ecart = df['CIR_TOTAL_ECART'].between(-500, 500) & df['CORRESPONDANCE_CIR'].eq("Non")
df_resultat = df[mask_ecart]

print(f"Nombre de lignes extraites: {len(df_resultat)}")

# Exporter les résultats
output_file = "lignes_ecart_total_entre_-500_et_500.csv"
df_resultat.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_resultat['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_resultat['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1552 lignes
Nombre de lignes extraites: 289
Somme CIR_TOTAL_DECLARE : 65,700,751.00 €
Somme CIR_TOTAL_CALCULE : 65,702,853.69 €
Résultats exportés dans: lignes_ecart_total_entre_-500_et_500.csv


In [18]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas lignes_ecart_total_entre_-500_et_500.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | siren_ecart_total500

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 1263
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Plafond Partout

In [19]:
import pandas as pd

# Charger le fichier avec les 703 lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où ECART_RELATIF_POURCENT est égal à -100
df_ecart_negatif = df[df['ECART_RELATIF_POURCENT'] == -100]
print(f"Nombre de lignes avec ECART_RELATIF_POURCENT = -100: {len(df_ecart_negatif)}")

# Exporter les résultats dans un fichier CSV
output_file = "lignes_ecart_relatif_moins_100.csv"
df_ecart_negatif.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_ecart_negatif['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_ecart_negatif['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1263 lignes
Nombre de lignes avec ECART_RELATIF_POURCENT = -100: 318
Somme CIR_TOTAL_DECLARE : 56,716,432.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_ecart_relatif_moins_100.csv


In [20]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas lignes_ecart_relatif_moins_100.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 945
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir innovation

In [21]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_INNOVATION_ECART'], atol=1.0)
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print("\nExemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"SIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_INNOVATION_ECART: {row['CIR_INNOVATION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_innovation.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 945 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): 204

Exemples :
SIREN: 301468211.0
CIR_TOTAL_ECART: 33729.64
CIR_INNOVATION_ECART: 33730.00
SIREN: 310863774.0
CIR_TOTAL_ECART: -46134.00
CIR_INNOVATION_ECART: -46134.00
SIREN: 311243067.0
CIR_TOTAL_ECART: 5465.40
CIR_INNOVATION_ECART: 5465.70
SIREN: 312940224.0
CIR_TOTAL_ECART: 40000.00
CIR_INNOVATION_ECART: 40000.00
SIREN: 316742980.0
CIR_TOTAL_ECART: 3981.88
CIR_INNOVATION_ECART: 3981.90
Somme CIR_TOTAL_DECLARE : 14,528,954.00 €
Somme CIR_TOTAL_CALCULE : 16,166,757.67 €

Résultats exportés dans: lignes_ecart_total_egal_innovation.csv


In [22]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas innovation_egal_total)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 740
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Collection

In [23]:
import pandas as pd
import numpy as np

# Charger le fichier
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Créer le masque pour identifier où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_COLLECTION_ECART'], atol=1.0)

# Filtrer les lignes
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1): {len(df_filtered)}")

# Afficher quelques statistiques sur ces lignes
if len(df_filtered) > 0:
    print("\nQuelques exemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"\nSIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_COLLECTION_ECART: {row['CIR_COLLECTION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_collection.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 740 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1): 12

Quelques exemples :

SIREN: 302306626.0
CIR_TOTAL_ECART: -94531.00
CIR_COLLECTION_ECART: -94531.00

SIREN: 322106873.0
CIR_TOTAL_ECART: -5567.10
CIR_COLLECTION_ECART: -5567.00

SIREN: 342973484.0
CIR_TOTAL_ECART: -40544.18
CIR_COLLECTION_ECART: -40544.00

SIREN: 344894985.0
CIR_TOTAL_ECART: 158947.00
CIR_COLLECTION_ECART: 158947.50

SIREN: 382456051.0
CIR_TOTAL_ECART: -16856.00
CIR_COLLECTION_ECART: -16856.00
Somme CIR_TOTAL_DECLARE : 1,428,913.00 €
Somme CIR_TOTAL_CALCULE : 1,556,337.42 €

Résultats exportés dans: lignes_ecart_total_egal_collection.csv


In [24]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas collection_egal_total)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 728
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Déclarer vaut 0 alors que sa ne l'est pas

In [25]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0
df_filtered = df[(df['ECART_RELATIF_POURCENT'] == 100) & (df['CIR_TOTAL_DECLARE'] == 0)]

print(f"Nombre de lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_ecart_100_cir_declare_0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 728 lignes
Nombre de lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0: 103
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 10,879,886.21 €
Résultats exportés dans: lignes_ecart_100_cir_declare_0.csv


In [26]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas ecart_100_cir_declare_0)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 624
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur liée à L6

In [27]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1)
mask_ecart = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_RECHERCHE_ECART'], atol=1.0)

# Filtrer les lignes où L6_ECART est strictement en dehors de [-1, 1]
mask_l6 = (df['L6_ECART'] < -1) | (df['L6_ECART'] > 1)

# Appliquer les deux filtres
df_filtered = df[mask_ecart & mask_l6]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1) et L6_ECART hors [-1, 1]: {len(df_filtered)}")
# Exporter les résultats
output_file = "lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 624 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1) et L6_ECART hors [-1, 1]: 44
Somme CIR_TOTAL_DECLARE : 4,535,248.00 €
Somme CIR_TOTAL_CALCULE : 4,872,529.08 €

Résultats exportés dans: lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv


In [28]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 580
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur liée à L26

In [29]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres demandés
mask = (
    ((df['L26A_ECART'] > 1) | (df['L26A_ECART'] < -1))
    & ((df['L14_ECART'] > -1) & (df['L14_ECART'] < 1))
    & ((df['L20_ECART'] > -1) & (df['L20_ECART'] < 1))
    & ((df['L21_ECART'] > -1) & (df['L21_ECART'] < 1))
    & ((df['L22_ECART'] > -1) & (df['L22_ECART'] < 1))
)

df_filtered = df[mask]
print(f"Nombre de lignes correspondant au cas demandé : {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print(df_filtered[['SIREN_DECLARANT', 'L26A_ECART', 'L14_ECART', 'L20_ECART', 'L21_ECART']].head())

# Exporter les résultats
output_file = "lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 580 lignes
Nombre de lignes correspondant au cas demandé : 3
     SIREN_DECLARANT  L26A_ECART  L14_ECART  L20_ECART  L21_ECART
437      830552808.0   122840.19       0.19        0.0        0.0
465      844724021.0   -53680.00       0.00        0.0        0.0
532      898441894.0   138123.45       0.45        0.0        0.0
Somme CIR_TOTAL_DECLARE : 167,892.00 €
Somme CIR_TOTAL_CALCULE : 215,086.50 €
Résultats exportés dans: lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv


In [30]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')
cas_l26a = pd.read_csv("lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])
siren_cas_l26a = set(cas_l26a['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1 | siren_cas_l26a

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 577
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur oublie CRC dans Cir Total Déclarer

In [31]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où les écarts CIR (recherche, collection, innovation, collaboratif) sont tous entre -1 et 1
mask_ecarts = (
    df['CIR_RECHERCHE_ECART'].between(-1, 1) &
    df['CIR_COLLECTION_ECART'].between(-1, 1) &
    df['CIR_INNOVATION_ECART'].between(-1, 1) &
    df['CIR_COLLABORATIF_ECART'].between(-1, 1)
)

# Filtrer les lignes où CIR_TOTAL_ECART = L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE (à ±1 près)
if 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE' in df.columns:
    mask_l91 = np.isclose(df['CIR_TOTAL_ECART'], df['L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE'], atol=1.0)
else:
    mask_l91 = pd.Series([False]*len(df), index=df.index)
    print("Colonne L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE absente du fichier.")

# Appliquer les deux filtres
df_filtered = df[mask_ecarts & mask_l91]

print(f"Nombre de lignes correspondant au cas demandé : {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print(df_filtered[['SIREN_DECLARANT', 'CIR_TOTAL_ECART', 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE']].head())

# Exporter les résultats
output_file = "lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 577 lignes
Colonne L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE absente du fichier.
Nombre de lignes correspondant au cas demandé : 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv


### mise à jour fichier lignes restante

In [32]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')
cas_l26a = pd.read_csv("lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv", sep=';', encoding='utf-8-sig')
ecarts_cir_l91 = pd.read_csv("lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])
siren_cas_l26a = set(cas_l26a['SIREN_DECLARANT'])
siren_ecarts_cir_l91 = set(ecarts_cir_l91['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1 | siren_cas_l26a | siren_ecarts_cir_l91

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 27923 lignes

Nombre de lignes restantes à analyser: 577
Résultats exportés dans: lignes_restantes_a_analyser.csv


In [33]:
import pandas as pd
# On suppose que df_resultat (issu de comparer_cir_declare_recalcule) contient la colonne 'CIR_TOTAL_CALCULE'
# et que le fichier traité dans la cellule 3 est "C://Users//msamb//Documents//new_millesime_CIR_2023_avec_deposants_ajoutes.xlsx"

# Charger le fichier de comparaison détaillée si besoin
df_resultat = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv", sep=';', encoding='utf-8-sig')

# Recherche des entreprises ayant un CIR calculé identique
cir_counts = df_resultat['CIR_TOTAL_CALCULE'].value_counts()
cir_duplicated = cir_counts[cir_counts > 1].index

# Filtrer les entreprises concernées
df_cir_identique = df_resultat[df_resultat['CIR_TOTAL_CALCULE'].isin(cir_duplicated)]
df_cir_identique.to_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output-specifique(2023)//entreprises_cir_identique.csv", sep=';', encoding='utf-8-sig', index=False)

print(f"Nombre d'entreprises ayant un CIR calculé identique à au moins une autre: {len(df_cir_identique)}")
print("Exemples d'entreprises avec CIR calculé identique :")
display(df_cir_identique[['SIREN_DECLARANT', 'DESIGNATION', 'CIR_TOTAL_CALCULE']].sort_values('CIR_TOTAL_CALCULE'))

Nombre d'entreprises ayant un CIR calculé identique à au moins une autre: 2115
Exemples d'entreprises avec CIR calculé identique :


,SIREN_DECLARANT,DESIGNATION,CIR_TOTAL_CALCULE
27306,919557447.0,NaN,-0.07
23765,881023683.0,NaN,-0.07
14122,662053388.0,AU LIEGEUR,0.00
14107,662006170.0,PIERRE FABRE SA,0.00
14097,655780088.0,CONTINENTAL HOLDING FRANCE SAS,0.00
...,...,...,...
6938,438277030.0,NaN,2927653.15
43,56501711.0,NaN,5071017.29
12335,525009809.0,NaN,5071017.29
13478,542005376.0,FAURECIA SIEGES D'AUTOMOBILE,16270413.37


# Correction Creance

In [34]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ===================================================================
# FONCTIONS UTILITAIRES
# ===================================================================

def format_siren(siren):
    """Formate les SIREN pour avoir exactement 9 caractères"""
    if pd.isna(siren):
        return siren
    siren_str = str(siren).strip()
    # Gère les formats type '123456789.0'
    if '.' in siren_str:
        siren_str = siren_str.split('.')[0]
    return siren_str.zfill(9)

def convertir_en_nombre(valeur, defaut=0.0):
    """Convertit une valeur en nombre flottant de façon sécurisée"""
    if pd.isna(valeur):
        return defaut
    valeur_str = str(valeur).strip()
    if valeur_str == '':
        return defaut
    try:
        # Remplace la virgule par un point pour la conversion décimale
        return float(valeur_str.replace(',', '.'))
    except ValueError:
        # Retourne la valeur par défaut en cas d'erreur de conversion
        return defaut
    except Exception:
         # Capture d'autres erreurs potentielles
        return defaut

# ===================================================================
# CHARGEMENT ET PRÉPARATION DES DONNÉES
# ===================================================================

def charger_donnees(file_path="Calcul_Creance_CIR_2023.csv"):
    """Charge et prépare les données du fichier CIR"""
    print(f"Chargement et préparation des données depuis: {file_path}...")
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                         converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str},
                         low_memory=False)
        print(f"Fichier principal chargé: {len(df):,} lignes")
    except FileNotFoundError:
        print(f"ERREUR: Fichier non trouvé: {file_path}.")
        return None
    except Exception as e:
        print(f"ERREUR lors du chargement du fichier principal: {str(e)}")
        return None

    print("Formatage des SIRENs...")
    if 'SIREN_DECLARANT' in df.columns:
        df['SIREN_DECLARANT'] = df['SIREN_DECLARANT'].apply(format_siren)
    else:
        print("ERREUR CRITIQUE: Colonne 'SIREN_DECLARANT' non trouvée.")
        return None
    if 'SIREN_DEPOSANT' in df.columns:
        df['SIREN_DEPOSANT'] = df['SIREN_DEPOSANT'].apply(format_siren)

    if 'TRAITEMENT_APPLIQUE' not in df.columns:
        df['TRAITEMENT_APPLIQUE'] = "Non traité"
    else:
         # Assure que la colonne est de type string
         df['TRAITEMENT_APPLIQUE'] = df['TRAITEMENT_APPLIQUE'].astype(str)

    # Liste exhaustive des colonnes numériques à convertir
    numeric_cols = [
        'L1_DOTATION_AMORT_IMMO', 'L2_DOTATION_AMORT_SINISTR', 'L3_DEPENSES_PERSONNEL_CHERCHEURS',
        'L4_REMUNERATION_INVENTEURS', 'L5_DEPENSES_JEUNES_DOCTEURS', 'L6_AUTRES_DEP_FONCT_CALCULE',
        'L7_TOTAL_DEP_FONCT_DECLARE', 'L7_TOTAL_DEP_FONCT_CALCULE', 'L8_FRAIS_BREVETS_COV',
        'L9_DEPENSES_DEFENSE_BREVETS', 'L10_DOTATION_AMORT_BREVETS', 'L11_DEPENSES_NORMALISATION_DECLARE',
        'L12_PRIMES_COTISATIONS_BRUT', 'L12_PRIMES_COTISATIONS_PLAFONNEES', 'L13_VEILLE_TECHNO_BRUT',
        'L13_VEILLE_TECHNO_PLAFONNEE', 'L14_TOTAL_DEPENSES_INTERNES_CALCULE', 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE',
        'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE', 'L23A_SUBVENTIONS', 'L23B_SOMMES_ENCAISSEES_TIERS',
        'L24_DEPENSES_CONSEIL_CIR', 'L25_REMBOURSEMENTS_SUBVENTIONS', 'L26A_MONTANT_NET_DEPENSES_CALCULE',
        'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE', 'CIR_RECHERCHE_CALCULE', 'CIR_COLLECTION_CALCULE',
        'CIR_INNOVATION_CALCULE', 'CIR_COLLABORATIF_CALCULE', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE',
        'CIR_TOTAL_ECART', 'ECART_RELATIF_POURCENT', 'CIR_RECHERCHE_ECART', 'CIR_INNOVATION_ECART',
        'CIR_COLLECTION_ECART', 'CIR_COLLABORATIF_ECART', 'L6_ECART', 'L14_ECART', 'L17_ECART', 'L18_ECART',
        'L19_ECART', 'L20_ECART', 'L21_ECART', 'L22_ECART', 'L26A_ECART', 'L26B_ECART',
        'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE', 'CIR_TOTAL_CORRIGE' # Inclure si existe déjà
    ]

    print("Conversion des colonnes numériques...")
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertir_en_nombre)

    # Initialisation de CIR_TOTAL_CORRIGE après les conversions
    if 'CIR_TOTAL_CORRIGE' not in df.columns:
        print("Initialisation de la colonne 'CIR_TOTAL_CORRIGE'...")
        if 'CIR_TOTAL_CALCULE' in df.columns:
            df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CALCULE'].copy()
        else:
            print("  Attention: 'CIR_TOTAL_CALCULE' non trouvé. 'CIR_TOTAL_CORRIGE' initialisé à 0.0.")
            df['CIR_TOTAL_CORRIGE'] = 0.0
    else:
        # Assurer qu'elle est bien numérique si elle existait
        df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)
        print("La colonne 'CIR_TOTAL_CORRIGE' existait déjà et a été (re)convertie en numérique.")

    # Vérification finale des colonnes essentielles
    essential_cols = ['SIREN_DECLARANT', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
    for col in essential_cols:
        if col not in df.columns:
            print(f"ERREUR CRITIQUE: Colonne essentielle '{col}' est manquante après chargement/préparation.")
            return None

    print("Préparation des données terminée.")
    return df

def charger_fichier_intermediaire(file_path, main_df_columns):
    """Charge un fichier intermédiaire, formate SIREN et convertit num."""
    try:
        df_inter = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                               converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str},
                               low_memory=False)
        if 'SIREN_DECLARANT' in df_inter.columns:
            df_inter['SIREN_DECLARANT'] = df_inter['SIREN_DECLARANT'].apply(format_siren)
        else:
             print(f"Attention: 'SIREN_DECLARANT' manquant dans '{file_path}'.")
             return pd.DataFrame(columns=main_df_columns)

        numeric_cols_inter = [
            'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_ECART',
            'CIR_RECHERCHE_ECART', 'CIR_COLLECTION_ECART', 'CIR_INNOVATION_ECART', 'CIR_COLLABORATIF_CALCULE',
             'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE'
        ]
        for col in numeric_cols_inter:
            if col in df_inter.columns:
                df_inter[col] = df_inter[col].apply(convertir_en_nombre)
        if 'EST_EGAL' in df_inter.columns:
             df_inter['EST_EGAL'] = df_inter['EST_EGAL'].apply(lambda x: str(x).strip().lower() == 'true')
        return df_inter
    except FileNotFoundError:
        print(f"Info: Fichier intermédiaire '{file_path}' non trouvé.")
        return pd.DataFrame(columns=main_df_columns)
    except Exception as e:
        print(f"Erreur chargement fichier intermédiaire '{file_path}': {str(e)}")
        return pd.DataFrame(columns=main_df_columns)

# ===================================================================
# FONCTIONS DE CORRECTION (1 à 12 Inchangées, 13 Modifiée, 14 Supprimée, 15 Inchangée)
# ===================================================================

# --- Fonctions 1 à 12 : Code simplifié pour la longueur, mais identique à la version précédente ---
def correction_erreurs_personnel(df, processed_sirens, fichier_intermediaire="lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"):
    """Corrige les erreurs de dépenses de personnel (jeunes docteurs)."""
    print("\n1. CORRECTION ERREURS DÉPENSES PERSONNEL (JEUNES DOCTEURS)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        l1_val=convertir_en_nombre(df.loc[idx,'L1_DOTATION_AMORT_IMMO']); l2_val=convertir_en_nombre(df.loc[idx,'L2_DOTATION_AMORT_SINISTR'])
        l4_val=convertir_en_nombre(df.loc[idx,'L4_REMUNERATION_INVENTEURS']); l5_val=convertir_en_nombre(df.loc[idx,'L5_DEPENSES_JEUNES_DOCTEURS'])
        l8_val=convertir_en_nombre(df.loc[idx,'L8_FRAIS_BREVETS_COV']); l9_val=convertir_en_nombre(df.loc[idx,'L9_DEPENSES_DEFENSE_BREVETS'])
        l10_val=convertir_en_nombre(df.loc[idx,'L10_DOTATION_AMORT_BREVETS']); l11_val=convertir_en_nombre(df.loc[idx,'L11_DEPENSES_NORMALISATION_DECLARE'])
        l12_brut_val=convertir_en_nombre(df.loc[idx,'L12_PRIMES_COTISATIONS_BRUT']); l13_brut_val=convertir_en_nombre(df.loc[idx,'L13_VEILLE_TECHNO_BRUT'])
        l21_calcule_val=convertir_en_nombre(df.loc[idx,'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']); l23a_val=convertir_en_nombre(df.loc[idx,'L23A_SUBVENTIONS'])
        l23b_val=convertir_en_nombre(df.loc[idx,'L23B_SOMMES_ENCAISSEES_TIERS']); l24_val=convertir_en_nombre(df.loc[idx,'L24_DEPENSES_CONSEIL_CIR'])
        l25_val=convertir_en_nombre(df.loc[idx,'L25_REMBOURSEMENTS_SUBVENTIONS']); cir_coll_val=convertir_en_nombre(df.loc[idx,'CIR_COLLECTION_CALCULE'])
        cir_inno_val=convertir_en_nombre(df.loc[idx,'CIR_INNOVATION_CALCULE']); cir_collab_val=convertir_en_nombre(df.loc[idx,'CIR_COLLABORATIF_CALCULE'])
        l26b_val=convertir_en_nombre(df.loc[idx,'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE'])
        df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']=l2_val; df.loc[idx,'L2_DOTATION_AMORT_SINISTR']=0.0
        dep_cherch_tech_updated=df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']; l2_updated=df.loc[idx,'L2_DOTATION_AMORT_SINISTR']
        autres_dep_fonct=(l1_val*0.75)+((dep_cherch_tech_updated+l4_val)*0.43)+l5_val; df.loc[idx,'L6_AUTRES_DEP_FONCT_CALCULE']=autres_dep_fonct
        total_dep_fonct=l1_val+l2_updated+dep_cherch_tech_updated+l4_val+l5_val+autres_dep_fonct; df.loc[idx,'L7_TOTAL_DEP_FONCT_CALCULE']=total_dep_fonct
        prim_cotiz_plaf=min(l12_brut_val,60000); dep_veil_techno_plaf=min(l13_brut_val,60000)
        total_dep_internes=total_dep_fonct+l8_val+l9_val+l10_val+l11_val+prim_cotiz_plaf+dep_veil_techno_plaf; df.loc[idx,'L14_TOTAL_DEPENSES_INTERNES_CALCULE']=total_dep_internes
        total_dep_recherche=total_dep_internes+l21_calcule_val; df.loc[idx,'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE']=total_dep_recherche
        montant_net=total_dep_recherche-l23a_val-l23b_val-l24_val+l25_val; df.loc[idx,'L26A_MONTANT_NET_DEPENSES_CALCULE']=montant_net
        taux_dom,taux_non_dom=0.50,0.30
        if 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns and l26b_val > 0 :
             montant_net_non_dom = max(0, montant_net - l26b_val); cir_recherche = (montant_net_non_dom * taux_non_dom) + (l26b_val * taux_dom)
        else: cir_recherche = montant_net * taux_non_dom
        cir_recherche=max(0, cir_recherche); df.loc[idx,'CIR_RECHERCHE_CALCULE']=cir_recherche
        cir_total=cir_recherche+cir_coll_val+cir_inno_val+cir_collab_val; df.loc[idx,'CIR_TOTAL_CALCULE']=cir_total
        df.loc[idx,'CIR_TOTAL_CORRIGE']=cir_total; df.loc[idx,'TRAITEMENT_APPLIQUE']="Err_dep_personnel"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count+=1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_frais_brevets(df, processed_sirens, fichier_intermediaire="lignes_L7_egal_L8.csv"):
    """Corrige les erreurs où L7 (déclaré) est égal à L8."""
    print("\n2. CORRECTION ERREUR L7 = L8 (FRAIS BREVETS)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        l1_val=convertir_en_nombre(df.loc[idx,'L1_DOTATION_AMORT_IMMO']); l2_val=convertir_en_nombre(df.loc[idx,'L2_DOTATION_AMORT_SINISTR'])
        l3_val=convertir_en_nombre(df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']); l4_val=convertir_en_nombre(df.loc[idx,'L4_REMUNERATION_INVENTEURS'])
        l5_val=convertir_en_nombre(df.loc[idx,'L5_DEPENSES_JEUNES_DOCTEURS']); l9_val=convertir_en_nombre(df.loc[idx,'L9_DEPENSES_DEFENSE_BREVETS'])
        l10_val=convertir_en_nombre(df.loc[idx,'L10_DOTATION_AMORT_BREVETS']); l11_val=convertir_en_nombre(df.loc[idx,'L11_DEPENSES_NORMALISATION_DECLARE'])
        l12_plaf_val=convertir_en_nombre(df.loc[idx,'L12_PRIMES_COTISATIONS_PLAFONNEES']); l13_plaf_val=convertir_en_nombre(df.loc[idx,'L13_VEILLE_TECHNO_PLAFONNEE'])
        l21_calcule_val=convertir_en_nombre(df.loc[idx,'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']); l23a_val=convertir_en_nombre(df.loc[idx,'L23A_SUBVENTIONS'])
        l23b_val=convertir_en_nombre(df.loc[idx,'L23B_SOMMES_ENCAISSEES_TIERS']); l24_val=convertir_en_nombre(df.loc[idx,'L24_DEPENSES_CONSEIL_CIR'])
        l25_val=convertir_en_nombre(df.loc[idx,'L25_REMBOURSEMENTS_SUBVENTIONS']); cir_coll_val=convertir_en_nombre(df.loc[idx,'CIR_COLLECTION_CALCULE'])
        cir_inno_val=convertir_en_nombre(df.loc[idx,'CIR_INNOVATION_CALCULE']); cir_collab_val=convertir_en_nombre(df.loc[idx,'CIR_COLLABORATIF_CALCULE'])
        l26b_val=convertir_en_nombre(df.loc[idx,'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE'])
        df.loc[idx, 'L8_FRAIS_BREVETS_COV'] = 0.0; l8_corrected_val = 0.0
        autres_dep_fonct = (l1_val*0.75) + ((l3_val + l4_val)*0.43) + l5_val; df.loc[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct
        total_dep_fonct = l1_val+l2_val+l3_val+l4_val+l5_val+autres_dep_fonct; df.loc[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct
        total_dep_internes = total_dep_fonct + l8_corrected_val + l9_val + l10_val + l11_val + l12_plaf_val + l13_plaf_val
        df.loc[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes
        total_dep_recherche = total_dep_internes + l21_calcule_val; df.loc[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche
        montant_net = total_dep_recherche - l23a_val - l23b_val - l24_val + l25_val; df.loc[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net
        taux_dom, taux_non_dom = 0.50, 0.30
        if 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns and l26b_val > 0 :
             montant_net_non_dom = max(0, montant_net - l26b_val); cir_recherche = (montant_net_non_dom * taux_non_dom) + (l26b_val * taux_dom)
        else: cir_recherche = montant_net * taux_non_dom
        cir_recherche = max(0, cir_recherche); df.loc[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche
        cir_total = cir_recherche + cir_coll_val + cir_inno_val + cir_collab_val; df.loc[idx, 'CIR_TOTAL_CALCULE'] = cir_total
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = cir_total; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_Frais_BRV"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_recherche_manquant(df, processed_sirens, fichier_intermediaire="lignes_depenses_non0_cir_recherche_0.csv"):
    """Corrige les cas où le CIR recherche déclaré est manquant ou incohérent."""
    print("\n3. CORRECTION CIR RECHERCHE MANQUANT/INCOHÉRENT")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_CIR_RECH"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_depenses_externalisees(df, processed_sirens, fichier_intermediaire="analyse_L26A_vs_CIR_TOTAL.csv"):
    """Corrige les erreurs liées aux dépenses externalisées (basé sur L26A vs CIR Total)."""
    print("\n4. CORRECTION ERREURS DÉPENSES EXTERNALISÉES (L26A vs CIR)")
    df_error_raw = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error_raw.empty: return 0
    df_error = df_error_raw
    if 'EST_EGAL' in df_error.columns: df_error = df_error_raw[df_error_raw['EST_EGAL']].copy()
    else: print(f"Attention: Colonne 'EST_EGAL' non trouvée dans '{fichier_intermediaire}'.")
    if df_error.empty: print(f"Aucune donnée pertinente (EST_EGAL=True) trouvée. Skipping."); return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_dep_ext_plaf"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues (EST_EGAL=True), {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_doublement_sans_motif(df, processed_sirens, fichier_intermediaire="lignes_ecart_relatif_50_innovation0_recherche1.csv"):
    """Corrige les erreurs de doublement apparent (écart relatif ~50%). Prend la moyenne."""
    print("\n5. CORRECTION DOUBLEMENT SANS MOTIF (ÉCART ~50%)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']; calculated = df.loc[idx, 'CIR_TOTAL_CALCULE']
        corrected = calculated if abs(declared) < 0.01 else (declared + calculated) / 2
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = corrected; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_dbl_creance_totale"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_ecart_petit(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_entre_-500_et_500.csv"):
    """Corrige les petits écarts (-500 à 500 EUR) en prenant la moyenne."""
    print("\n6. CORRECTION PETIT ÉCART (-500 à 500 EUR)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']; calculated = df.loc[idx, 'CIR_TOTAL_CALCULE']
        corrected = (declared + calculated) / 2
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = corrected; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_petite"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_plafond_partout(df, processed_sirens, fichier_intermediaire="lignes_ecart_relatif_moins_100.csv"):
    """Corrige les cas où l'écart est -100% (calculé=0), utilise la valeur calculée (0)."""
    print("\n7. CORRECTION PLAFOND PARTOUT (ÉCART -100%)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE'] # Should be ~0
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_PLAF_EVW"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_innovation(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_innovation.csv"):
    """Corrige les cas où l'écart total correspond au CIR Innovation calculé."""
    print("\n8. CORRECTION CIR INNOVATION MANQUANT (ÉCART = INNOVATION)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "innov_manquant"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_collection(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_collection.csv"):
    """Corrige les cas où l'écart total correspond au CIR Collection calculé."""
    print("\n9. CORRECTION CIR COLLECTION MANQUANT (ÉCART = COLLECTION)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "collect_manquant"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_declare_zero(df, processed_sirens, fichier_intermediaire="lignes_ecart_100_cir_declare_0.csv"):
    """Corrige les cas où le CIR déclaré est 0 mais le calculé est positif."""
    print("\n10. CORRECTION CIR DÉCLARÉ = 0 (ÉCART +100%)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = 0.0 # Déclaré est 0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_cir_total_declarer"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_l6(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"):
    """Corrige les erreurs liées à la ligne L6 (Autres dépenses de fonctionnement)."""
    print("\n11. CORRECTION ERREURS LIÉES À L6")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_l6"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_l26(df, processed_sirens, fichier_intermediaire="lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv"):
    """Corrige les erreurs liées à la ligne L26 (Montant net des dépenses)."""
    print("\n12. CORRECTION ERREURS LIÉES À L26")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_l26"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_crc_manquant_cir_total(df, processed_sirens, fichier_intermediaire="lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv"):
    """Corrige les erreurs où CRC est manquant dans le CIR total (cas spécifique Total=L91)."""
    print("\n13. CORRECTION OUBLI CRC (Cas spécifique Total=L91)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    if 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE' not in df.columns:
        print("Colonne 'L91...' manquante. Skipping correction 13.")
        return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']
        crc = df.loc[idx, 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = declared + crc
        df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_oubli_crc"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_valeur_calculee_reste(df, processed_sirens, fichier_intermediaire="lignes_restantes_a_analyser.csv"):
    """Applique la valeur calculée aux cas restants identifiés."""
    print("\n15. VALEUR CALCULÉE PRIVILÉGIÉE POUR LE RESTE IDENTIFIÉ")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value
        df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Valeur calculée privilégiée(Reste)"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

# ===================================================================
# FONCTION PRINCIPALE (Orchestration - Modifiée)
# ===================================================================

def corriger_cir(file_path="Calcul_Creance_CIR_2023.csv"):
    """Fonction principale pour charger, corriger et sauvegarder le CIR."""
    df = charger_donnees(file_path)
    if df is None:
        print("\nArrêt du traitement.")
        return None

    original_declared = df['CIR_TOTAL_DECLARE'].sum()
    original_calculated = df['CIR_TOTAL_CALCULE'].sum()
    initial_corrected_sum = df['CIR_TOTAL_CORRIGE'].sum()
    print(f"\nSommes avant corrections spécifiques:")
    print(f"  - CIR Déclaré Total: {original_declared:,.2f} €")
    print(f"  - CIR Calculé Total: {original_calculated:,.2f} €")
    print(f"  - CIR Corrigé Initial Total: {initial_corrected_sum:,.2f} €")

    processed_sirens = set()
    total_corrected_count = 0

    print("\n--- Début de l'application des corrections spécifiques ---")
    # Liste des fonctions de correction et de leurs fichiers associés
    correction_definitions = [
        (correction_erreurs_personnel, "lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"), #1
        (correction_erreurs_frais_brevets, "lignes_L7_egal_L8.csv"), #2
        (correction_cir_recherche_manquant, "lignes_depenses_non0_cir_recherche_0.csv"), #3
        (correction_erreurs_depenses_externalisees, "analyse_L26A_vs_CIR_TOTAL.csv"), #4
        (correction_doublement_sans_motif, "lignes_ecart_relatif_50_innovation0_recherche1.csv"), #5
        (correction_ecart_petit, "lignes_ecart_total_entre_-500_et_500.csv"), #6
        (correction_plafond_partout, "lignes_ecart_relatif_moins_100.csv"), #7
        (correction_cir_innovation, "lignes_ecart_total_egal_innovation.csv"), #8
        (correction_cir_collection, "lignes_ecart_total_egal_collection.csv"), #9
        (correction_cir_declare_zero, "lignes_ecart_100_cir_declare_0.csv"), #10
        (correction_erreurs_l6, "lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"), #11
        (correction_erreurs_l26, "lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv"), #12
        (correction_crc_manquant_cir_total, "lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv"), #13
        (correction_valeur_calculee_reste, "lignes_restantes_a_analyser.csv") #15
    ]

    # Exécution séquentielle des corrections
    for func, file in correction_definitions:
        count = func(df, processed_sirens, file)
        total_corrected_count += count

    print(f"\n--- Fin de l'application des corrections spécifiques ---")
    print(f"Nombre total de modifications appliquées par les fonctions actives: {total_corrected_count:,}")
    print(f"Nombre de SIRENs uniques traités: {len(processed_sirens):,}")

    # --- Ajustement final : Mise à zéro des CIR corrigés négatifs ---
    print("\n--- Ajustement final : Mise à zéro des CIR corrigés négatifs ---")
    if 'CIR_TOTAL_CORRIGE' in df.columns:
        # Assurer le type numérique avant la comparaison
        if not pd.api.types.is_numeric_dtype(df['CIR_TOTAL_CORRIGE']):
             df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)

        negative_mask = df['CIR_TOTAL_CORRIGE'] < 0
        count_negative = negative_mask.sum()
        if count_negative > 0:
            sum_before_zeroing = df.loc[negative_mask, 'CIR_TOTAL_CORRIGE'].sum()
            df.loc[negative_mask, 'CIR_TOTAL_CORRIGE'] = 0.0
            print(f"{count_negative:,} lignes avec CIR_TOTAL_CORRIGE négatif ont été mises à 0 (Impact: {-sum_before_zeroing:,.2f} €).")
        else: print("Aucune valeur négative trouvée dans CIR_TOTAL_CORRIGE après corrections.")
    else: print("Erreur: Colonne 'CIR_TOTAL_CORRIGE' non trouvée pour l'ajustement final.")

    # --- Finalisation des lignes 'Non traité' ---
    mask_non_traite_final = df['TRAITEMENT_APPLIQUE'] == "Non traité"
    count_non_traite = mask_non_traite_final.sum()
    print(f"\nConfirmation pour les lignes restées 'Non traité' ({count_non_traite:,}):")
    if count_non_traite > 0:
        # Assurer numérique et non-négatif
        df.loc[mask_non_traite_final, 'CIR_TOTAL_CORRIGE'] = df.loc[mask_non_traite_final, 'CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)
        neg_in_non_traite_mask = mask_non_traite_final & (df['CIR_TOTAL_CORRIGE'] < 0)
        count_neg_in_non_traite = neg_in_non_traite_mask.sum()
        if count_neg_in_non_traite > 0:
             sum_neg_non_traite = df.loc[neg_in_non_traite_mask, 'CIR_TOTAL_CORRIGE'].sum()
             df.loc[neg_in_non_traite_mask, 'CIR_TOTAL_CORRIGE'] = 0.0
             print(f"  - {count_neg_in_non_traite} de ces lignes étaient négatives et mises à 0 (impact: {-sum_neg_non_traite:,.2f} €).")
        else: print("  - Aucune valeur négative parmi les lignes 'Non traité'.")
    else: print("  - Aucune ligne marquée comme 'Non traité'.")

    # --- Calcul des totaux finaux ---
    final_total_corrected = df['CIR_TOTAL_CORRIGE'].sum()

    # --- Sauvegarde avec nouveau chemin local ---
    output_file = "C:/Users/msamb/Documents/Calcul_Creance_CIR_Corrige_2023.csv"
    print(f"\nSauvegarde du fichier corrigé sous: {output_file}")
    try:
        df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False, decimal='.')
        print(f"Fichier sauvegardé avec succès.")
    except Exception as e: 
        print(f"ERREUR lors de la sauvegarde: {str(e)}")

    # --- Résumé Final ---
    print("\n=====================================================")
    print("                RÉSUMÉ FINAL")
    print("=====================================================")
    print(f"Montant total CIR déclaré (Original):       {original_declared:,.2f} €")
    print(f"Montant total CIR calculé (Original):       {original_calculated:,.2f} €")
    print(f"Montant total CIR après corrections:        {final_total_corrected:,.2f} €")
    initial_ecart = original_calculated - original_declared
    final_ecart = final_total_corrected - original_declared
    improvement = abs(initial_ecart) - abs(final_ecart)
    print(f"\nÉcart initial (calculé - déclaré):        {initial_ecart:,.2f} €")
    print(f"Écart final (corrigé - déclaré):          {final_ecart:,.2f} €")
    print(f"Réduction de l'écart absolu:              {improvement:,.2f} €")
    if abs(original_declared) > 0.01:
        print(f"  Écart relatif initial:                  {initial_ecart / original_declared * 100:.2f}%")
        print(f"  Écart relatif final:                    {final_ecart / original_declared * 100:.2f}%")
        if abs(initial_ecart) > 0.01: print(f"  Réduction relative de l'écart:          {improvement / abs(initial_ecart) * 100:.2f}%")
    else: print("\nCalculs relatifs non pertinents (CIR déclaré total proche de zéro).")

    print("\n--- Détails par Type de Traitement Appliqué ---")
    if 'TRAITEMENT_APPLIQUE' in df.columns:
        treatment_counts = df['TRAITEMENT_APPLIQUE'].value_counts().sort_index()
        print("\nNombre d'entreprises par type de traitement final:")
        for treatment, count in treatment_counts.items(): print(f"  - {treatment:<40}: {count:10,}")
        print("\nDétails financiers par type de traitement final:")
        treatment_order = treatment_counts.index.tolist()
        for treatment in treatment_order:
            subset = df[df['TRAITEMENT_APPLIQUE'] == treatment]
            if len(subset) == 0: continue
            declared_subset = subset['CIR_TOTAL_DECLARE'].sum()
            corrected_subset = subset['CIR_TOTAL_CORRIGE'].sum()
            ecart_final_subset = corrected_subset - declared_subset
            print(f"\nTraitement: {treatment}")
            print(f"  Nombre d'entreprises:                 {len(subset):<10,}")
            print(f"  CIR déclaré (groupe):                 {declared_subset:>18,.2f} €")
            print(f"  CIR corrigé final (groupe):           {corrected_subset:>18,.2f} €")
            print(f"  Écart final (corrigé-déclaré groupe): {ecart_final_subset:>18,.2f} €")
            if abs(declared_subset) > 0.01:
                impact_relatif = ecart_final_subset / declared_subset * 100
                print(f"  Impact relatif final (groupe):        {impact_relatif:>17.2f}%")
    else: print("\nColonne 'TRAITEMENT_APPLIQUE' non trouvée.")

    print("\n=====================================================")
    print("Traitement terminé.")
    print("=====================================================")
    return df

# ===================================================================
# EXÉCUTION
# ===================================================================
if __name__ == "__main__":
    # Chemin adapté
    input_filename = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv"
    print(f"*** DÉBUT DU SCRIPT DE CORRECTION CIR ***")
    # Affichage de la date et heure actuelles
    current_time_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"Date et heure: {current_time_str}")
    print(f"Fichier d'entrée: {input_filename}")
    df_corrige = corriger_cir(input_filename)
    if df_corrige is not None: print("\nScript terminé avec succès.")
    else: print("\nLe script n'a pas pu se terminer correctement.")

*** DÉBUT DU SCRIPT DE CORRECTION CIR ***
Date et heure: 2025-06-04 14:23:53
Fichier d'entrée: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv
Chargement et préparation des données depuis: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2023.csv...
Fichier principal chargé: 27,923 lignes
Formatage des SIRENs...
Conversion des colonnes numériques...
Initialisation de la colonne 'CIR_TOTAL_CORRIGE'...
Préparation des données terminée.

Sommes avant corrections spécifiques:
  - CIR Déclaré Total: 7,986,816,564.00 €
  - CIR Calculé Total: 7,868,711,717.18 €
  - CIR Corrigé Initial Total: 7,868,711,717.18 €

--- Début de l'application des corrections spécifiques ---

1. CORRECTION ERREURS DÉPENSES PERSONNEL (JEUNES DOCTEURS)
-> 8 lignes lues, 8 corrigées. Impact local: 201,129.11 €

2. CORRECTION ERREUR L7 = L8 (FRAIS BREVETS)

3. CORRECTION CIR RECHERCHE MANQUANT/INCOHÉRENT
-> 18 lignes lues, 18 corrigées. Impact local: 1,023,479.10 

In [35]:
import pandas as pd
df_init = pd.read_excel("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//new_millesime_CIR_2023_corrected_sans_doublon.xlsx") 
df_corrige = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_2023.csv", 
                         sep=';', encoding='utf-8-sig', decimal='.', low_memory=False)


# Fusion des 2 fichiers pour avoir toute les variables dont nous avons besoin pour l'etude 

In [36]:
import pandas as pd

# 1. Charger les fichiers originaux
df_init = pd.read_excel("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//new_millesime_CIR_2023_corrected_sans_doublon.xlsx")
df_corrige = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_2023.csv", 
                         sep=';', encoding='utf-8-sig', decimal='.', low_memory=False)

# 2. Standardiser les SIREN déposants puisque c'est la meilleure correspondance
def standardize_siren(siren):
    """Standardise le SIREN en supprimant tous les caractères non numériques et en complétant avec des zéros."""
    if pd.isna(siren):
        return None
    siren_str = str(siren).replace(' ', '').replace('-', '').replace('.', '')
    # Supprimer tous les caractères non numériques
    siren_str = ''.join(c for c in siren_str if c.isdigit())
    # Garder seulement les 9 derniers chiffres pour les SIREN longs
    if len(siren_str) > 9:
        siren_str = siren_str[-9:]
    # Compléter avec des zéros au début si nécessaire
    return siren_str.zfill(9)

# 3. Appliquer la standardisation
df_init['siren_deposant_std'] = df_init['siren_deposant'].apply(standardize_siren)
df_corrige['SIREN_DEPOSANT_STD'] = df_corrige['SIREN_DEPOSANT'].apply(standardize_siren)

# 4. Vérifier la correspondance
common_sirens = set(df_corrige['SIREN_DEPOSANT_STD']) & set(df_init['siren_deposant_std'])
print(f"Nombre de SIREN communs: {len(common_sirens)}")
print(f"Nombre de SIREN uniques dans df_corrige: {df_corrige['SIREN_DEPOSANT_STD'].nunique()}")
print(f"Nombre de SIREN uniques dans df_init: {df_init['siren_deposant_std'].nunique()}")

# 5. Créer les dictionnaires de correspondance pour les deux colonnes
siren_to_type = dict(zip(df_init['siren_deposant_std'], df_init['type_final']))
siren_to_mere = dict(zip(df_init['siren_deposant_std'], df_init['mere_final']))

# 6. Appliquer les deux correspondances en même temps
df_corrige['type_final'] = df_corrige['SIREN_DEPOSANT_STD'].map(siren_to_type)
df_corrige['mere_final'] = df_corrige['SIREN_DEPOSANT_STD'].map(siren_to_mere)

# 7. Vérifier les résultats
type_non_null = df_corrige['type_final'].notna().sum()
mere_non_null = df_corrige['mere_final'].notna().sum()
print(f"Nombre de lignes avec type_final non-null: {type_non_null} sur {len(df_corrige)}")
print(f"Nombre de lignes avec mere_final non-null: {mere_non_null} sur {len(df_corrige)}")

# 8. Supprimer la colonne temporaire
df_corrige = df_corrige.drop(columns=['SIREN_DEPOSANT_STD'])

# 9. Sauvegarder le résultat
df_corrige.to_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_Final_2023.csv", 
                  sep=';', encoding='utf-8-sig', index=False)

print("Fichier sauvegardé avec succès avec les deux colonnes ajoutées.")

# 10. Afficher des exemples des résultats
print("\nExemples de résultats avec les deux colonnes ajoutées:")
sample = df_corrige[['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'TRAITEMENT_APPLIQUE', 'type_final', 'mere_final']].sample(5)
print(sample)

# 11. Afficher les distributions des valeurs
print("\nDistribution des valeurs dans type_final:")
print(df_corrige['type_final'].value_counts().head(10))

print("\nDistribution des valeurs dans mere_final:")
print(df_corrige['mere_final'].value_counts().head(10))

Nombre de SIREN communs: 26734
Nombre de SIREN uniques dans df_corrige: 26734
Nombre de SIREN uniques dans df_init: 26734
Nombre de lignes avec type_final non-null: 27923 sur 27923
Nombre de lignes avec mere_final non-null: 7240 sur 27923
Fichier sauvegardé avec succès avec les deux colonnes ajoutées.

Exemples de résultats avec les deux colonnes ajoutées:
       SIREN_DECLARANT  SIREN_DEPOSANT                 TRAITEMENT_APPLIQUE  \
2203         343564639       343564639                          Non traité   
13607        552049447       552049447  Valeur calculée privilégiée(Reste)   
17844        813140357       813140357                          Non traité   
16307        800683468       800683468                          Non traité   
8137         451518021       451518021                          Non traité   

      type_final   mere_final  
2203         IND          NaN  
13607      FILLE  552049447.0  
17844        IND          NaN  
16307       MERE  800683468.0  
8137        

In [38]:
# 1. Charger le fichier corrigé existant
df_corrige = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_Final_2023.csv", 
                         sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser et nettoyer les SIREN
df_corrige['SIREN_DECLARANT_STD'] = df_corrige['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)
df_corrige['SIREN_DEPOSANT_STD'] = df_corrige['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Créer les ensembles de SIREN déclarants et déposants
set_declarants = set(df_corrige['SIREN_DECLARANT_STD'])
set_deposants = set(df_corrige['SIREN_DEPOSANT_STD'])

# 4. Identifier les SIREN qui sont déposants mais pas déclarants (en filtrant les SIREN non valides)
#if siren and not (siren.strip('0') == '')
deposants_non_declarants = {siren for siren in (set_deposants - set_declarants) }
print(f"Nombre de SIREN qui sont déposants mais pas déclarants: {len(deposants_non_declarants)}")

# 5. Créer les nouvelles lignes pour les déposants manquants
new_rows = []

for siren in deposants_non_declarants:
    # Créer un dictionnaire pour la nouvelle ligne
    new_row = {}
    
    # Copier la structure d'une ligne existante pour obtenir toutes les colonnes
    for col in df_corrige.columns:
        if col in ['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD']:
            continue  # On n'inclut pas ces colonnes temporaires
        
        # Déterminer le type et la valeur appropriée
        if col == 'SIREN_DECLARANT' or col == 'SIREN_DEPOSANT':
            # Utiliser toujours le format chaîne pour éviter les problèmes
            new_row[col] = siren
        elif col == 'TRAITEMENT_APPLIQUE':
            new_row[col] = 'Déposant ajouté'
        elif col == 'type_final':  # Ajouter "MERE" dans la colonne type_retenu
            new_row[col] = 'MERE'
        elif col == 'DESIGNATION' or col == 'COMPLEMENT_DESIGNATION':
            new_row[col] = '' # Champs texte vide
        elif pd.api.types.is_numeric_dtype(df_corrige[col]):
            # Pour les colonnes numériques, mettre 0
            new_row[col] = 0
        else:
            # Pour les autres colonnes non numériques, mettre une valeur vide
            new_row[col] = ''
    
    # Ajouter la ligne au tableau de nouvelles lignes
    new_rows.append(new_row)

# 6. Créer un DataFrame avec les nouvelles lignes
if new_rows:
    df_new_rows = pd.DataFrame(new_rows)
    
    # 7. Concaténer avec le DataFrame original
    df_corrige_complet = pd.concat([df_corrige.drop(['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD'], axis=1),
                                df_new_rows],
                               ignore_index=True)
    
    # 8. Afficher des informations sur le résultat
    print(f"DataFrame original: {len(df_corrige)} lignes")
    print(f"Nouvelles lignes ajoutées: {len(new_rows)} lignes")
    print(f"DataFrame final: {len(df_corrige_complet)} lignes")
    
    # 9. Sauvegarder le résultat
    output_file = "C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv"
    df_corrige_complet.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
    print(f"\nFichier sauvegardé: {output_file}")
    
    # 10. Afficher un exemple des nouvelles lignes
    print("\nExemple d'une nouvelle ligne ajoutée:")
    if not df_new_rows.empty:
        cols_to_show = ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'TRAITEMENT_APPLIQUE', 'type_final', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
        print(df_new_rows[cols_to_show].iloc[0])
else:
    print("Aucun déposant non déclarant trouvé, aucune ligne à ajouter.")

Nombre de SIREN qui sont déposants mais pas déclarants: 386
DataFrame original: 27923 lignes
Nouvelles lignes ajoutées: 386 lignes
DataFrame final: 28309 lignes

Fichier sauvegardé: C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv

Exemple d'une nouvelle ligne ajoutée:
SIREN_DECLARANT              348939513
SIREN_DEPOSANT               348939513
TRAITEMENT_APPLIQUE    Déposant ajouté
type_final                        MERE
CIR_TOTAL_DECLARE                    0
CIR_TOTAL_CALCULE                    0
CIR_TOTAL_CORRIGE                    0
Name: 0, dtype: object


In [ ]:


# 1. Charger le fichier corrigé existant
df_corrige = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output-specifique(2023)//Calcul_Creance_CIR_2023-format_complet.csv", 
                         sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser et nettoyer les SIREN
df_corrige['SIREN_DECLARANT_STD'] = df_corrige['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)
df_corrige['SIREN_DEPOSANT_STD'] = df_corrige['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Créer les ensembles de SIREN déclarants et déposants
set_declarants = set(df_corrige['SIREN_DECLARANT_STD'])
set_deposants = set(df_corrige['SIREN_DEPOSANT_STD'])

# 4. Identifier les SIREN qui sont déposants mais pas déclarants (en filtrant les SIREN non valides)
#if siren and not (siren.strip('0') == '')
deposants_non_declarants = {siren for siren in (set_deposants - set_declarants) }
print(f"Nombre de SIREN qui sont déposants mais pas déclarants: {len(deposants_non_declarants)}")

# 5. Créer les nouvelles lignes pour les déposants manquants
new_rows = []

for siren in deposants_non_declarants:
    # Créer un dictionnaire pour la nouvelle ligne
    new_row = {}
    
    # Copier la structure d'une ligne existante pour obtenir toutes les colonnes
    for col in df_corrige.columns:
        if col in ['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD']:
            continue  # On n'inclut pas ces colonnes temporaires
        
        # Déterminer le type et la valeur appropriée
        if col == 'SIREN_DECLARANT' or col == 'SIREN_DEPOSANT':
            # Utiliser toujours le format chaîne pour éviter les problèmes
            new_row[col] = siren
        elif col == 'TRAITEMENT_APPLIQUE':
            new_row[col] = 'Déposant ajouté'
        elif col == 'type_final':  # Ajouter "MERE" dans la colonne type_retenu
            new_row[col] = 'MERE'
        elif col == 'DESIGNATION' or col == 'COMPLEMENT_DESIGNATION':
            new_row[col] = '' # Champs texte vide
        elif pd.api.types.is_numeric_dtype(df_corrige[col]):
            # Pour les colonnes numériques, mettre 0
            new_row[col] = 0
        else:
            # Pour les autres colonnes non numériques, mettre une valeur vide
            new_row[col] = ''
    
    # Ajouter la ligne au tableau de nouvelles lignes
    new_rows.append(new_row)

# 6. Créer un DataFrame avec les nouvelles lignes
if new_rows:
    df_new_rows = pd.DataFrame(new_rows)
    
    # 7. Concaténer avec le DataFrame original
    df_corrige_complet = pd.concat([df_corrige.drop(['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD'], axis=1),
                                df_new_rows],
                               ignore_index=True)
    
    # 8. Afficher des informations sur le résultat
    print(f"DataFrame original: {len(df_corrige)} lignes")
    print(f"Nouvelles lignes ajoutées: {len(new_rows)} lignes")
    print(f"DataFrame final: {len(df_corrige_complet)} lignes")
    
    # 9. Sauvegarder le résultat
    output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output-specifique(2023)//Calcul_Creance_CIR_2023-format_completavec_deposants.csv"
    df_corrige_complet.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
    print(f"\nFichier sauvegardé: {output_file}")
    
    # 10. Afficher un exemple des nouvelles lignes
    print("\nExemple d'une nouvelle ligne ajoutée:")
    if not df_new_rows.empty:
        cols_to_show = ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'TRAITEMENT_APPLIQUE', 'type_final', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
        print(df_new_rows[cols_to_show].iloc[0])
else:
    print("Aucun déposant non déclarant trouvé, aucune ligne à ajouter.")

Nombre de SIREN qui sont déposants mais pas déclarants: 0
Aucun déposant non déclarant trouvé, aucune ligne à ajouter.


## Difference pour verifier si tous les deposants sont declarants.

In [39]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv", 
                 sep=';', encoding='utf-8-sig', low_memory=False)

# Standardiser les SIREN (en string, 9 caractères, sans espaces)
set_declarants = set(df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9))
set_deposants = set(df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9))

# Différence symétrique (éléments dans l'un ou l'autre ensemble, mais pas les deux)
difference_symetrique = set_declarants ^ set_deposants

print(f"Nombre de SIREN présents soit comme déclarant, soit comme déposant (mais pas les deux) : {len(difference_symetrique)}")

if len(difference_symetrique) == 0:
    print("La différence symétrique est vide.")
else:
    print("Exemples de SIREN présents dans un seul ensemble :", list(difference_symetrique)[:5])
    
    # Si vous voulez aussi voir spécifiquement les SIREN qui sont uniquement déclarants
    seulement_declarants = set_declarants - set_deposants
    print(f"\nNombre de SIREN qui sont uniquement déclarants : {len(seulement_declarants)}")
    if len(seulement_declarants) > 0:
        print("Exemples:", list(seulement_declarants)[:5])
    
    # Et les SIREN qui sont uniquement déposants
    seulement_deposants = set_deposants - set_declarants
    print(f"Nombre de SIREN qui sont uniquement déposants : {len(seulement_deposants)}")
    if len(seulement_deposants) > 0:
        print("Exemples:", list(seulement_deposants)[:5])

Nombre de SIREN présents soit comme déclarant, soit comme déposant (mais pas les deux) : 1477
Exemples de SIREN présents dans un seul ensemble : ['423538644', '972500508', '448252254', '327250031', '718206964']

Nombre de SIREN qui sont uniquement déclarants : 1477
Exemples: ['945752137', '513947630', '352876197', '822026621', '898192075']
Nombre de SIREN qui sont uniquement déposants : 0


In [40]:
import pandas as pd
import numpy as np

# 1. Charger le fichier
print("Chargement du fichier...")
df = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv", 
                 sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser les SIREN des déposants pour éviter les problèmes de format
print("Standardisation des SIREN...")
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Calculer la somme du CIR par déposant
print("Calcul de la somme du CIR par déposant...")
# Utiliser CIR_TOTAL_CORRIGE qui est le montant final retenu après corrections
cir_par_deposant = df.groupby('SIREN_DEPOSANT_STD')['CIR_TOTAL_CORRIGE'].sum().reset_index()
cir_par_deposant.rename(columns={'CIR_TOTAL_CORRIGE': 'cir_benef_total_temp'}, inplace=True)

# 4. Fusionner cette somme avec le DataFrame original
print("Ajout de la nouvelle colonne au DataFrame...")
df = df.merge(cir_par_deposant, on='SIREN_DEPOSANT_STD', how='left')

# 5. Mettre à zéro la colonne cir_benef_total pour toutes les lignes qui ne sont pas de type MERE
print("Attribution des valeurs selon le type...")
# Créer la colonne finale 'cir_benef_total'
df['cir_benef_total'] = 0.0  # Initialiser à zéro pour toutes les lignes

# Mettre la somme calculée uniquement pour les lignes de type 'MERE' et 'IND'
df.loc[df['type_final'] == 'MERE', 'cir_benef_total'] = df.loc[df['type_final'] == 'MERE', 'cir_benef_total_temp']
df.loc[df['type_final'] == 'IND', 'cir_benef_total'] = df.loc[df['type_final'] == 'IND', 'cir_benef_total_temp']

# Supprimer la colonne temporaire
df.drop(['SIREN_DEPOSANT_STD', 'cir_benef_total_temp'], axis=1, inplace=True)

# 6. Afficher quelques statistiques
print(f"\nNombre total d'entreprises: {len(df)}")
print(f"Nombre de déposants uniques: {df['SIREN_DEPOSANT'].nunique()}")

# Vérifier la distribution des valeurs cir_benef_total selon le type
print("\nDistribution des valeurs cir_benef_total par type:")
print(df.groupby('type_final')['cir_benef_total'].describe())

# Trier par montant de cir_benef_total pour voir les déposants avec le plus grand total
top_deposants = df[df['cir_benef_total'] > 0][['SIREN_DEPOSANT', 'cir_benef_total']].drop_duplicates().sort_values('cir_benef_total', ascending=False)
print("\nTop 5 des déposants (MERE) avec le plus grand total de CIR:")
print(top_deposants.head(5))

# 7. Sauvegarder le résultat
output_file = "C://Users//msamb//Documents//Calcul_Creance_CIR_avec_benef_total.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier sauvegardé: {output_file}")

# 8. Vérification - calcul de la somme totale de CIR
total_cir_corrige = df['CIR_TOTAL_CORRIGE'].sum()
total_cir_benef = df['cir_benef_total'].sum()
print(f"\nSomme totale de CIR_TOTAL_CORRIGE: {total_cir_corrige:,.2f} €")
print(f"Somme totale de cir_benef_total (pour les MERE et IND): {total_cir_benef:,.2f} €")

Chargement du fichier...
Standardisation des SIREN...
Calcul de la somme du CIR par déposant...
Ajout de la nouvelle colonne au DataFrame...
Attribution des valeurs selon le type...

Nombre total d'entreprises: 28309
Nombre de déposants uniques: 26734

Distribution des valeurs cir_benef_total par type:
              count           mean           std  min         25%       50%  \
type_final                                                                    
FILLE        6044.0       0.000000  0.000000e+00  0.0      0.0000      0.00   
IND         20683.0  173715.311999  8.006087e+05  0.0  22250.5700  54363.89   
MERE         1582.0  732067.927342  3.900243e+06  0.0  19464.1275  84860.08   

                    75%          max  
type_final                            
FILLE            0.0000         0.00  
IND         121256.6050  31196803.29  
MERE        339894.9925  90632009.12  

Top 5 des déposants (MERE) avec le plus grand total de CIR:
       SIREN_DEPOSANT  cir_benef_total
28212

In [41]:
import pandas as pd
import numpy as np

# Charger le fichier
df = pd.read_csv(output_file, sep=';', encoding='utf-8-sig', low_memory=False)

# Standardiser les SIREN des déposants et des déclarants
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)
df['SIREN_DECLARANT_STD'] = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# Créer une colonne pour la correspondance
df['IS_MATCH'] = df['SIREN_DEPOSANT_STD'] == df['SIREN_DECLARANT_STD']

# Calculer la somme du CIR par déposant
cir_total_par_deposant = df.groupby('SIREN_DEPOSANT_STD')['CIR_TOTAL_CORRIGE'].sum().reset_index()
cir_total_par_deposant.rename(columns={'CIR_TOTAL_CORRIGE': 'TOTAL_CIR'}, inplace=True)

# Initialiser la nouvelle colonne
df['cir_benef_total'] = 0.0

# Méthode simplifiée: traiter chaque SIREN déposant séparément
for siren in cir_total_par_deposant['SIREN_DEPOSANT_STD'].unique():
    montant_total = cir_total_par_deposant.loc[cir_total_par_deposant['SIREN_DEPOSANT_STD'] == siren, 'TOTAL_CIR'].values[0]
    
    # Sélectionner toutes les lignes avec ce SIREN déposant
    lignes_siren = df[df['SIREN_DEPOSANT_STD'] == siren]
    
    # Vérifier s'il y a des lignes correspondantes (SIREN_DEPOSANT = SIREN_DECLARANT)
    lignes_match = lignes_siren[lignes_siren['IS_MATCH']]
    
    if len(lignes_match) > 0:
        # S'il y a des correspondances, distribuer le montant également entre ces lignes
        montant_par_ligne = montant_total / len(lignes_match)
        df.loc[lignes_match.index, 'cir_benef_total'] = montant_par_ligne
    else:
        # S'il n'y a pas de correspondance, mettre le montant total sur la première ligne
        df.loc[lignes_siren.index[0], 'cir_benef_total'] = montant_total

# Vérification
total_cir_corrige = df['CIR_TOTAL_CORRIGE'].sum()
total_cir_benef = df['cir_benef_total'].sum()

# Créer un fichier de vérification avec les totaux par SIREN déposant
verification = df.groupby('SIREN_DEPOSANT_STD')[['CIR_TOTAL_CORRIGE', 'cir_benef_total']].sum()
verification['difference'] = verification['CIR_TOTAL_CORRIGE'] - verification['cir_benef_total']
verification.to_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//verification_totaux_par_siren.csv", sep=';')

# Comptage des SIREN sans correspondance
siren_avec_match = df[df['IS_MATCH']]['SIREN_DEPOSANT_STD'].unique()
siren_tous = df['SIREN_DEPOSANT_STD'].unique()
siren_sans_match = set(siren_tous) - set(siren_avec_match)

# Supprimer les colonnes temporaires avant de sauvegarder
df = df.drop(['SIREN_DEPOSANT_STD', 'SIREN_DECLARANT_STD', 'IS_MATCH'], axis=1)

# Sauvegarder le résultat
output_file_final = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df.to_csv(output_file_final, sep=';', encoding='utf-8-sig', index=False)

print(f"Fichier sauvegardé: {output_file_final}")
print(f"\nSomme totale de CIR_TOTAL_CORRIGE: {total_cir_corrige:,.2f} €")
print(f"Somme totale de cir_benef_total: {total_cir_benef:,.2f} €")
print(f"Différence entre les deux sommes: {total_cir_corrige - total_cir_benef:,.2f} €")
print(f"\nNombre de SIREN déposants sans correspondance: {len(siren_sans_match)}")

# Montant CIR pour les SIREN sans correspondance
cir_sans_match = sum(verification.loc[[s for s in siren_sans_match if s in verification.index], 'CIR_TOTAL_CORRIGE'])
print(f"Montant CIR pour ces SIREN: {cir_sans_match:,.2f} € ({cir_sans_match/total_cir_corrige*100:.2f}% du total)")

Fichier sauvegardé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv

Somme totale de CIR_TOTAL_CORRIGE: 7,869,827,573.24 €
Somme totale de cir_benef_total: 7,869,827,573.24 €
Différence entre les deux sommes: 0.00 €

Nombre de SIREN déposants sans correspondance: 6
Montant CIR pour ces SIREN: 6,266,675.16 € (0.08% du total)


In [42]:
import pandas as pd

# Charger le fichier CSV
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')

# Dictionnaire de renommage des colonnes
renommage = {
    # Renommage des colonnes CIR recherche
    'CIR_RECHERCHE_DECLARE': 'crerd_gen_declare',
    'CIR_RECHERCHE_CALCULE': 'crerd_gen_calcule',
    'CIR_RECHERCHE_ECART': 'crerd_gen_ecart',
    
    # Renommage des colonnes CIR collection
    'CIR_COLLECTION_DECLARE': 'crecoll_gen_declare',
    'CIR_COLLECTION_CALCULE': 'crecoll_gen_calcule',
    'CIR_COLLECTION_ECART': 'crecoll_gen_ecart',
    
    # Renommage des colonnes CIR innovation
    'CIR_INNOVATION_DECLARE': 'creinno_gen_declare',
    'CIR_INNOVATION_CALCULE': 'creinno_gen_calcule',
    'CIR_INNOVATION_ECART': 'creinno_gen_ecart',
    
    # Renommage des colonnes CIR collaboratif
    'CIR_COLLABORATIF_DECLARE': 'crecrc_declare',
    'CIR_COLLABORATIF_CALCULE': 'crecrc_calcule',
    'CIR_COLLABORATIF_ECART': 'crecrc_ecart',
    
    # Renommage des colonnes CIR total
    'CIR_TOTAL_DECLARE': 'cretot_gen_declare',
    'CIR_TOTAL_CALCULE': 'cretot_gen_calcule',
    'CIR_TOTAL_ECART': 'cretot_gen_ecart',
    'CIR_TOTAL_CORRIGE': 'cretot_gen_corrige'
}

# Appliquer le renommage (uniquement pour les colonnes qui existent)
colonnes_existantes = {col: nouveau_nom for col, nouveau_nom in renommage.items() if col in df.columns}
df = df.rename(columns=colonnes_existantes)

# Sauvegarder le fichier avec les nouvelles colonnes
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final_renamed.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)

print(f"Fichier sauvegardé avec succès: {output_file}")
print(f"Colonnes renommées: {list(colonnes_existantes.keys())}")

Fichier sauvegardé avec succès: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final_renamed.csv
Colonnes renommées: ['CIR_RECHERCHE_DECLARE', 'CIR_RECHERCHE_CALCULE', 'CIR_RECHERCHE_ECART', 'CIR_COLLECTION_DECLARE', 'CIR_COLLECTION_CALCULE', 'CIR_COLLECTION_ECART', 'CIR_INNOVATION_DECLARE', 'CIR_INNOVATION_CALCULE', 'CIR_INNOVATION_ECART', 'CIR_COLLABORATIF_DECLARE', 'CIR_COLLABORATIF_CALCULE', 'CIR_COLLABORATIF_ECART', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_ECART', 'CIR_TOTAL_CORRIGE']


In [43]:
import pandas as pd
import numpy as np

# Charger le fichier avec les types de données spécifiés pour améliorer les performances
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')

print(f"Nombre total de lignes: {len(df)}")

# 1. RENOMMER TOUTES LES COLONNES (opération rapide)
renommage_complet = {
    'CIR_RECHERCHE_DECLARE': 'crerd_gen_declare',
    'CIR_RECHERCHE_CALCULE': 'crerd_gen',
    'CIR_RECHERCHE_ECART': 'crerd_gen_ecart',
    'CIR_COLLECTION_DECLARE': 'crecoll_gen_declare',
    'CIR_COLLECTION_CALCULE': 'crecoll_gen',
    'CIR_COLLECTION_ECART': 'crecoll_gen_ecart',
    'CIR_INNOVATION_DECLARE': 'creinno_gen_declare',
    'CIR_INNOVATION_CALCULE': 'creinno_gen',
    'CIR_INNOVATION_ECART': 'creinno_gen_ecart',
    'CIR_COLLABORATIF_DECLARE': 'crecrc_declare',
    'CIR_COLLABORATIF_CALCULE': 'crecrc',
    'CIR_COLLABORATIF_ECART': 'crecrc_ecart',
    'CIR_TOTAL_DECLARE': 'cretot_gen_declare',
    'CIR_TOTAL_CALCULE': 'cretot_gen_calcule',
    'CIR_TOTAL_ECART': 'cretot_gen_ecart',
    'CIR_TOTAL_CORRIGE': 'cretot_gen'
}

# Renommer uniquement les colonnes qui existent
colonnes_existantes = {col: nouveau_nom for col, nouveau_nom in renommage_complet.items() if col in df.columns}
df = df.rename(columns=colonnes_existantes)

print(f"Colonnes renommées: {len(colonnes_existantes)}")

# 2. CALCULER LES CRÉDITS BÉNÉFICIAIRES DE MANIÈRE OPTIMISÉE
# Standardiser les SIREN une seule fois
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['SIREN_DECLARANT_STD'] = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['IS_MATCH'] = df['SIREN_DEPOSANT_STD'] == df['SIREN_DECLARANT_STD']

# Version vectorisée et optimisée pour calculer les valeurs bénéficiaires
def calculer_benef_optimise(df, colonne_source):
    # Créer un DataFrame avec les informations essentielles
    df_temp = df[['SIREN_DEPOSANT_STD', 'IS_MATCH', colonne_source]].copy()
    
    # Calculer la somme totale par déposant
    sommes = df_temp.groupby('SIREN_DEPOSANT_STD')[colonne_source].sum().to_dict()
    
    # Créer une série pour stocker les résultats
    resultats = pd.Series(index=df.index, dtype='float64')
    
    # Pour chaque SIREN déposant unique
    for siren, montant in sommes.items():
        # Lignes correspondant à ce SIREN
        mask_siren = df['SIREN_DEPOSANT_STD'] == siren
        
        # Lignes où il y a correspondance
        mask_match = mask_siren & df['IS_MATCH']
        count_match = mask_match.sum()
        
        if count_match > 0:
            # Distribuer le montant également entre les lignes correspondantes
            resultats[mask_match] = montant / count_match
        else:
            # Première ligne du SIREN
            if mask_siren.any():
                idx = df[mask_siren].index[0]
                resultats[idx] = montant
    
    return resultats

# Calculer tous les bénéficiaires d'un coup pour gagner du temps
print("Calcul des montants bénéficiaires...")
colonnes_source = ['crerd_gen', 'crecoll_gen', 'creinno_gen', 'crecrc', 'cretot_gen']
colonnes_benef = ['cirrech_benef', 'cic_benef', 'cii_benef', 'crecrc_benef', 'cirtot_benef']

for col_source, col_benef in zip(colonnes_source, colonnes_benef):
    if col_source in df.columns:
        print(f"  Calcul de {col_benef}...")
        df[col_benef] = calculer_benef_optimise(df, col_source)
    else:
        print(f"  Colonne {col_source} non trouvée, {col_benef} initialisé à 0")
        df[col_benef] = 0

# 3. VÉRIFIER LA COHÉRENCE DES MONTANTS
print("Vérification de la cohérence des montants...")

# Vérifier que cir_benef_total correspond à cirtot_benef (si présent)
if 'cir_benef_total' in df.columns:
    ecart_total = (df['cirtot_benef'] - df['cir_benef_total']).abs().sum()
    print(f"Écart entre cirtot_benef et cir_benef_total: {ecart_total:.2f}")

# Vérifier que la somme des composantes bénéficiaires égale le total bénéficiaire (vectorisé)
df['somme_composantes_benef'] = df[['cirrech_benef', 'cic_benef', 'cii_benef', 'crecrc_benef']].fillna(0).sum(axis=1)
ecart_composantes = (df['somme_composantes_benef'] - df['cirtot_benef']).abs().sum()
print(f"Écart entre somme des composantes et total bénéficiaire: {ecart_composantes:.2f}")

# Comparer les totaux (vectorisé)
print("\nVérification des totaux générés vs bénéficiaires:")
print(f"{'Type de crédit':<15}{'Généré':<15}{'Bénéficiaire':<15}{'Écart':<15}{'Écart %':<10}")
print("-" * 65)

for col_gen, col_benef in zip(colonnes_source, colonnes_benef):
    if col_gen in df.columns and col_benef in df.columns:
        total_gen = df[col_gen].sum()
        total_benef = df[col_benef].sum()
        ecart = total_gen - total_benef
        ecart_pct = 0 if total_gen == 0 else ecart / total_gen * 100
        print(f"{col_gen:<15}{total_gen:<15.2f}{total_benef:<15.2f}{ecart:<15.2f}{ecart_pct:<10.2f}%")

# Ajouter les variables pour les DOM-TOM en une seule opération vectorisée
for col in ['crerd_genom', 'crecoll_genom', 'creinno_genom', 'cretot_genom']:
    df[col] = 0

# Supprimer les colonnes temporaires en une seule opération
df = df.drop(['SIREN_DEPOSANT_STD', 'SIREN_DECLARANT_STD', 'IS_MATCH', 'somme_composantes_benef'], axis=1)

# Sauvegarder le fichier
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021.csv"
print(f"Sauvegarde du fichier: {output_file}")
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)

print("Traitement terminé avec succès!")

Nombre total de lignes: 28309
Colonnes renommées: 16
Calcul des montants bénéficiaires...
  Calcul de cirrech_benef...
  Calcul de cic_benef...
  Calcul de cii_benef...
  Calcul de crecrc_benef...
  Calcul de cirtot_benef...
Vérification de la cohérence des montants...
Écart entre cirtot_benef et cir_benef_total: 0.00
Écart entre somme des composantes et total bénéficiaire: 853099.92

Vérification des totaux générés vs bénéficiaires:
Type de crédit Généré         Bénéficiaire   Écart          Écart %   
-----------------------------------------------------------------
crerd_gen      7324271677.45  7324271677.45  0.00           0.00      %
crecoll_gen    27543075.80    27543075.80    0.00           0.00      %
creinno_gen    500767614.75   500767614.75   0.00           0.00      %
crecrc         16402985.70    16402985.70    0.00           0.00      %
cretot_gen     7869827573.24  7869827573.24  0.00           0.00      %
Sauvegarde du fichier: M://str-dgri-gecir-donnees-fiscales//x-pou

In [44]:
import pandas as pd
import numpy as np

# Charger le fichier obtenu précédemment
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')

print(f"Nombre total de lignes: {len(df)}")
print(f"Colonnes disponibles: {len(df.columns)}")



print("\n=== CALCUL DES CRÉDITS DOM-TOM ===")

# Rechercher les colonnes DOM spécifiques
colonnes_dom_rd = [col for col in df.columns if 'DOM' in col and ('RECHERCHE' in col or 'DEPENSES' in col)]
colonnes_dom_coll = [col for col in df.columns if 'DOM' in col and 'COLLECTION' in col]
colonnes_dom_inno = [col for col in df.columns if 'DOM' in col and 'INNOVATION' in col]

print(f"Colonnes DOM R&D trouvées: {colonnes_dom_rd}")
print(f"Colonnes DOM Collection trouvées: {colonnes_dom_coll}")
print(f"Colonnes DOM Innovation trouvées: {colonnes_dom_inno}")

# Calculer les crédits DOM-TOM avec les taux spécifiques
# CIR Recherche DOM : 50%
if 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns:
    df['crerd_genom'] = df['L26B_MONTANT_NET_DEPENSES_DOM_CALCULE'].fillna(0) * 0.50
    print(f"CIR Recherche DOM calculé: {df['crerd_genom'].sum():,.2f} €")
else:
    df['crerd_genom'] = 0
    print("Pas de données DOM pour la recherche trouvées")

# CIR Collection DOM : 50%
col_coll_dom = next((col for col in df.columns if 'L33B' in col and 'DOM' in col), None)
if col_coll_dom:
    df['crecoll_genom'] = df[col_coll_dom].fillna(0) * 0.50
    print(f"CIR Collection DOM calculé: {df['crecoll_genom'].sum():,.2f} €")
else:
    df['crecoll_genom'] = 0
    print("Pas de données DOM pour la collection trouvées")

# CIR Innovation DOM : 60% + Corse
col_inno_dom = next((col for col in df.columns if 'L76B' in col and 'DOM' in col), None)
col_inno_corse_mpe = next((col for col in df.columns if 'L76C' in col and 'CORSE' in col), None)
col_inno_corse_me = next((col for col in df.columns if 'L76D' in col and 'CORSE' in col), None)

if col_inno_dom:
    cii_dom = df[col_inno_dom].fillna(0) * 0.60
else:
    cii_dom = 0

if col_inno_corse_mpe:
    cii_corse_mpe = df[col_inno_corse_mpe].fillna(0) * 0.40
else:
    cii_corse_mpe = 0

if col_inno_corse_me:
    cii_corse_me = df[col_inno_corse_me].fillna(0) * 0.35
else:
    cii_corse_me = 0

df['creinno_genom'] = cii_dom + cii_corse_mpe + cii_corse_me
print(f"CIR Innovation DOM+Corse calculé: {df['creinno_genom'].sum():,.2f} €")

# CIR Total DOM
df['cretot_genom'] = df['crerd_genom'] + df['crecoll_genom'] + df['creinno_genom']
print(f"CIR Total DOM calculé: {df['cretot_genom'].sum():,.2f} €")



print("\n=== CALCUL DES MONTANTS NETS BÉNÉFICIAIRES ===")

# Standardiser les SIREN pour assurer la cohérence
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['SIREN_DECLARANT_STD'] = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['IS_MATCH'] = df['SIREN_DEPOSANT_STD'] == df['SIREN_DECLARANT_STD']

# Recherche automatique des colonnes de dépenses
def trouver_colonne_depenses(patterns, description):
    """Trouve la meilleure colonne correspondant aux patterns donnés"""
    for pattern in patterns:
        colonnes_trouvees = [col for col in df.columns if pattern in col]
        if colonnes_trouvees:
            print(f"Colonne {description} trouvée: {colonnes_trouvees[0]}")
            return colonnes_trouvees[0]
    print(f"Aucune colonne {description} trouvée")
    return None

# Rechercher les colonnes de dépenses
colonne_depenses_rd = trouver_colonne_depenses([
    'L26A_MONTANT_NET_DEPENSES_CALCULE',
    'L26A_MONTANT_NET_DEPENSES_DECLARE',
    'MONTANT_NET_DEPENSES'
], "dépenses R&D")

col_dep_coll = trouver_colonne_depenses([
    'L33A_MONTANT_NET_COLLECTION_CALCULE',
    'L33A_MONTANT_NET_COLLECTION_DECLARE',
    'MONTANT_NET_COLLECTION'
], "dépenses Collection")

col_dep_inno = trouver_colonne_depenses([
    'L76A_MONTANT_NET_INNOVATION_CALCULE',
    'L76A_MONTANT_NET_INNOVATION_DECLARE',
    'MONTANT_NET_INNOVATION'
], "dépenses Innovation")

# Fonction optimisée pour calculer les montants nets bénéficiaires
def calculer_benef_optimise(df, colonne_source, nom_calcul=""):
    """Calcule les montants bénéficiaires de manière optimisée"""
    if colonne_source is None or colonne_source not in df.columns:
        print(f"  {nom_calcul}: Colonne non trouvée, initialisation à 0")
        return pd.Series(0, index=df.index, dtype='float64')
    
    print(f"  {nom_calcul}: Calcul en cours...")
    
    # Créer un DataFrame temporaire avec les données essentielles
    df_temp = df[['SIREN_DEPOSANT_STD', 'IS_MATCH', colonne_source]].copy()
    df_temp[colonne_source] = df_temp[colonne_source].fillna(0)
    
    # Calculer la somme totale par déposant
    sommes = df_temp.groupby('SIREN_DEPOSANT_STD')[colonne_source].sum().to_dict()
    
    # Créer une série pour stocker les résultats
    resultats = pd.Series(0, index=df.index, dtype='float64')
    
    # Compteurs pour le rapport
    count_match = 0
    count_no_match = 0
    
    # Pour chaque SIREN déposant unique
    for siren, montant in sommes.items():
        if montant == 0:
            continue
            
        # Identifier les lignes correspondant à ce SIREN
        mask_siren = df['SIREN_DEPOSANT_STD'] == siren
        
        # Identifier les lignes où il y a correspondance (déclarant = déposant)
        mask_match_siren = mask_siren & df['IS_MATCH']
        nb_match = mask_match_siren.sum()
        
        if nb_match > 0:
            # Distribuer le montant également entre les lignes correspondantes
            resultats[mask_match_siren] = montant / nb_match
            count_match += nb_match
        else:
            # Attribuer à la première ligne du SIREN s'il n'y a pas de correspondance
            if mask_siren.any():
                idx = df[mask_siren].index[0]
                resultats[idx] = montant
                count_no_match += 1
    
    total_benef = resultats.sum()
    total_gen = df_temp[colonne_source].sum()
    print(f"    Total généré: {total_gen:,.2f} € | Total bénéficiaire: {total_benef:,.2f} € | Écart: {total_gen-total_benef:,.2f} €")
    print(f"    Répartition: {count_match} avec correspondance, {count_no_match} sans correspondance")
    
    return resultats

# Calculer les montants nets des dépenses bénéficiaires
print("Calcul des montants nets des dépenses bénéficiaires:")
df['deprd_benef'] = calculer_benef_optimise(df, colonne_depenses_rd, "Dépenses R&D")
df['depcoll_benef'] = calculer_benef_optimise(df, col_dep_coll, "Dépenses Collection")
df['depinno_benef'] = calculer_benef_optimise(df, col_dep_inno, "Dépenses Innovation")

# Calculer le montant total des dépenses bénéficiaires
df['deptot_benef'] = df['deprd_benef'] + df['depcoll_benef'] + df['depinno_benef']
print(f"Montant total des dépenses bénéficiaires: {df['deptot_benef'].sum():,.2f} €")


print("\n=== CRÉATION DES INDICATRICES ===")

# i_dep_rd : "Oui" si dépenses R&D > 0
if colonne_depenses_rd and colonne_depenses_rd in df.columns:
    df['i_dep_rd'] = np.where(df[colonne_depenses_rd] > 0, "Oui", "Non")
else:
    df['i_dep_rd'] = "Non"

# i_dep_inno : "Oui" si dépenses inno > 0
if col_dep_inno and col_dep_inno in df.columns:
    df['i_dep_inno'] = np.where(df[col_dep_inno] > 0, "Oui", "Non")
else:
    df['i_dep_inno'] = "Non"

# i_dep_coll : "Oui" si dépenses collection > 0
if col_dep_coll and col_dep_coll in df.columns:
    df['i_dep_coll'] = np.where(df[col_dep_coll] > 0, "Oui", "Non")
else:
    df['i_dep_coll'] = "Non"

# i_bef : "Oui" si bénéficiaire du CIR total
df['i_bef'] = np.where(df['cirtot_benef'] > 0, "Oui", "Non")

# i_bef_rech : "Oui" si bénéficiaire du CIR recherche
df['i_bef_rech'] = np.where(df['cirrech_benef'] > 0, "Oui", "Non")

# i_bef_inno : "Oui" si bénéficiaire du CII
df['i_bef_inno'] = np.where(df['cii_benef'] > 0, "Oui", "Non")

# i_bef_crc : "Oui" si bénéficiaire du CRC
df['i_bef_crc'] = np.where(df['crecrc_benef'] > 0, "Oui", "Non")

# i_bef_coll : "Oui" si bénéficiaire du CI collection
df['i_bef_coll'] = np.where(df['cic_benef'] > 0, "Oui", "Non")

print("Indicatrices créées avec succès")



print("\n" + "="*60)
print("RAPPORT DÉTAILLÉ DE VÉRIFICATION DES DONNÉES")
print("="*60)

# Vérifier les totaux des montants nets de dépenses
print("\n1. MONTANTS NETS DES DÉPENSES BÉNÉFICIAIRES:")
print(f"   • Dépenses R&D bénéficiaires: {df['deprd_benef'].sum():,.2f} €")
print(f"   • Dépenses Collection bénéficiaires: {df['depcoll_benef'].sum():,.2f} €")
print(f"   • Dépenses Innovation bénéficiaires: {df['depinno_benef'].sum():,.2f} €")
print(f"   • TOTAL dépenses bénéficiaires: {df['deptot_benef'].sum():,.2f} €")

# Vérifier les totaux des crédits DOM-TOM
print("\n2. CRÉDITS D'IMPÔT DOM-TOM:")
print(f"   • CIR Recherche DOM: {df['crerd_genom'].sum():,.2f} €")
print(f"   • CIR Collection DOM: {df['crecoll_genom'].sum():,.2f} €")
print(f"   • CIR Innovation DOM: {df['creinno_genom'].sum():,.2f} €")
print(f"   • TOTAL CIR DOM: {df['cretot_genom'].sum():,.2f} €")

# Vérifier les comptes des indicatrices
print("\n3. RÉPARTITION DES INDICATRICES:")
indicatrices = ['i_dep_rd', 'i_dep_inno', 'i_dep_coll', 'i_bef', 'i_bef_rech', 'i_bef_inno', 'i_bef_crc', 'i_bef_coll']

for indicatrice in indicatrices:
    if indicatrice in df.columns:
        count_oui = (df[indicatrice] == "Oui").sum()
        count_non = (df[indicatrice] == "Non").sum()
        pct_oui = count_oui/len(df)*100
        pct_non = count_non/len(df)*100
        print(f"   • {indicatrice:<12}: {count_oui:>6,} Oui ({pct_oui:>5.1f}%) | {count_non:>6,} Non ({pct_non:>5.1f}%)")

# Vérification de cohérence des bénéficiaires
print("\n4. VÉRIFICATIONS DE COHÉRENCE:")

# Cohérence entre i_bef et les autres indicatrices bénéficiaires
nb_benef_total = (df['i_bef'] == "Oui").sum()
nb_benef_rech = (df['i_bef_rech'] == "Oui").sum()
nb_benef_coll = (df['i_bef_coll'] == "Oui").sum()
nb_benef_inno = (df['i_bef_inno'] == "Oui").sum()
nb_benef_crc = (df['i_bef_crc'] == "Oui").sum()

print(f"   • Bénéficiaires totaux: {nb_benef_total:,}")
print(f"   • Détail: Recherche {nb_benef_rech:,} | Collection {nb_benef_coll:,} | Innovation {nb_benef_inno:,} | CRC {nb_benef_crc:,}")

# Vérifier la cohérence entre dépenses et bénéfices
coherence_rd = ((df['i_dep_rd'] == "Oui") & (df['i_bef_rech'] == "Non")).sum()
coherence_coll = ((df['i_dep_coll'] == "Oui") & (df['i_bef_coll'] == "Non")).sum()
coherence_inno = ((df['i_dep_inno'] == "Oui") & (df['i_bef_inno'] == "Non")).sum()

print(f"   • Entreprises avec dépenses R&D mais sans bénéfice recherche: {coherence_rd:,}")
print(f"   • Entreprises avec dépenses Collection mais sans bénéfice collection: {coherence_coll:,}")
print(f"   • Entreprises avec dépenses Innovation mais sans bénéfice innovation: {coherence_inno:,}")

# ---------------------------------------------------
# 5. FINALISATION ET SAUVEGARDE
# ---------------------------------------------------

# Supprimer les colonnes temporaires
colonnes_temp = ['SIREN_DEPOSANT_STD', 'SIREN_DECLARANT_STD', 'IS_MATCH']
for col in colonnes_temp:
    if col in df.columns:
        df = df.drop(col, axis=1)

# Ajouter une colonne de contrôle avec la date de traitement
from datetime import datetime
df['date_traitement'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Sauvegarder le fichier avec les nouvelles colonnes
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021_complet.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)

print(f"\n{'='*60}")
print(f"TRAITEMENT TERMINÉ AVEC SUCCÈS")
print(f"{'='*60}")
print(f"Fichier sauvegardé: {output_file}")
print(f"Nombre total de lignes: {len(df):,}")
print(f"Nombre total de colonnes: {len(df.columns):,}")
print(f"Taille du fichier final: {len(df) * len(df.columns):,} cellules")

# Résumé des nouvelles colonnes créées
nouvelles_colonnes = [
    'crerd_genom', 'crecoll_genom', 'creinno_genom', 'cretot_genom',
    'deprd_benef', 'depcoll_benef', 'depinno_benef', 'deptot_benef',
    'i_dep_rd', 'i_dep_inno', 'i_dep_coll', 'i_bef', 'i_bef_rech', 
    'i_bef_inno', 'i_bef_crc', 'i_bef_coll', 'date_traitement'
]

print(f"\nNOUVELLES COLONNES CRÉÉES ({len(nouvelles_colonnes)}):")
for i, col in enumerate(nouvelles_colonnes, 1):
    print(f"  {i:2d}. {col}")

print(f"\nFichier prêt ")

Nombre total de lignes: 28309
Colonnes disponibles: 185

=== CALCUL DES CRÉDITS DOM-TOM ===
Colonnes DOM R&D trouvées: ['L26B_MONTANT_NET_DEPENSES_DOM_DECLARE', 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE', 'L38B_CREDIT_IMPOT_RECHERCHE_DOM', 'L46B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM', 'L47B_DEPENSES_RECHERCHE_DOM_LIMITE', 'L53B_CREDIT_IMPOT_RECHERCHE_DOM_PLUS_100M', 'L64B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M']
Colonnes DOM Collection trouvées: ['L33B_MONTANT_NET_COLLECTION_DOM_DECLARE', 'L33B_MONTANT_NET_COLLECTION_DOM_CALCULE', 'L46B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM', 'L54B_MONTANT_NET_COLLECTION_DOM_PLUS_100M', 'L64B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M']
Colonnes DOM Innovation trouvées: ['L76B_MONTANT_NET_INNOVATION_DOM', 'L79B_CREDIT_IMPOT_INNOVATION_DOM']
CIR Recherche DOM calculé: 14,015,514.50 €
CIR Collection DOM calculé: 199,838.00 €
CIR Innovation DOM+Corse calculé: 5,224,736.25 €
CIR Total DOM calculé: 19,440,088.75 €

=== CALCUL 

In [ ]:
# -*- coding: utf-8 -*-
"""
Ce script :
1. Charge et prépare un fichier de référence des unités légales (StockUniteLegale).
2. Charge une base de données principale à enrichir ("base_a_sireniser").
3. Joint ces deux bases sur le numéro SIREN.
4. Charge une nomenclature d'activités (Nomenclature Sittar).
5. Joint le résultat précédent avec cette nomenclature sur le code APE.

librairie pandas installée requis: pip install pandas openpyxl
(openpyxl est nécessaire pour lire les fichiers Excel .xlsx)
"""

import pandas as pd # Librairie pour la manipulation de données
import re           # Librairie pour les expressions régulières (nettoyage des noms de colonnes)
# La librairie 'os' n'est plus nécessaire car on ne crée plus de dossier/fichiers de test.

# --- Fonction Auxiliaire pour Nettoyer les Noms de Colonnes ---
def nettoyer_noms_colonnes(df_a_nettoyer):
    """
    Nettoie les noms de colonnes d'un DataFrame :
    1. Convertit en minuscules.
    2. Remplace les espaces et caractères spéciaux par des tirets bas ('_').
    3. Supprime les tirets bas en début ou fin de nom.
    Retourne un nouveau DataFrame avec les noms de colonnes nettoyés.
    """
    df = df_a_nettoyer.copy() # Travailler sur une copie
    nouveaux_noms = {}
    for col in df.columns:
        nouveau_nom = str(col).lower()
        nouveau_nom = re.sub(r'\s+', '_', nouveau_nom)
        nouveau_nom = re.sub(r'[^a-z0-9_]', '', nouveau_nom)
        nouveau_nom = re.sub(r'_+', '_', nouveau_nom)
        nouveau_nom = nouveau_nom.strip('_')
        nouveaux_noms[col] = nouveau_nom
    df.rename(columns=nouveaux_noms, inplace=True)
    return df

# --- 1. Chargement et Préparation des Données 'StockUniteLegale' ---
def charger_et_preparer_stock_ul(chemin_fichier_stock_ul):
    """
    Charge et prépare les données du fichier 'StockUniteLegale'.
    """
    print(f"Chargement et préparation de '{chemin_fichier_stock_ul}'...")
    colonnes_a_garder = [
        'siren', 'nicSiegeUniteLegale', 'denominationUniteLegale',
        'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale',
        'denominationUsuelle3UniteLegale', 'dateCreationUniteLegale',
        'activitePrincipaleUniteLegale', 'categorieEntreprise',
        'trancheEffectifsUniteLegale'
    ]
    try:
        stock_ul_df = pd.read_csv(chemin_fichier_stock_ul, 
                                  usecols=colonnes_a_garder, 
                                  dtype=str, 
                                  encoding='utf-8')
    except FileNotFoundError:
        print(f"ERREUR : Le fichier '{chemin_fichier_stock_ul}' est introuvable.")
        raise
    except ValueError as e:
        print(f"ERREUR : Vérifiez que les colonnes {colonnes_a_garder} existent bien dans '{chemin_fichier_stock_ul}'. Détail : {e}")
        raise

    stock_ul_df['nic'] = stock_ul_df['nicSiegeUniteLegale'].str.zfill(5)
    stock_ul_df['siren'] = stock_ul_df['siren'].str.zfill(9)
    stock_ul_df['siret'] = stock_ul_df['siren'] + stock_ul_df['nic']
    stock_ul_df.rename(columns={'siren': 'SIREN_DECLARANT'}, inplace=True)
    print(f"Colonnes de stock_ul_extract après préparation : {stock_ul_df.columns.tolist()}")
    return stock_ul_df

# --- Fonction pour Charger la 'base_a_sireniser' ---
def charger_base_a_sireniser(chemin_fichier_base, nom_colonne_siren='SIREN_DECLARANT'):
    """
    Charge les données de la 'base_a_sireniser'.
    'nom_colonne_siren' doit être le nom de la colonne SIREN dans ce fichier.
    """
    print(f"Chargement de la base à siréniser depuis '{chemin_fichier_base}'...")
    try:
        base_df = pd.read_csv(chemin_fichier_base,
                      dtype={nom_colonne_siren: str}, 
                      encoding='utf-8',
                      sep=';') # Ajoutez sep=';' ou le séparateur correct
    except FileNotFoundError:
        print(f"ERREUR : Le fichier '{chemin_fichier_base}' est introuvable.")
        raise
    except KeyError:
        print(f"ERREUR : La colonne '{nom_colonne_siren}' (spécifiée pour le SIREN) est introuvable dans '{chemin_fichier_base}'.")
        raise
        
    if nom_colonne_siren in base_df.columns:
        base_df[nom_colonne_siren] = base_df[nom_colonne_siren].str.zfill(9)
    print(f"Colonnes de base_a_sireniser : {base_df.columns.tolist()}")
    return base_df

# --- 3. Chargement et Préparation des Données de la 'Nomenclature Sittar' ---
def charger_et_preparer_nomenclature(chemin_fichier_nomenclature, nom_original_col_ape='code_ape_entreprise'):
    """
    Charge et prépare les données du fichier 'Nomenclature_secteur_sittar.xlsx'.
    'nom_original_col_ape' est le nom original de la colonne du code APE dans le fichier Excel.
    """
    print(f"Chargement et préparation de la nomenclature '{chemin_fichier_nomenclature}'...")
    try:
        nomenclature_df = pd.read_excel(chemin_fichier_nomenclature)
    except FileNotFoundError:
        print(f"ERREUR : Le fichier '{chemin_fichier_nomenclature}' est introuvable.")
        raise

    nomenclature_df = nettoyer_noms_colonnes(nomenclature_df)
    col_ape_nettoyee_candidate = nettoyer_noms_colonnes(pd.DataFrame(columns=[nom_original_col_ape])).columns[0]
    nom_cible_col_ape_pour_jointure = 'activiteprincipaleunitelegale'

    if col_ape_nettoyee_candidate in nomenclature_df.columns:
        nomenclature_df.rename(columns={col_ape_nettoyee_candidate: nom_cible_col_ape_pour_jointure}, inplace=True)
    elif nom_original_col_ape.lower() in nomenclature_df.columns and nom_original_col_ape.lower() != nom_cible_col_ape_pour_jointure :
         nomenclature_df.rename(columns={nom_original_col_ape.lower(): nom_cible_col_ape_pour_jointure}, inplace=True)
    elif nom_cible_col_ape_pour_jointure not in nomenclature_df.columns:
        print(f"Attention : Colonne APE ('{col_ape_nettoyee_candidate}' ou '{nom_original_col_ape.lower()}') non trouvée "
              f"pour renommage en '{nom_cible_col_ape_pour_jointure}'. Colonnes dispo : {nomenclature_df.columns.tolist()}")

    if nom_cible_col_ape_pour_jointure in nomenclature_df.columns:
        cols = [nom_cible_col_ape_pour_jointure] + [col for col in nomenclature_df.columns if col != nom_cible_col_ape_pour_jointure]
        nomenclature_df = nomenclature_df[cols]
    print(f"Colonnes de nomenclature_sitar après préparation : {nomenclature_df.columns.tolist()}")
    return nomenclature_df

# --- Processus Principal de Sirénisation ---
def processus_principal_sirenisation():
    """
    Fonction principale orchestrant le processus de sirénisation.
    """
    # --- Configuration : Définissez vos chemins de fichiers ici ---
    chemin_stock_ul = "C://Users//msamb//Documents//StockUniteLegale_utf8.csv"
    chemin_base_a_sireniser = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021_complet.csv"
    nom_col_siren_dans_base = "SIREN_DECLARANT" 
    chemin_nomenclature_sittar = "C://Users//msamb//Documents//Nomenclature_secteur_sittar.xlsx"
    nom_original_col_ape_nomenclature = "Code APE Entreprise"
    # --- Fin de la Configuration ---

    try:
        # 1. Charger et préparer 'stock_ul_extract'
        stock_ul_extrait_df = charger_et_preparer_stock_ul(chemin_stock_ul)

        # 2. Charger 'base_a_sireniser'
        base_a_sireniser_df = charger_base_a_sireniser(chemin_base_a_sireniser, nom_colonne_siren=nom_col_siren_dans_base)

        # 3. Première jointure (sirénisation)
        print(f"\nJonction de 'base_a_sireniser' avec 'stock_ul_extrait' sur la colonne '{nom_col_siren_dans_base}'...")
        base_finale_def_sirenise = pd.merge(
            left=base_a_sireniser_df,
            right=stock_ul_extrait_df,
            left_on=nom_col_siren_dans_base,
            right_on='SIREN_DECLARANT',
            how='left'
        )
        print(f"Nombre de lignes après la première jonction : {len(base_finale_def_sirenise)}")

        # 4. Charger et préparer 'nomenclature_sitar'
        nomenclature_sitar_df = charger_et_preparer_nomenclature(
            chemin_nomenclature_sittar, 
            nom_original_col_ape=nom_original_col_ape_nomenclature
        )

        # 5. Seconde jointure avec la nomenclature
        cle_jointure_ape_base = 'activitePrincipaleUniteLegale'
        cle_jointure_ape_nomenclature = 'activiteprincipaleunitelegale'

        if cle_jointure_ape_base not in base_finale_def_sirenise.columns:
            print(f"ERREUR : Colonne '{cle_jointure_ape_base}' absente de la base après première jointure.")
            return
        if cle_jointure_ape_nomenclature not in nomenclature_sitar_df.columns:
            print(f"ERREUR : Colonne '{cle_jointure_ape_nomenclature}' absente de la nomenclature préparée.")
            return

        # Harmonisation des clés de jointure APE pour robustesse
        base_finale_def_sirenise[cle_jointure_ape_base + '_pour_jointure'] = base_finale_def_sirenise[cle_jointure_ape_base].astype(str).str.strip().str.lower()
        nomenclature_sitar_df[cle_jointure_ape_nomenclature + '_pour_jointure'] = nomenclature_sitar_df[cle_jointure_ape_nomenclature].astype(str).str.strip().str.lower()
        
        print(f"\nJonction avec 'nomenclature_sitar' sur la colonne APE (harmonisées)...")
        base_finale_df = pd.merge(
            base_finale_def_sirenise,
            nomenclature_sitar_df,
            left_on=cle_jointure_ape_base + '_pour_jointure',
            right_on=cle_jointure_ape_nomenclature + '_pour_jointure',
            how='left',
            suffixes=('_base', '_nomenclature')
        )
        
        # Optionnel: supprimer les colonnes temporaires de jointure APE
        base_finale_df.drop(columns=[cle_jointure_ape_base + '_pour_jointure', cle_jointure_ape_nomenclature + '_pour_jointure'], inplace=True, errors='ignore')
        
        print(f"Nombre de lignes après la seconde jonction : {len(base_finale_df)}")
        print(f"Colonnes finales : {base_finale_df.columns.tolist()}")

        print("\n✓ Processus de sirénisation terminé.")
        print("\nAperçu du DataFrame final (base_finale_df) :")
        print(base_finale_df.head())
        base_finale_df.to_csv("C://Users//msamb//Documents//base_finale_sirenise.csv", sep=';', encoding='utf-8-sig', index=False)


    except FileNotFoundError:
        print("ERREUR CRITIQUE : Un ou plusieurs fichiers d'entrée sont introuvables. Vérifiez les chemins dans la 'Configuration'.")
    except KeyError as e:
        print(f"ERREUR DE COLONNE (KeyError) : {e}. Vérifiez les noms de colonnes dans vos fichiers et la configuration.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {e}")
        import traceback
        traceback.print_exc()

# --- Point d'Entrée du Script ---
if __name__ == "__main__":
    # Ce code est exécuté lorsque vous lancez le script directement.
    
    # Exécuter le processus principal de sirénisation.
    # Assurez-vous d'avoir bien configuré les chemins et noms de colonnes
    # dans la section 'Configuration' de la fonction 'processus_principal_sirenisation'.
    processus_principal_sirenisation()

Chargement et préparation de 'C://Users//msamb//Documents//StockUniteLegale_utf8.csv'...
Colonnes de stock_ul_extract après préparation : ['SIREN_DECLARANT', 'dateCreationUniteLegale', 'trancheEffectifsUniteLegale', 'categorieEntreprise', 'denominationUniteLegale', 'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale', 'denominationUsuelle3UniteLegale', 'activitePrincipaleUniteLegale', 'nicSiegeUniteLegale', 'nic', 'siret']
Chargement de la base à siréniser depuis 'M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021_complet.csv'...
Colonnes de base_a_sireniser : ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'DESIGNATION', 'L1_DOTATION_AMORT_IMMO', 'L2_DOTATION_AMORT_SINISTR', 'L3_DEPENSES_PERSONNEL_CHERCHEURS', 'L4_REMUNERATION_INVENTEURS', 'L5_DEPENSES_JEUNES_DOCTEURS', 'L6_AUTRES_DEP_FONCT_DECLARE', 'L6_AUTRES_DEP_FONCT_CALCULE', 'L6_ECART', 'L7_TOTAL_DEP_FONCT_DECLARE', 'L7_TOTAL_DEP_FONCT_CALCULE', 'L7_ECART', 'L8_FRAIS_BREVETS_COV', 

In [9]:
import pandas as pd

# Charger la base finale sirénisée
base_finale = pd.read_excel('C://Users//msamb//Documents//base_finale_sirenise.xlsx')

# Chercher les SIREN déclarants en doublon
doublons_declarant = base_finale[base_finale.duplicated(subset=['SIREN_DECLARANT'], keep=False)]

print(f"Nombre de SIREN déclarants en doublon : {doublons_declarant['SIREN_DECLARANT'].nunique()}")
print("Exemples de doublons :")
display(doublons_declarant.sort_values('SIREN_DECLARANT').head(10))
doublons_declarant.to_csv("C://Users//msamb//Documents//doublons_siren_declarant.csv", sep=';', encoding='utf-8-sig', index=False)

Nombre de SIREN déclarants en doublon : 79
Exemples de doublons :


,SIREN_DECLARANT,SIREN_DEPOSANT,DESIGNATION,L1_DOTATION_AMORT_IMMO,L2_DOTATION_AMORT_SINISTR,L3_DEPENSES_PERSONNEL_CHERCHEURS,L4_REMUNERATION_INVENTEURS,L5_DEPENSES_JEUNES_DOCTEURS,L6_AUTRES_DEP_FONCT_DECLARE,L6_AUTRES_DEP_FONCT_CALCULE,...,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,activitePrincipaleUniteLegale,nicSiegeUniteLegale,nic,siret,activiteprincipaleunitelegale,secteur_dactivit,libell_code_ape_entreprise
1084,324284611,324284611,SAS,1147,0,0,88491,0,38911,38911.38,...,NaN,NaN,NaN,62.01Z,70.0,70.0,3.242846e+13,62.01Z,CONSEIL ET ASSISTANCE EN INFORMATIQUE,Programmation informatique
25116,324284611,892447798,VIDEOR,1147,0,0,88491,0,38911,38911.38,...,NaN,NaN,NaN,62.01Z,70.0,70.0,3.242846e+13,62.01Z,CONSEIL ET ASSISTANCE EN INFORMATIQUE,Programmation informatique
1699,334380409,334380409,NaN,0,0,0,0,0,0,0.00,...,NaN,NaN,NaN,58.29C,30.0,30.0,3.343804e+13,58.29C,CONSEIL ET ASSISTANCE EN INFORMATIQUE,Edition de logiciels applicatifs
1700,334380409,334380409,NaN,0,0,0,0,0,0,0.00,...,NaN,NaN,NaN,58.29C,30.0,30.0,3.343804e+13,58.29C,CONSEIL ET ASSISTANCE EN INFORMATIQUE,Edition de logiciels applicatifs
1698,334380409,334380409,SASU,0,0,0,0,0,0,0.00,...,NaN,NaN,NaN,58.29C,30.0,30.0,3.343804e+13,58.29C,CONSEIL ET ASSISTANCE EN INFORMATIQUE,Edition de logiciels applicatifs
2098,342174018,334381746,GEOFIT,42781,0,197507,0,0,117013,117013.76,...,NaN,NaN,NaN,71.12A,354.0,354.0,3.421740e+13,71.12A,SERVICES D'ARCHITECTURE ET D'INGIENERIE,Activité des géomètres
2099,342174018,380853507,GEOFIT GROUP,38988,0,779507,0,26707,391136,391136.01,...,NaN,NaN,NaN,71.12A,354.0,354.0,3.421740e+13,71.12A,SERVICES D'ARCHITECTURE ET D'INGIENERIE,Activité des géomètres
3595,387558315,300349636,KRAMPOUZ,19878,0,135791,0,0,73299,73298.63,...,NaN,NaN,NaN,27.51Z,33.0,33.0,3.875583e+13,27.51Z,INDUSTRIE DES EQUIPEMENTS DU FOYER,Fabrication d'appareils électroménagers
3596,387558315,387558315,SAS,6912,0,76005,0,0,37866,37866.15,...,NaN,NaN,NaN,27.51Z,33.0,33.0,3.875583e+13,27.51Z,INDUSTRIE DES EQUIPEMENTS DU FOYER,Fabrication d'appareils électroménagers
4115,393819636,899429377,NOMOTECH NS,0,0,101219,0,0,43524,43524.17,...,NaN,NaN,NaN,61.90Z,217.0,217.0,3.938196e+13,61.90Z,SERVICES DE TELECOMMUNICATIONS,Autres activités de télécommunication


In [12]:
import pandas as pd

# Lire le fichier Excel
df = pd.read_csv("C://Users//msamb//Documents//base_finale_sirenise.csv", sep=';', encoding='utf-8-sig', low_memory=False)

# Définir les colonnes clés pour identifier les doublons
colonnes_clés = ['SIREN_DECLARANT', 'cretot_gen','mere_final']

# Supprimer les doublons en gardant la première occurrence
df_sans_doublons = df.drop_duplicates(subset=colonnes_clés, keep='first')

# Ou si vous voulez garder la dernière occurrence :
# df_sans_doublons = df.drop_duplicates(subset=colonnes_clés, keep='last')

# Sauvegarder le résultat dans un nouveau fichier
df_sans_doublons.to_excel('C://Users//msamb//Documents//base_finale_sirenise.xlsx', index=False)

# Afficher quelques statistiques
print(f"Nombre de lignes original : {len(df)}")
print(f"Nombre de lignes après suppression des doublons : {len(df_sans_doublons)}")
print(f"Nombre de doublons supprimés : {len(df) - len(df_sans_doublons)}")

Nombre de lignes original : 28309
Nombre de lignes après suppression des doublons : 28299
Nombre de doublons supprimés : 10


In [111]:
import pandas as pd
import re

# 1) Chemins
input_path  = "C://Users//msamb//Documents//base_finale_sirenise.csv"
output_path = "C://Users//msamb//Documents//base_finale_sirenise_transforme.csv"

# 2) Chargement du CSV
df = pd.read_csv(input_path, sep=";")

# 3) Mapping des colonnes “statiques” à conserver et renommer
static_map = {
    "SIREN_DECLARANT": "siren",
    "SIREN_DEPOSANT":  "siren_g",
    "DESIGNATION":     "nom",
}

# 4) Ordre voulu des colonnes “génériques” (déjà recalculées)
generic_order = [
    "crerd_gen","crerd_genom",
    "crecoll_gen","crecoll_genom",
    "creinno_gen","creinno_genom",
    "cretot_gen","cretot_genom",
    "cirrech_benef","cic_benef",
    "cii_benef","cirtot_benef",
]

# 5) Détection des colonnes à garder
# — statiques réellement présentes
statics = [c for c in static_map if c in df.columns]

# — génériques dans l’ordre souhaité, mais uniquement celles qui existent
generics = [c for c in generic_order if c in df.columns]

# — toutes les colonnes recalculées (suffixe _CALCULE)
calcules = [c for c in df.columns if c.upper().endswith("_CALCULE")]

# (Optionnel) affichage pour debug
print("Statique :", statics)
print("Génériques :", generics)
print("Calculées :", calcules)

# 6) Sélection de la sous-partie du DataFrame
df2 = df[statics + generics + calcules].copy()

# 7) Construction du mapping de renommage complet
rename_map = {}

# — statiques
for c in statics:
    rename_map[c] = static_map[c]

# — génériques → minuscules
for c in generics:
    rename_map[c] = c.lower()

# — dynamiques L*…_CALCULE → _<num><a/b?>
# — autres *_CALCULE hors L* → nom minuscules sans "_calcule"
for c in calcules:
    m = re.match(r"^L(\d+)([AB])?_.*_CALCULE$", c, flags=re.IGNORECASE)
    if m:
        num    = m.group(1)
        letter = m.group(2).lower() if m.group(2) else ""
        rename_map[c] = f"_{num}{letter}"
    else:
        rename_map[c] = c.lower().replace("_calcule", "")

df2 = df2.rename(columns=rename_map)

# 8) Réordonnancement final des colonnes
# — statiques renommées
final_statics = [static_map[c] for c in statics]

# — génériques renommés
final_generics = [c.lower() for c in generics]

# — dynamiques L* triées par numéro et lettre
dyn_tuples = []
for c in calcules:
    if re.match(r"^L\d+([AB])?_.*_CALCULE$", c, flags=re.IGNORECASE):
        m = re.match(r"^L(\d+)([AB])?_.*_CALCULE$", c, flags=re.IGNORECASE)
        num    = int(m.group(1))
        letter = m.group(2).lower() if m.group(2) else ""
        dyn_tuples.append((num, letter, rename_map[c]))

final_dyn = [name for _, _, name in sorted(dyn_tuples, key=lambda x: (x[0], x[1]))]

# — autres *_CALCULE hors L* placées après
other_calcules = [
    rename_map[c] for c in calcules
    if not re.match(r"^L\d+([AB])?_.*_CALCULE$", c, flags=re.IGNORECASE)
]

# — ordre complet
final_columns = final_statics + final_generics + final_dyn + other_calcules
df2 = df2[final_columns]

# 9) Export vers CSV
df2.to_csv(output_path, index=False, sep=";")
print(f"Transformé écrit dans : {output_path}")


C:\Users\msamb\AppData\Local\Temp\ipykernel_14148\3955505606.py:9: DtypeWarning: Columns (204) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, sep=";")


Statique : ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'DESIGNATION']
Génériques : ['crerd_gen', 'crerd_genom', 'crecoll_gen', 'crecoll_genom', 'creinno_gen', 'creinno_genom', 'cretot_gen', 'cretot_genom', 'cirrech_benef', 'cic_benef', 'cii_benef', 'cirtot_benef']
Calculées : ['L6_AUTRES_DEP_FONCT_CALCULE', 'L7_TOTAL_DEP_FONCT_CALCULE', 'L14_TOTAL_DEPENSES_INTERNES_CALCULE', 'L17_TOTAL_DEP_EXTERNALISEES_CALCULE', 'L18_PLAFOND_GLOBAL_CALCULE', 'L19_PLAFOND_ORG_LIES_CALCULE', 'L20_PLAFOND_ORG_NON_LIES_CALCULE', 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE', 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE', 'L26A_MONTANT_NET_DEPENSES_CALCULE', 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE', 'L29_TOTAL_DEPENSES_COLLECTION_CALCULE', 'L33A_MONTANT_NET_COLLECTION_CALCULE', 'L33B_MONTANT_NET_COLLECTION_DOM_CALCULE', 'L70_TOTAL_DEPENSES_INNOVATION_CALCULE', 'L71_DEPENSES_INNOVATION_PLAFONNEES_CALCULE', 'L76A_MONTANT_NET_INNOVATION_CALCULE', 'L83_DEPENSES_RC_PLAFONNEES_CALCULE', 'L86_MONTANT_NET_RC_CALCULE', 'cretot_gen_calc